# Statisics and Anomalies for the Period of Record: wy2001-wy2019

**Files generated from this notebook were used as inputs to the MEDS Snow Today Shiny App**

In [11]:
import netCDF4
import xarray
import numpy as np
from osgeo import gdal
import rioxarray 
import warnings
from numpy import array

# File Paths

In [12]:
# snow cover file paths
wy2001_sf = '../data/snow_fraction_data/Sierra2001.h5'
wy2002_sf = '../data/snow_fraction_data/Sierra2002.h5'
wy2003_sf = '../data/snow_fraction_data/Sierra2003.h5'
wy2004_sf = '../data/snow_fraction_data/Sierra2004.h5' # leap year
wy2005_sf = '../data/snow_fraction_data/Sierra2005.h5'
wy2006_sf = '../data/snow_fraction_data/Sierra2006.h5'
wy2007_sf = '../data/snow_fraction_data/Sierra2007.h5'
wy2008_sf = '../data/snow_fraction_data/Sierra2008.h5' # leap year
wy2009_sf = '../data/snow_fraction_data/Sierra2009.h5'
wy2010_sf = '../data/snow_fraction_data/Sierra2010.h5'
wy2011_sf = '../data/snow_fraction_data/Sierra2011.h5'
wy2012_sf = '../data/snow_fraction_data/Sierra2012.h5' # leap year
wy2013_sf = '../data/snow_fraction_data/Sierra2013.h5'
wy2014_sf = '../data/snow_fraction_data/Sierra2014.h5'
wy2015_sf = '../data/snow_fraction_data/Sierra2015.h5'
wy2016_sf = '../data/snow_fraction_data/Sierra2016.h5' # leap year
wy2017_sf = '../data/snow_fraction_data/Sierra2017.h5'
wy2018_sf = '../data/snow_fraction_data/Sierra2018.h5'
wy2019_sf = '../data/snow_fraction_data/Sierra2019.h5'

In [13]:
wy2001_albedo = '../data/albedo_data/SierraAlbedo2001.h5'
wy2002_albedo = '../data/albedo_data/SierraAlbedo2002.h5'
wy2003_albedo = '../data/albedo_data/SierraAlbedo2003.h5'
wy2004_albedo = '../data/albedo_data/SierraAlbedo2004.h5'
wy2005_albedo = '../data/albedo_data/SierraAlbedo2005.h5'
wy2006_albedo = '../data/albedo_data/SierraAlbedo2006.h5'
wy2007_albedo = '../data/albedo_data/SierraAlbedo2007.h5'
wy2008_albedo = '../data/albedo_data/SierraAlbedo2008.h5'
wy2009_albedo = '../data/albedo_data/SierraAlbedo2009.h5'
wy2010_albedo = '../data/albedo_data/SierraAlbedo2010.h5'
wy2011_albedo = '../data/albedo_data/SierraAlbedo2011.h5'
wy2012_albedo = '../data/albedo_data/SierraAlbedo2012.h5'
wy2013_albedo = '../data/albedo_data/SierraAlbedo2013.h5'
wy2014_albedo = '../data/albedo_data/SierraAlbedo2014.h5'
wy2015_albedo = '../data/albedo_data/SierraAlbedo2015.h5'
wy2016_albedo = '../data/albedo_data/SierraAlbedo2016.h5'
wy2017_albedo = '../data/albedo_data/SierraAlbedo2017.h5'
wy2018_albedo = '../data/albedo_data/SierraAlbedo2018.h5'
wy2019_albedo = '../data/albedo_data/SierraAlbedo2019.h5'

# Global Variables

In [14]:
january_start_index = 92
january_end_index = 123
february_start_index = 123
february_end_index = 151
march_start_index = 151
march_end_index = 182
april_start_index = 182
april_end_index = 212
may_start_index = 212 
may_end_index = 243
june_start_index = 243 
june_end_index = 263
july_start_index = 273 
july_end_index = 304
august_start_index = 304
august_end_index = 335
september_start_index = 335
september_end_index = 365
october_start_index = 0
october_end_index =31
november_start_index = 31
november_end_index = 61
december_start_index = 61
december_end_index = 92
february_leapyear_start_index = 123
february_leapyear_end_index = 152
march_leapyear_start_index = 152
march_leapyear_end_index = 183
april_leapyear_start_index = 183
april_leapyear_end_index = 213
may_leapyear_start_index = 213
may_leapyear_end_index = 244
june_leapyear_start_index = 244
june_leapyear_end_index = 274
july_leapyear_start_index = 274
july_leapyear_end_index = 305
august_leapyear_start_index = 305
august_leapyear_end_index = 336
september_leapyear_start_index = 336
september_leapyear_end_index = 366

# Geotiff Transformation Functions
## Projection Info

In [15]:
# x dimension of array
X_width = 1334
# y dimension of array
Y_height = 1841
# Projection data of sample GeoTiff
Projection = 'PROJCS["Albers Conical Equal Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PARAMETER["standard_parallel_2",40.5],PARAMETER["false_easting",0],PARAMETER["false_northing",-4000000],UNIT["meters",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'
# transformation data of array
ReferencingMatrix = [-285750.0, 500.0, 0.0, 500250.0, 0.0, -500.0]

In [16]:
def CreateGeoTiffStack(name, array, geo_transform, projection):
    
    driver = gdal.GetDriverByName('GTiff')

    DataSet = driver.Create(name, array.shape[2], array.shape[1], array.shape[0], gdal.GDT_Float32)
    DataSet.SetGeoTransform(geo_transform)
    DataSet.SetProjection(projection)
    for i, image in enumerate(array, 1):
        DataSet.GetRasterBand(i).WriteArray( image )
    DataSet.FlushCache()
    return name

# Snow Cover Percent

 
## Annual
### Mean

In [17]:
def AnnualMeanSnowCoverPercent(filepath):
    sierra_snow_cover = rioxarray.open_rasterio(filepath)
    snow_cover_xarray = sierra_snow_cover['Grid_MODIS_GRID_500m_snow_fraction']
    snow_cover_xarray = snow_cover_xarray.where(snow_cover_xarray != 255)
    annual_mean_snow_cover_percent = snow_cover_xarray.isel(band=slice(0, 364)).mean(dim="band") 
    annual_mean_snow_cover_percent_numpy = annual_mean_snow_cover_percent.to_numpy()
    annual_mean_snow_cover_percent_numpy = np.transpose(annual_mean_snow_cover_percent_numpy)
    #annual_mean_snow_cover_percent_numpy[annual_mean_snow_cover_percent_numpy == 0] = np.nan
    
    return(annual_mean_snow_cover_percent_numpy)

In [18]:
warnings.filterwarnings('ignore')

# these are numpy arrays
annual_mean_snow_cover_percent_2001 = AnnualMeanSnowCoverPercent(wy2001_sf)
annual_mean_snow_cover_percent_2002 = AnnualMeanSnowCoverPercent(wy2002_sf)
annual_mean_snow_cover_percent_2003 = AnnualMeanSnowCoverPercent(wy2003_sf)
annual_mean_snow_cover_percent_2004 = AnnualMeanSnowCoverPercent(wy2004_sf)
annual_mean_snow_cover_percent_2005 = AnnualMeanSnowCoverPercent(wy2005_sf)
annual_mean_snow_cover_percent_2006 = AnnualMeanSnowCoverPercent(wy2006_sf)
annual_mean_snow_cover_percent_2007 = AnnualMeanSnowCoverPercent(wy2007_sf)
annual_mean_snow_cover_percent_2008 = AnnualMeanSnowCoverPercent(wy2008_sf)
annual_mean_snow_cover_percent_2009 = AnnualMeanSnowCoverPercent(wy2009_sf)
annual_mean_snow_cover_percent_2010 = AnnualMeanSnowCoverPercent(wy2010_sf)
annual_mean_snow_cover_percent_2011 = AnnualMeanSnowCoverPercent(wy2011_sf)
annual_mean_snow_cover_percent_2012 = AnnualMeanSnowCoverPercent(wy2012_sf)
annual_mean_snow_cover_percent_2013 = AnnualMeanSnowCoverPercent(wy2013_sf)
annual_mean_snow_cover_percent_2014 = AnnualMeanSnowCoverPercent(wy2014_sf)
annual_mean_snow_cover_percent_2015 = AnnualMeanSnowCoverPercent(wy2015_sf)
annual_mean_snow_cover_percent_2016 = AnnualMeanSnowCoverPercent(wy2016_sf)
annual_mean_snow_cover_percent_2017 = AnnualMeanSnowCoverPercent(wy2017_sf)
annual_mean_snow_cover_percent_2018 = AnnualMeanSnowCoverPercent(wy2018_sf)
annual_mean_snow_cover_percent_2019 = AnnualMeanSnowCoverPercent(wy2019_sf)

In [19]:
mean_annual_snow_cover_percent = (annual_mean_snow_cover_percent_2001 + annual_mean_snow_cover_percent_2002 + annual_mean_snow_cover_percent_2003 +
                                  annual_mean_snow_cover_percent_2004 + annual_mean_snow_cover_percent_2005 + annual_mean_snow_cover_percent_2006 +
                                  annual_mean_snow_cover_percent_2007 + annual_mean_snow_cover_percent_2008 + annual_mean_snow_cover_percent_2009 +
                                  annual_mean_snow_cover_percent_2010 + annual_mean_snow_cover_percent_2011 + annual_mean_snow_cover_percent_2012 +
                                  annual_mean_snow_cover_percent_2013 + annual_mean_snow_cover_percent_2014 + annual_mean_snow_cover_percent_2015 +
                                  annual_mean_snow_cover_percent_2016 + annual_mean_snow_cover_percent_2017 + annual_mean_snow_cover_percent_2018 +
                                  annual_mean_snow_cover_percent_2019) / 19

'stacked_tiffs/january_mean_albedo_stack.tif'

In [20]:
# stack of annual mean snow cover, including average for period of record and average for each year
annual_snow_cover_percent_stack = array([annual_mean_snow_cover_percent_2001, annual_mean_snow_cover_percent_2002,
                                        annual_mean_snow_cover_percent_2003, annual_mean_snow_cover_percent_2004, annual_mean_snow_cover_percent_2005,
                                        annual_mean_snow_cover_percent_2006, annual_mean_snow_cover_percent_2007, annual_mean_snow_cover_percent_2008, 
                                        annual_mean_snow_cover_percent_2009, annual_mean_snow_cover_percent_2010, annual_mean_snow_cover_percent_2011, 
                                        annual_mean_snow_cover_percent_2012, annual_mean_snow_cover_percent_2013, annual_mean_snow_cover_percent_2014, 
                                        annual_mean_snow_cover_percent_2015, annual_mean_snow_cover_percent_2016, annual_mean_snow_cover_percent_2017, 
                                        annual_mean_snow_cover_percent_2018, annual_mean_snow_cover_percent_2019, mean_annual_snow_cover_percent])
annual_snow_cover_percent_stack[annual_snow_cover_percent_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/annual_snow_cover_percent_stack.tif', annual_snow_cover_percent_stack, ReferencingMatrix, Projection)

'stacked_tiffs/annual_snow_cover_percent_stack.tif'

In [237]:
np.nanmin(annual_snow_cover_percent_stack)

0.00014459224985540777

In [239]:
np.nanmax(annual_snow_cover_percent_stack)

94.81593406593407

In [21]:
#annual_snow_cover_percent_stack.shape

In [22]:
#type(annual_snow_cover_percent_stack)

### Anomaly

In [23]:
wy2001_scp_anomaly = annual_mean_snow_cover_percent_2001 - mean_annual_snow_cover_percent
wy2002_scp_anomaly = annual_mean_snow_cover_percent_2002 - mean_annual_snow_cover_percent
wy2003_scp_anomaly = annual_mean_snow_cover_percent_2003 - mean_annual_snow_cover_percent
wy2004_scp_anomaly = annual_mean_snow_cover_percent_2004 - mean_annual_snow_cover_percent
wy2005_scp_anomaly = annual_mean_snow_cover_percent_2005 - mean_annual_snow_cover_percent
wy2006_scp_anomaly = annual_mean_snow_cover_percent_2006 - mean_annual_snow_cover_percent
wy2007_scp_anomaly = annual_mean_snow_cover_percent_2007 - mean_annual_snow_cover_percent
wy2008_scp_anomaly = annual_mean_snow_cover_percent_2008 - mean_annual_snow_cover_percent
wy2009_scp_anomaly = annual_mean_snow_cover_percent_2009 - mean_annual_snow_cover_percent
wy2010_scp_anomaly = annual_mean_snow_cover_percent_2010 - mean_annual_snow_cover_percent
wy2011_scp_anomaly = annual_mean_snow_cover_percent_2011 - mean_annual_snow_cover_percent
wy2012_scp_anomaly = annual_mean_snow_cover_percent_2012 - mean_annual_snow_cover_percent
wy2013_scp_anomaly = annual_mean_snow_cover_percent_2013 - mean_annual_snow_cover_percent
wy2014_scp_anomaly = annual_mean_snow_cover_percent_2014 - mean_annual_snow_cover_percent
wy2015_scp_anomaly = annual_mean_snow_cover_percent_2015 - mean_annual_snow_cover_percent
wy2016_scp_anomaly = annual_mean_snow_cover_percent_2016 - mean_annual_snow_cover_percent
wy2017_scp_anomaly = annual_mean_snow_cover_percent_2017 - mean_annual_snow_cover_percent
wy2018_scp_anomaly = annual_mean_snow_cover_percent_2018 - mean_annual_snow_cover_percent
wy2019_scp_anomaly = annual_mean_snow_cover_percent_2019 - mean_annual_snow_cover_percent

annual_scp_anomaly_stack = array([wy2001_scp_anomaly, wy2002_scp_anomaly, wy2003_scp_anomaly, wy2004_scp_anomaly, wy2005_scp_anomaly,
                                 wy2006_scp_anomaly, wy2007_scp_anomaly, wy2008_scp_anomaly, wy2009_scp_anomaly, wy2010_scp_anomaly,
                                 wy2011_scp_anomaly, wy2012_scp_anomaly, wy2013_scp_anomaly, wy2014_scp_anomaly, wy2015_scp_anomaly,
                                 wy2016_scp_anomaly, wy2017_scp_anomaly, wy2018_scp_anomaly, wy2019_scp_anomaly])

annual_scp_anomaly_stack[annual_scp_anomaly_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/annual_scp_anomaly_stack.tif', annual_scp_anomaly_stack, ReferencingMatrix, Projection)


'stacked_tiffs/annual_scp_anomaly_stack.tif'

In [240]:
np.nanmin(annual_scp_anomaly_stack)

-46.09008097165992

In [241]:
np.nanmax(annual_scp_anomaly_stack)

32.752024291497975

In [24]:
#type(annual_mean_snow_cover_percent_2001)

In [25]:
#annual_mean_snow_cover_percent_2001.shape

In [26]:
#annual_mean_snow_cover_percent_2001

## Monthly
### Mean

In [27]:
def MonthlySnowCoverPercent(filepath, start_index, end_index):
    sierra_snow_cover = rioxarray.open_rasterio(filepath)
    snow_cover_xarray = sierra_snow_cover['Grid_MODIS_GRID_500m_snow_fraction']
    snow_cover_xarray = snow_cover_xarray.where(snow_cover_xarray != 255)
    monthly_mean_snow_cover_percent = snow_cover_xarray.isel(band=slice(start_index, end_index)).mean(dim="band")
    monthly_mean_snow_cover_percent_numpy = monthly_mean_snow_cover_percent.to_numpy()
    monthly_mean_snow_cover_percent_numpy = np.transpose(monthly_mean_snow_cover_percent_numpy)
    #monthly_mean_snow_cover_percent_numpy[monthly_mean_snow_cover_percent_numpy == 0] = np.nan
    
    return(monthly_mean_snow_cover_percent_numpy)

In [28]:
warnings.filterwarnings('ignore')

# wy2001
january_scp_wy2001 = MonthlySnowCoverPercent(wy2001_sf, january_start_index, january_end_index)
february_scp_wy2001 = MonthlySnowCoverPercent(wy2001_sf, february_start_index, february_end_index)
march_scp_wy2001 = MonthlySnowCoverPercent(wy2001_sf, march_start_index, march_end_index)
april_scp_wy2001 = MonthlySnowCoverPercent(wy2001_sf, april_start_index, april_end_index)
may_scp_wy2001 = MonthlySnowCoverPercent(wy2001_sf, may_start_index, may_end_index)
june_scp_wy2001 = MonthlySnowCoverPercent(wy2001_sf, june_start_index, june_end_index)
july_scp_wy2001 = MonthlySnowCoverPercent(wy2001_sf, july_start_index, july_end_index)
august_scp_wy2001 = MonthlySnowCoverPercent(wy2001_sf, august_start_index, august_end_index)
september_scp_wy2001 = MonthlySnowCoverPercent(wy2001_sf, september_start_index, september_end_index)
october_scp_wy2001 = MonthlySnowCoverPercent(wy2001_sf, october_start_index, october_end_index)
november_scp_wy2001 = MonthlySnowCoverPercent(wy2001_sf, november_start_index, november_end_index)
december_scp_wy2001 = MonthlySnowCoverPercent(wy2001_sf, december_start_index, december_end_index)

# wy2001_monthly_mean_scp_stack = array([january_scp_wy2001, february_scp_wy2001, march_scp_wy2001, april_scp_wy2001,
#                                       may_scp_wy2001, june_scp_wy2001, july_scp_wy2001, august_scp_wy2001, 
#                                        september_scp_wy2001, october_scp_wy2001, november_scp_wy2001, december_scp_wy2001])

# wy2001_monthly_mean_scp_stack[wy2001_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2001_monthly_mean_scp_stack.tif', wy2001_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [29]:
warnings.filterwarnings('ignore')

# wy2002
january_scp_wy2002 = MonthlySnowCoverPercent(wy2002_sf, january_start_index, january_end_index)
february_scp_wy2002 = MonthlySnowCoverPercent(wy2002_sf, february_start_index, february_end_index)
march_scp_wy2002 = MonthlySnowCoverPercent(wy2002_sf, march_start_index, march_end_index)
april_scp_wy2002 = MonthlySnowCoverPercent(wy2002_sf, april_start_index, april_end_index)
may_scp_wy2002 = MonthlySnowCoverPercent(wy2002_sf, may_start_index, may_end_index)
june_scp_wy2002 = MonthlySnowCoverPercent(wy2002_sf, june_start_index, june_end_index)
july_scp_wy2002 = MonthlySnowCoverPercent(wy2002_sf, july_start_index, july_end_index)
august_scp_wy2002 = MonthlySnowCoverPercent(wy2002_sf, august_start_index, august_end_index)
september_scp_wy2002 = MonthlySnowCoverPercent(wy2002_sf, september_start_index, september_end_index)
october_scp_wy2002 = MonthlySnowCoverPercent(wy2002_sf, october_start_index, october_end_index)
november_scp_wy2002 = MonthlySnowCoverPercent(wy2002_sf, november_start_index, november_end_index)
december_scp_wy2002 = MonthlySnowCoverPercent(wy2002_sf, december_start_index, december_end_index)

# wy2002_monthly_mean_scp_stack = array([january_scp_wy2002, february_scp_wy2002, march_scp_wy2002, april_scp_wy2002,
#                                       may_scp_wy2002, june_scp_wy2002, july_scp_wy2002, august_scp_wy2002, 
#                                        september_scp_wy2002, october_scp_wy2002, november_scp_wy2002, december_scp_wy2002])

# wy2002_monthly_mean_scp_stack[wy2002_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2002_monthly_mean_scp_stack.tif', wy2002_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [30]:
warnings.filterwarnings('ignore')

# wy2003
january_scp_wy2003 = MonthlySnowCoverPercent(wy2003_sf, january_start_index, january_end_index)
february_scp_wy2003 = MonthlySnowCoverPercent(wy2003_sf, february_start_index, february_end_index)
march_scp_wy2003 = MonthlySnowCoverPercent(wy2003_sf, march_start_index, march_end_index)
april_scp_wy2003 = MonthlySnowCoverPercent(wy2003_sf, april_start_index, april_end_index)
may_scp_wy2003 = MonthlySnowCoverPercent(wy2003_sf, may_start_index, may_end_index)
june_scp_wy2003 = MonthlySnowCoverPercent(wy2003_sf, june_start_index, june_end_index)
july_scp_wy2003 = MonthlySnowCoverPercent(wy2003_sf, july_start_index, july_end_index)
august_scp_wy2003 = MonthlySnowCoverPercent(wy2003_sf, august_start_index, august_end_index)
september_scp_wy2003 = MonthlySnowCoverPercent(wy2003_sf, september_start_index, september_end_index)
october_scp_wy2003 = MonthlySnowCoverPercent(wy2003_sf, october_start_index, october_end_index)
november_scp_wy2003 = MonthlySnowCoverPercent(wy2003_sf, november_start_index, november_end_index)
december_scp_wy2003 = MonthlySnowCoverPercent(wy2003_sf, december_start_index, december_end_index)

# wy2003_monthly_mean_scp_stack = array([january_scp_wy2003, february_scp_wy2003, march_scp_wy2003, april_scp_wy2003,
#                                       may_scp_wy2003, june_scp_wy2003, july_scp_wy2003, august_scp_wy2003, 
#                                        september_scp_wy2003, october_scp_wy2003, november_scp_wy2003, december_scp_wy2003])

# wy2003_monthly_mean_scp_stack[wy2003_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2003_monthly_mean_scp_stack.tif', wy2003_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [31]:
warnings.filterwarnings('ignore')

# wy2004
january_scp_wy2004 = MonthlySnowCoverPercent(wy2004_sf, january_start_index, january_end_index)
february_scp_wy2004 = MonthlySnowCoverPercent(wy2004_sf, february_leapyear_start_index, february_leapyear_end_index)
march_scp_wy2004 = MonthlySnowCoverPercent(wy2004_sf, march_leapyear_start_index, march_leapyear_end_index)
april_scp_wy2004 = MonthlySnowCoverPercent(wy2004_sf, april_leapyear_start_index, april_leapyear_end_index)
may_scp_wy2004 = MonthlySnowCoverPercent(wy2004_sf, may_leapyear_start_index, may_leapyear_end_index)
june_scp_wy2004 = MonthlySnowCoverPercent(wy2004_sf, june_leapyear_start_index, june_leapyear_end_index)
july_scp_wy2004 = MonthlySnowCoverPercent(wy2004_sf, july_leapyear_start_index, july_leapyear_end_index)
august_scp_wy2004 = MonthlySnowCoverPercent(wy2004_sf, august_leapyear_start_index, august_leapyear_end_index)
september_scp_wy2004 = MonthlySnowCoverPercent(wy2004_sf, september_leapyear_start_index, september_leapyear_end_index)
october_scp_wy2004 = MonthlySnowCoverPercent(wy2004_sf, october_start_index, october_end_index)
november_scp_wy2004 = MonthlySnowCoverPercent(wy2004_sf, november_start_index, november_end_index)
december_scp_wy2004 = MonthlySnowCoverPercent(wy2004_sf, december_start_index, december_end_index)

# wy2004_monthly_mean_scp_stack = array([january_scp_wy2004, february_scp_wy2004, march_scp_wy2004, april_scp_wy2004,
#                                       may_scp_wy2004, june_scp_wy2004, july_scp_wy2004, august_scp_wy2004, 
#                                        september_scp_wy2004, october_scp_wy2004, november_scp_wy2004, december_scp_wy2004])

# wy2004_monthly_mean_scp_stack[wy2004_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2004_monthly_mean_scp_stack.tif', wy2004_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [32]:
warnings.filterwarnings('ignore')

# wy2005
january_scp_wy2005 = MonthlySnowCoverPercent(wy2005_sf, january_start_index, january_end_index)
february_scp_wy2005 = MonthlySnowCoverPercent(wy2005_sf, february_start_index, february_end_index)
march_scp_wy2005 = MonthlySnowCoverPercent(wy2005_sf, march_start_index, march_end_index)
april_scp_wy2005 = MonthlySnowCoverPercent(wy2005_sf, april_start_index, april_end_index)
may_scp_wy2005 = MonthlySnowCoverPercent(wy2005_sf, may_start_index, may_end_index)
june_scp_wy2005 = MonthlySnowCoverPercent(wy2005_sf, june_start_index, june_end_index)
july_scp_wy2005 = MonthlySnowCoverPercent(wy2005_sf, july_start_index, july_end_index)
august_scp_wy2005 = MonthlySnowCoverPercent(wy2005_sf, august_start_index, august_end_index)
september_scp_wy2005 = MonthlySnowCoverPercent(wy2005_sf, september_start_index, september_end_index)
october_scp_wy2005 = MonthlySnowCoverPercent(wy2005_sf, october_start_index, october_end_index)
november_scp_wy2005 = MonthlySnowCoverPercent(wy2005_sf, november_start_index, november_end_index)
december_scp_wy2005 = MonthlySnowCoverPercent(wy2005_sf, december_start_index, december_end_index)

# wy2005_monthly_mean_scp_stack = array([january_scp_wy2005, february_scp_wy2005, march_scp_wy2005, april_scp_wy2005,
#                                       may_scp_wy2005, june_scp_wy2005, july_scp_wy2005, august_scp_wy2005, 
#                                        september_scp_wy2005, october_scp_wy2005, november_scp_wy2005, december_scp_wy2005])

# wy2005_monthly_mean_scp_stack[wy2005_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2005_monthly_mean_scp_stack.tif', wy2005_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [33]:
warnings.filterwarnings('ignore')

# wy2006
january_scp_wy2006 = MonthlySnowCoverPercent(wy2006_sf, january_start_index, january_end_index)
february_scp_wy2006 = MonthlySnowCoverPercent(wy2006_sf, february_start_index, february_end_index)
march_scp_wy2006 = MonthlySnowCoverPercent(wy2006_sf, march_start_index, march_end_index)
april_scp_wy2006 = MonthlySnowCoverPercent(wy2006_sf, april_start_index, april_end_index)
may_scp_wy2006 = MonthlySnowCoverPercent(wy2006_sf, may_start_index, may_end_index)
june_scp_wy2006 = MonthlySnowCoverPercent(wy2006_sf, june_start_index, june_end_index)
july_scp_wy2006 = MonthlySnowCoverPercent(wy2006_sf, july_start_index, july_end_index)
august_scp_wy2006 = MonthlySnowCoverPercent(wy2006_sf, august_start_index, august_end_index)
september_scp_wy2006 = MonthlySnowCoverPercent(wy2006_sf, september_start_index, september_end_index)
october_scp_wy2006 = MonthlySnowCoverPercent(wy2006_sf, october_start_index, october_end_index)
november_scp_wy2006 = MonthlySnowCoverPercent(wy2006_sf, november_start_index, november_end_index)
december_scp_wy2006 = MonthlySnowCoverPercent(wy2006_sf, december_start_index, december_end_index)

# wy2006_monthly_mean_scp_stack = array([january_scp_wy2006, february_scp_wy2006, march_scp_wy2006, april_scp_wy2006,
#                                       may_scp_wy2006, june_scp_wy2006, july_scp_wy2006, august_scp_wy2006, 
#                                        september_scp_wy2006, october_scp_wy2006, november_scp_wy2006, december_scp_wy2006])

# wy2006_monthly_mean_scp_stack[wy2006_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2006_monthly_mean_scp_stack.tif', wy2006_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [34]:
warnings.filterwarnings('ignore')

# wy2007
january_scp_wy2007 = MonthlySnowCoverPercent(wy2007_sf, january_start_index, january_end_index)
february_scp_wy2007 = MonthlySnowCoverPercent(wy2007_sf, february_start_index, february_end_index)
march_scp_wy2007 = MonthlySnowCoverPercent(wy2007_sf, march_start_index, march_end_index)
april_scp_wy2007 = MonthlySnowCoverPercent(wy2007_sf, april_start_index, april_end_index)
may_scp_wy2007 = MonthlySnowCoverPercent(wy2007_sf, may_start_index, may_end_index)
june_scp_wy2007 = MonthlySnowCoverPercent(wy2007_sf, june_start_index, june_end_index)
july_scp_wy2007 = MonthlySnowCoverPercent(wy2007_sf, july_start_index, july_end_index)
august_scp_wy2007 = MonthlySnowCoverPercent(wy2007_sf, august_start_index, august_end_index)
september_scp_wy2007 = MonthlySnowCoverPercent(wy2007_sf, september_start_index, september_end_index)
october_scp_wy2007 = MonthlySnowCoverPercent(wy2007_sf, october_start_index, october_end_index)
november_scp_wy2007 = MonthlySnowCoverPercent(wy2007_sf, november_start_index, november_end_index)
december_scp_wy2007 = MonthlySnowCoverPercent(wy2007_sf, december_start_index, december_end_index)

# wy2007_monthly_mean_scp_stack = array([january_scp_wy2007, february_scp_wy2007, march_scp_wy2007, april_scp_wy2007,
#                                       may_scp_wy2007, june_scp_wy2007, july_scp_wy2007, august_scp_wy2007, 
#                                        september_scp_wy2007, october_scp_wy2007, november_scp_wy2007, december_scp_wy2007])

# wy2007_monthly_mean_scp_stack[wy2007_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2007_monthly_mean_scp_stack.tif', wy2007_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [35]:
warnings.filterwarnings('ignore')

# wy2008
january_scp_wy2008 = MonthlySnowCoverPercent(wy2008_sf, january_start_index, january_end_index)
february_scp_wy2008 = MonthlySnowCoverPercent(wy2008_sf, february_leapyear_start_index, february_leapyear_end_index)
march_scp_wy2008 = MonthlySnowCoverPercent(wy2008_sf, march_leapyear_start_index, march_leapyear_end_index)
april_scp_wy2008 = MonthlySnowCoverPercent(wy2008_sf, april_leapyear_start_index, april_leapyear_end_index)
may_scp_wy2008 = MonthlySnowCoverPercent(wy2008_sf, may_leapyear_start_index, may_leapyear_end_index)
june_scp_wy2008 = MonthlySnowCoverPercent(wy2008_sf, june_leapyear_start_index, june_leapyear_end_index)
july_scp_wy2008 = MonthlySnowCoverPercent(wy2008_sf, july_leapyear_start_index, july_leapyear_end_index)
august_scp_wy2008 = MonthlySnowCoverPercent(wy2008_sf, august_leapyear_start_index, august_leapyear_end_index)
september_scp_wy2008 = MonthlySnowCoverPercent(wy2008_sf, september_leapyear_start_index, september_leapyear_end_index)
october_scp_wy2008 = MonthlySnowCoverPercent(wy2008_sf, october_start_index, october_end_index)
november_scp_wy2008 = MonthlySnowCoverPercent(wy2008_sf, november_start_index, november_end_index)
december_scp_wy2008 = MonthlySnowCoverPercent(wy2008_sf, december_start_index, december_end_index)

# wy2008_monthly_mean_scp_stack = array([january_scp_wy2008, february_scp_wy2008, march_scp_wy2008, april_scp_wy2008,
#                                       may_scp_wy2008, june_scp_wy2008, july_scp_wy2008, august_scp_wy2008, 
#                                        september_scp_wy2008, october_scp_wy2008, november_scp_wy2008, december_scp_wy2008])

# wy2008_monthly_mean_scp_stack[wy2008_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2008_monthly_mean_scp_stack.tif', wy2008_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [36]:
warnings.filterwarnings('ignore')

# wy2009
january_scp_wy2009 = MonthlySnowCoverPercent(wy2009_sf, january_start_index, january_end_index)
february_scp_wy2009 = MonthlySnowCoverPercent(wy2009_sf, february_start_index, february_end_index)
march_scp_wy2009 = MonthlySnowCoverPercent(wy2009_sf, march_start_index, march_end_index)
april_scp_wy2009 = MonthlySnowCoverPercent(wy2009_sf, april_start_index, april_end_index)
may_scp_wy2009 = MonthlySnowCoverPercent(wy2009_sf, may_start_index, may_end_index)
june_scp_wy2009 = MonthlySnowCoverPercent(wy2009_sf, june_start_index, june_end_index)
july_scp_wy2009 = MonthlySnowCoverPercent(wy2009_sf, july_start_index, july_end_index)
august_scp_wy2009 = MonthlySnowCoverPercent(wy2009_sf, august_start_index, august_end_index)
september_scp_wy2009 = MonthlySnowCoverPercent(wy2009_sf, september_start_index, september_end_index)
october_scp_wy2009 = MonthlySnowCoverPercent(wy2009_sf, october_start_index, october_end_index)
november_scp_wy2009 = MonthlySnowCoverPercent(wy2009_sf, november_start_index, november_end_index)
december_scp_wy2009 = MonthlySnowCoverPercent(wy2009_sf, december_start_index, december_end_index)

# wy2009_monthly_mean_scp_stack = array([january_scp_wy2009, february_scp_wy2009, march_scp_wy2009, april_scp_wy2009,
#                                       may_scp_wy2009, june_scp_wy2009, july_scp_wy2009, august_scp_wy2009, 
#                                        september_scp_wy2009, october_scp_wy2009, november_scp_wy2009, december_scp_wy2009])

# wy2009_monthly_mean_scp_stack[wy2009_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2009_monthly_mean_scp_stack.tif', wy2009_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [37]:
warnings.filterwarnings('ignore')

# wy2010
january_scp_wy2010 = MonthlySnowCoverPercent(wy2010_sf, january_start_index, january_end_index)
february_scp_wy2010 = MonthlySnowCoverPercent(wy2010_sf, february_start_index, february_end_index)
march_scp_wy2010 = MonthlySnowCoverPercent(wy2010_sf, march_start_index, march_end_index)
april_scp_wy2010 = MonthlySnowCoverPercent(wy2010_sf, april_start_index, april_end_index)
may_scp_wy2010 = MonthlySnowCoverPercent(wy2010_sf, may_start_index, may_end_index)
june_scp_wy2010 = MonthlySnowCoverPercent(wy2010_sf, june_start_index, june_end_index)
july_scp_wy2010 = MonthlySnowCoverPercent(wy2010_sf, july_start_index, july_end_index)
august_scp_wy2010 = MonthlySnowCoverPercent(wy2010_sf, august_start_index, august_end_index)
september_scp_wy2010 = MonthlySnowCoverPercent(wy2010_sf, september_start_index, september_end_index)
october_scp_wy2010 = MonthlySnowCoverPercent(wy2010_sf, october_start_index, october_end_index)
november_scp_wy2010 = MonthlySnowCoverPercent(wy2010_sf, november_start_index, november_end_index)
december_scp_wy2010 = MonthlySnowCoverPercent(wy2010_sf, december_start_index, december_end_index)

# wy2010_monthly_mean_scp_stack = array([january_scp_wy2010, february_scp_wy2010, march_scp_wy2010, april_scp_wy2010,
#                                       may_scp_wy2010, june_scp_wy2010, july_scp_wy2010, august_scp_wy2010, 
#                                        september_scp_wy2010, october_scp_wy2010, november_scp_wy2010, december_scp_wy2010])

# wy2010_monthly_mean_scp_stack[wy2010_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2010_monthly_mean_scp_stack.tif', wy2010_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [38]:
warnings.filterwarnings('ignore')

# wy2011
january_scp_wy2011 = MonthlySnowCoverPercent(wy2011_sf, january_start_index, january_end_index)
february_scp_wy2011 = MonthlySnowCoverPercent(wy2011_sf, february_start_index, february_end_index)
march_scp_wy2011 = MonthlySnowCoverPercent(wy2011_sf, march_start_index, march_end_index)
april_scp_wy2011 = MonthlySnowCoverPercent(wy2011_sf, april_start_index, april_end_index)
may_scp_wy2011 = MonthlySnowCoverPercent(wy2011_sf, may_start_index, may_end_index)
june_scp_wy2011 = MonthlySnowCoverPercent(wy2011_sf, june_start_index, june_end_index)
july_scp_wy2011 = MonthlySnowCoverPercent(wy2011_sf, july_start_index, july_end_index)
august_scp_wy2011 = MonthlySnowCoverPercent(wy2011_sf, august_start_index, august_end_index)
september_scp_wy2011 = MonthlySnowCoverPercent(wy2011_sf, september_start_index, september_end_index)
october_scp_wy2011 = MonthlySnowCoverPercent(wy2011_sf, october_start_index, october_end_index)
november_scp_wy2011 = MonthlySnowCoverPercent(wy2011_sf, november_start_index, november_end_index)
december_scp_wy2011 = MonthlySnowCoverPercent(wy2011_sf, december_start_index, december_end_index)

# wy2011_monthly_mean_scp_stack = array([january_scp_wy2011, february_scp_wy2011, march_scp_wy2011, april_scp_wy2011,
#                                       may_scp_wy2011, june_scp_wy2011, july_scp_wy2011, august_scp_wy2011, 
#                                        september_scp_wy2011, october_scp_wy2011, november_scp_wy2011, december_scp_wy2011])

# wy2011_monthly_mean_scp_stack[wy2011_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2011_monthly_mean_scp_stack.tif', wy2011_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [39]:
warnings.filterwarnings('ignore')

# wy2012
january_scp_wy2012 = MonthlySnowCoverPercent(wy2012_sf, january_start_index, january_end_index)
february_scp_wy2012 = MonthlySnowCoverPercent(wy2012_sf, february_leapyear_start_index, february_leapyear_end_index)
march_scp_wy2012 = MonthlySnowCoverPercent(wy2012_sf, march_leapyear_start_index, march_leapyear_end_index)
april_scp_wy2012 = MonthlySnowCoverPercent(wy2012_sf, april_leapyear_start_index, april_leapyear_end_index)
may_scp_wy2012 = MonthlySnowCoverPercent(wy2012_sf, may_leapyear_start_index, may_leapyear_end_index)
june_scp_wy2012 = MonthlySnowCoverPercent(wy2012_sf, june_leapyear_start_index, june_leapyear_end_index)
july_scp_wy2012 = MonthlySnowCoverPercent(wy2012_sf, july_leapyear_start_index, july_leapyear_end_index)
august_scp_wy2012 = MonthlySnowCoverPercent(wy2012_sf, august_leapyear_start_index, august_leapyear_end_index)
september_scp_wy2012 = MonthlySnowCoverPercent(wy2012_sf, september_leapyear_start_index, september_leapyear_end_index)
october_scp_wy2012 = MonthlySnowCoverPercent(wy2012_sf, october_start_index, october_end_index)
november_scp_wy2012 = MonthlySnowCoverPercent(wy2012_sf, november_start_index, november_end_index)
december_scp_wy2012 = MonthlySnowCoverPercent(wy2012_sf, december_start_index, december_end_index)

# wy2012_monthly_mean_scp_stack = array([january_scp_wy2012, february_scp_wy2012, march_scp_wy2012, april_scp_wy2012,
#                                       may_scp_wy2012, june_scp_wy2012, july_scp_wy2012, august_scp_wy2012, 
#                                        september_scp_wy2012, october_scp_wy2012, november_scp_wy2012, december_scp_wy2012])

# wy2012_monthly_mean_scp_stack[wy2012_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2012_monthly_mean_scp_stack.tif', wy2012_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [40]:
warnings.filterwarnings('ignore')

# wy2013
january_scp_wy2013 = MonthlySnowCoverPercent(wy2013_sf, january_start_index, january_end_index)
february_scp_wy2013 = MonthlySnowCoverPercent(wy2013_sf, february_start_index, february_end_index)
march_scp_wy2013 = MonthlySnowCoverPercent(wy2013_sf, march_start_index, march_end_index)
april_scp_wy2013 = MonthlySnowCoverPercent(wy2013_sf, april_start_index, april_end_index)
may_scp_wy2013 = MonthlySnowCoverPercent(wy2013_sf, may_start_index, may_end_index)
june_scp_wy2013 = MonthlySnowCoverPercent(wy2013_sf, june_start_index, june_end_index)
july_scp_wy2013 = MonthlySnowCoverPercent(wy2013_sf, july_start_index, july_end_index)
august_scp_wy2013 = MonthlySnowCoverPercent(wy2013_sf, august_start_index, august_end_index)
september_scp_wy2013 = MonthlySnowCoverPercent(wy2013_sf, september_start_index, september_end_index)
october_scp_wy2013 = MonthlySnowCoverPercent(wy2013_sf, october_start_index, october_end_index)
november_scp_wy2013 = MonthlySnowCoverPercent(wy2013_sf, november_start_index, november_end_index)
december_scp_wy2013 = MonthlySnowCoverPercent(wy2013_sf, december_start_index, december_end_index)

# wy2013_monthly_mean_scp_stack = array([january_scp_wy2013, february_scp_wy2013, march_scp_wy2013, april_scp_wy2013,
#                                       may_scp_wy2013, june_scp_wy2013, july_scp_wy2013, august_scp_wy2013, 
#                                        september_scp_wy2013, october_scp_wy2013, november_scp_wy2013, december_scp_wy2013])

# wy2013_monthly_mean_scp_stack[wy2013_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2013_monthly_mean_scp_stack.tif', wy2013_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [41]:
warnings.filterwarnings('ignore')

# wy2014
january_scp_wy2014 = MonthlySnowCoverPercent(wy2014_sf, january_start_index, january_end_index)
february_scp_wy2014 = MonthlySnowCoverPercent(wy2014_sf, february_start_index, february_end_index)
march_scp_wy2014 = MonthlySnowCoverPercent(wy2014_sf, march_start_index, march_end_index)
april_scp_wy2014 = MonthlySnowCoverPercent(wy2014_sf, april_start_index, april_end_index)
may_scp_wy2014 = MonthlySnowCoverPercent(wy2014_sf, may_start_index, may_end_index)
june_scp_wy2014 = MonthlySnowCoverPercent(wy2014_sf, june_start_index, june_end_index)
july_scp_wy2014 = MonthlySnowCoverPercent(wy2014_sf, july_start_index, july_end_index)
august_scp_wy2014 = MonthlySnowCoverPercent(wy2014_sf, august_start_index, august_end_index)
september_scp_wy2014 = MonthlySnowCoverPercent(wy2014_sf, september_start_index, september_end_index)
october_scp_wy2014 = MonthlySnowCoverPercent(wy2014_sf, october_start_index, october_end_index)
november_scp_wy2014 = MonthlySnowCoverPercent(wy2014_sf, november_start_index, november_end_index)
december_scp_wy2014 = MonthlySnowCoverPercent(wy2014_sf, december_start_index, december_end_index)

# wy2014_monthly_mean_scp_stack = array([january_scp_wy2014, february_scp_wy2014, march_scp_wy2014, april_scp_wy2014,
#                                       may_scp_wy2014, june_scp_wy2014, july_scp_wy2014, august_scp_wy2014, 
#                                        september_scp_wy2014, october_scp_wy2014, november_scp_wy2014, december_scp_wy2014])

# wy2014_monthly_mean_scp_stack[wy2014_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2014_monthly_mean_scp_stack.tif', wy2014_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [42]:
warnings.filterwarnings('ignore')

# wy2015
january_scp_wy2015 = MonthlySnowCoverPercent(wy2015_sf, january_start_index, january_end_index)
february_scp_wy2015 = MonthlySnowCoverPercent(wy2015_sf, february_start_index, february_end_index)
march_scp_wy2015 = MonthlySnowCoverPercent(wy2015_sf, march_start_index, march_end_index)
april_scp_wy2015 = MonthlySnowCoverPercent(wy2015_sf, april_start_index, april_end_index)
may_scp_wy2015 = MonthlySnowCoverPercent(wy2015_sf, may_start_index, may_end_index)
june_scp_wy2015 = MonthlySnowCoverPercent(wy2015_sf, june_start_index, june_end_index)
july_scp_wy2015 = MonthlySnowCoverPercent(wy2015_sf, july_start_index, july_end_index)
august_scp_wy2015 = MonthlySnowCoverPercent(wy2015_sf, august_start_index, august_end_index)
september_scp_wy2015 = MonthlySnowCoverPercent(wy2015_sf, september_start_index, september_end_index)
october_scp_wy2015 = MonthlySnowCoverPercent(wy2015_sf, october_start_index, october_end_index)
november_scp_wy2015 = MonthlySnowCoverPercent(wy2015_sf, november_start_index, november_end_index)
december_scp_wy2015 = MonthlySnowCoverPercent(wy2015_sf, december_start_index, december_end_index)

# wy2015_monthly_mean_scp_stack = array([january_scp_wy2015, february_scp_wy2015, march_scp_wy2015, april_scp_wy2015,
#                                       may_scp_wy2015, june_scp_wy2015, july_scp_wy2015, august_scp_wy2015, 
#                                        september_scp_wy2015, october_scp_wy2015, november_scp_wy2015, december_scp_wy2015])

# wy2015_monthly_mean_scp_stack[wy2015_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2015_monthly_mean_scp_stack.tif', wy2015_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [43]:
warnings.filterwarnings('ignore')

# wy2016
january_scp_wy2016 = MonthlySnowCoverPercent(wy2016_sf, january_start_index, january_end_index)
february_scp_wy2016 = MonthlySnowCoverPercent(wy2016_sf, february_leapyear_start_index, february_leapyear_end_index)
march_scp_wy2016 = MonthlySnowCoverPercent(wy2016_sf, march_leapyear_start_index, march_leapyear_end_index)
april_scp_wy2016 = MonthlySnowCoverPercent(wy2016_sf, april_leapyear_start_index, april_leapyear_end_index)
may_scp_wy2016 = MonthlySnowCoverPercent(wy2016_sf, may_leapyear_start_index, may_leapyear_end_index)
june_scp_wy2016 = MonthlySnowCoverPercent(wy2016_sf, june_leapyear_start_index, june_leapyear_end_index)
july_scp_wy2016 = MonthlySnowCoverPercent(wy2016_sf, july_leapyear_start_index, july_leapyear_end_index)
august_scp_wy2016 = MonthlySnowCoverPercent(wy2016_sf, august_leapyear_start_index, august_leapyear_end_index)
september_scp_wy2016 = MonthlySnowCoverPercent(wy2016_sf, september_leapyear_start_index, september_leapyear_end_index)
october_scp_wy2016 = MonthlySnowCoverPercent(wy2016_sf, october_start_index, october_end_index)
november_scp_wy2016 = MonthlySnowCoverPercent(wy2016_sf, november_start_index, november_end_index)
december_scp_wy2016 = MonthlySnowCoverPercent(wy2016_sf, december_start_index, december_end_index)

# wy2016_monthly_mean_scp_stack = array([january_scp_wy2016, february_scp_wy2016, march_scp_wy2016, april_scp_wy2016,
#                                       may_scp_wy2016, june_scp_wy2016, july_scp_wy2016, august_scp_wy2016, 
#                                        september_scp_wy2016, october_scp_wy2016, november_scp_wy2016, december_scp_wy2016])

# wy2016_monthly_mean_scp_stack[wy2016_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2016_monthly_mean_scp_stack.tif', wy2016_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [44]:
warnings.filterwarnings('ignore')

# wy2017
january_scp_wy2017 = MonthlySnowCoverPercent(wy2017_sf, january_start_index, january_end_index)
february_scp_wy2017 = MonthlySnowCoverPercent(wy2017_sf, february_start_index, february_end_index)
march_scp_wy2017 = MonthlySnowCoverPercent(wy2017_sf, march_start_index, march_end_index)
april_scp_wy2017 = MonthlySnowCoverPercent(wy2017_sf, april_start_index, april_end_index)
may_scp_wy2017 = MonthlySnowCoverPercent(wy2017_sf, may_start_index, may_end_index)
june_scp_wy2017 = MonthlySnowCoverPercent(wy2017_sf, june_start_index, june_end_index)
july_scp_wy2017 = MonthlySnowCoverPercent(wy2017_sf, july_start_index, july_end_index)
august_scp_wy2017 = MonthlySnowCoverPercent(wy2017_sf, august_start_index, august_end_index)
september_scp_wy2017 = MonthlySnowCoverPercent(wy2017_sf, september_start_index, september_end_index)
october_scp_wy2017 = MonthlySnowCoverPercent(wy2017_sf, october_start_index, october_end_index)
november_scp_wy2017 = MonthlySnowCoverPercent(wy2017_sf, november_start_index, november_end_index)
december_scp_wy2017 = MonthlySnowCoverPercent(wy2017_sf, december_start_index, december_end_index)

# wy2017_monthly_mean_scp_stack = array([january_scp_wy2017, february_scp_wy2017, march_scp_wy2017, april_scp_wy2017,
#                                       may_scp_wy2017, june_scp_wy2017, july_scp_wy2017, august_scp_wy2017, 
#                                        september_scp_wy2017, october_scp_wy2017, november_scp_wy2017, december_scp_wy2017])

# wy2017_monthly_mean_scp_stack[wy2017_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2017_monthly_mean_scp_stack.tif', wy2017_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [45]:
warnings.filterwarnings('ignore')

# wy2018
january_scp_wy2018 = MonthlySnowCoverPercent(wy2018_sf, january_start_index, january_end_index)
february_scp_wy2018 = MonthlySnowCoverPercent(wy2018_sf, february_start_index, february_end_index)
march_scp_wy2018 = MonthlySnowCoverPercent(wy2018_sf, march_start_index, march_end_index)
april_scp_wy2018 = MonthlySnowCoverPercent(wy2018_sf, april_start_index, april_end_index)
may_scp_wy2018 = MonthlySnowCoverPercent(wy2018_sf, may_start_index, may_end_index)
june_scp_wy2018 = MonthlySnowCoverPercent(wy2018_sf, june_start_index, june_end_index)
july_scp_wy2018 = MonthlySnowCoverPercent(wy2018_sf, july_start_index, july_end_index)
august_scp_wy2018 = MonthlySnowCoverPercent(wy2018_sf, august_start_index, august_end_index)
september_scp_wy2018 = MonthlySnowCoverPercent(wy2018_sf, september_start_index, september_end_index)
october_scp_wy2018 = MonthlySnowCoverPercent(wy2018_sf, october_start_index, october_end_index)
november_scp_wy2018 = MonthlySnowCoverPercent(wy2018_sf, november_start_index, november_end_index)
december_scp_wy2018 = MonthlySnowCoverPercent(wy2018_sf, december_start_index, december_end_index)

# wy2018_monthly_mean_scp_stack = array([january_scp_wy2018, february_scp_wy2018, march_scp_wy2018, april_scp_wy2018,
#                                       may_scp_wy2018, june_scp_wy2018, july_scp_wy2018, august_scp_wy2018, 
#                                        september_scp_wy2018, october_scp_wy2018, november_scp_wy2018, december_scp_wy2018])

# wy2018_monthly_mean_scp_stack[wy2018_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2018_monthly_mean_scp_stack.tif', wy2018_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [46]:
warnings.filterwarnings('ignore')

# wy2019
january_scp_wy2019 = MonthlySnowCoverPercent(wy2019_sf, january_start_index, january_end_index)
february_scp_wy2019 = MonthlySnowCoverPercent(wy2019_sf, february_start_index, february_end_index)
march_scp_wy2019 = MonthlySnowCoverPercent(wy2019_sf, march_start_index, march_end_index)
april_scp_wy2019 = MonthlySnowCoverPercent(wy2019_sf, april_start_index, april_end_index)
may_scp_wy2019 = MonthlySnowCoverPercent(wy2019_sf, may_start_index, may_end_index)
june_scp_wy2019 = MonthlySnowCoverPercent(wy2019_sf, june_start_index, june_end_index)
july_scp_wy2019 = MonthlySnowCoverPercent(wy2019_sf, july_start_index, july_end_index)
august_scp_wy2019 = MonthlySnowCoverPercent(wy2019_sf, august_start_index, august_end_index)
september_scp_wy2019 = MonthlySnowCoverPercent(wy2019_sf, september_start_index, september_end_index)
october_scp_wy2019 = MonthlySnowCoverPercent(wy2019_sf, october_start_index, october_end_index)
november_scp_wy2019 = MonthlySnowCoverPercent(wy2019_sf, november_start_index, november_end_index)
december_scp_wy2019 = MonthlySnowCoverPercent(wy2019_sf, december_start_index, december_end_index)

# wy2019_monthly_mean_scp_stack = array([january_scp_wy2019, february_scp_wy2019, march_scp_wy2019, april_scp_wy2019,
#                                       may_scp_wy2019, june_scp_wy2019, july_scp_wy2019, august_scp_wy2019, 
#                                        september_scp_wy2019, october_scp_wy2019, november_scp_wy2019, december_scp_wy2019])

# wy2019_monthly_mean_scp_stack[wy2019_monthly_mean_scp_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2019_monthly_mean_scp_stack.tif', wy2019_monthly_mean_scp_stack, ReferencingMatrix, Projection)

In [161]:
january_mean_scp = (january_scp_wy2001 + january_scp_wy2002 + january_scp_wy2003 + january_scp_wy2004 + january_scp_wy2005 + 
                    january_scp_wy2006 + january_scp_wy2007 + january_scp_wy2008 + january_scp_wy2009 + january_scp_wy2010 + 
                    january_scp_wy2011 + january_scp_wy2012 + january_scp_wy2013 + january_scp_wy2014 + january_scp_wy2015 + 
                    january_scp_wy2016 + january_scp_wy2017 + january_scp_wy2018 + january_scp_wy2019) / 19

january_mean_scp_stack = array([january_scp_wy2001, january_scp_wy2002, january_scp_wy2003, january_scp_wy2004, january_scp_wy2005, 
                                january_scp_wy2006, january_scp_wy2007, january_scp_wy2008, january_scp_wy2009, january_scp_wy2010, 
                                january_scp_wy2011, january_scp_wy2012, january_scp_wy2013, january_scp_wy2014, january_scp_wy2015, 
                                january_scp_wy2016, january_scp_wy2017, january_scp_wy2018, january_scp_wy2019, january_mean_scp])

january_mean_scp_stack[january_mean_scp_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/january_mean_scp_stack.tif', january_mean_scp_stack, ReferencingMatrix, Projection)

'stacked_tiffs/january_mean_scp_stack.tif'

In [162]:
february_mean_scp = (february_scp_wy2001 + february_scp_wy2002 + february_scp_wy2003 + february_scp_wy2004 + february_scp_wy2005 + 
                    february_scp_wy2006 + february_scp_wy2007 + february_scp_wy2008 + february_scp_wy2009 + february_scp_wy2010 + 
                    february_scp_wy2011 + february_scp_wy2012 + february_scp_wy2013 + february_scp_wy2014 + february_scp_wy2015 + 
                    february_scp_wy2016 + february_scp_wy2017 + february_scp_wy2018 + february_scp_wy2019) / 19 

february_mean_scp_stack = array([february_scp_wy2001, february_scp_wy2002, february_scp_wy2003, february_scp_wy2004, february_scp_wy2005, 
                                february_scp_wy2006, february_scp_wy2007, february_scp_wy2008, february_scp_wy2009, february_scp_wy2010, 
                                february_scp_wy2011, february_scp_wy2012, february_scp_wy2013, february_scp_wy2014, february_scp_wy2015, 
                                february_scp_wy2016, february_scp_wy2017, february_scp_wy2018, february_scp_wy2019, february_mean_scp])

february_mean_scp_stack[february_mean_scp_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/february_mean_scp_stack.tif', february_mean_scp_stack, ReferencingMatrix, Projection)

'stacked_tiffs/february_mean_scp_stack.tif'

In [49]:
march_mean_scp = (march_scp_wy2001 + march_scp_wy2002 + march_scp_wy2003 + march_scp_wy2004 + march_scp_wy2005 + 
                    march_scp_wy2006 + march_scp_wy2007 + march_scp_wy2008 + march_scp_wy2009 + march_scp_wy2010 + 
                    march_scp_wy2011 + march_scp_wy2012 + march_scp_wy2013 + march_scp_wy2014 + march_scp_wy2015 + 
                    march_scp_wy2016 + march_scp_wy2017 + march_scp_wy2018 + march_scp_wy2019) / 19 

march_mean_scp_stack = array([march_scp_wy2001, march_scp_wy2002, march_scp_wy2003, march_scp_wy2004, march_scp_wy2005, 
                                march_scp_wy2006, march_scp_wy2007, march_scp_wy2008, march_scp_wy2009, march_scp_wy2010, 
                                march_scp_wy2011, march_scp_wy2012, march_scp_wy2013, march_scp_wy2014, march_scp_wy2015, 
                                march_scp_wy2016, march_scp_wy2017, march_scp_wy2018, march_scp_wy2019, march_mean_scp])

march_mean_scp_stack[march_mean_scp_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/march_mean_scp_stack.tif', march_mean_scp_stack, ReferencingMatrix, Projection)

'stacked_tiffs/march_mean_scp_stack.tif'

In [50]:
april_mean_scp = (april_scp_wy2001 + april_scp_wy2002 + april_scp_wy2003 + april_scp_wy2004 + april_scp_wy2005 + 
                    april_scp_wy2006 + april_scp_wy2007 + april_scp_wy2008 + april_scp_wy2009 + april_scp_wy2010 + 
                    april_scp_wy2011 + april_scp_wy2012 + april_scp_wy2013 + april_scp_wy2014 + april_scp_wy2015 + 
                    april_scp_wy2016 + april_scp_wy2017 + april_scp_wy2018 + april_scp_wy2019) / 19 

april_mean_scp_stack = array([april_scp_wy2001, april_scp_wy2002, april_scp_wy2003, april_scp_wy2004, april_scp_wy2005, 
                                april_scp_wy2006, april_scp_wy2007, april_scp_wy2008, april_scp_wy2009, april_scp_wy2010, 
                                april_scp_wy2011, april_scp_wy2012, april_scp_wy2013, april_scp_wy2014, april_scp_wy2015, 
                                april_scp_wy2016, april_scp_wy2017, april_scp_wy2018, april_scp_wy2019, april_mean_scp])

april_mean_scp_stack[april_mean_scp_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/april_mean_scp_stack.tif', april_mean_scp_stack, ReferencingMatrix, Projection)

'stacked_tiffs/april_mean_scp_stack.tif'

In [51]:
may_mean_scp = (may_scp_wy2001 + may_scp_wy2002 + may_scp_wy2003 + may_scp_wy2004 + may_scp_wy2005 + 
                    may_scp_wy2006 + may_scp_wy2007 + may_scp_wy2008 + may_scp_wy2009 + may_scp_wy2010 + 
                    may_scp_wy2011 + may_scp_wy2012 + may_scp_wy2013 + may_scp_wy2014 + may_scp_wy2015 + 
                    may_scp_wy2016 + may_scp_wy2017 + may_scp_wy2018 + may_scp_wy2019) / 19 

may_mean_scp_stack = array([may_scp_wy2001, may_scp_wy2002, may_scp_wy2003, may_scp_wy2004, may_scp_wy2005, 
                                may_scp_wy2006, may_scp_wy2007, may_scp_wy2008, may_scp_wy2009, may_scp_wy2010, 
                                may_scp_wy2011, may_scp_wy2012, may_scp_wy2013, may_scp_wy2014, may_scp_wy2015, 
                                may_scp_wy2016, may_scp_wy2017, may_scp_wy2018, may_scp_wy2019, may_mean_scp])

may_mean_scp_stack[may_mean_scp_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/may_mean_scp_stack.tif', may_mean_scp_stack, ReferencingMatrix, Projection)

'stacked_tiffs/may_mean_scp_stack.tif'

In [52]:
june_mean_scp = (june_scp_wy2001 + june_scp_wy2002 + june_scp_wy2003 + june_scp_wy2004 + june_scp_wy2005 + 
                    june_scp_wy2006 + june_scp_wy2007 + june_scp_wy2008 + june_scp_wy2009 + june_scp_wy2010 + 
                    june_scp_wy2011 + june_scp_wy2012 + june_scp_wy2013 + june_scp_wy2014 + june_scp_wy2015 + 
                    june_scp_wy2016 + june_scp_wy2017 + june_scp_wy2018 + june_scp_wy2019) / 19 

june_mean_scp_stack = array([june_scp_wy2001, june_scp_wy2002, june_scp_wy2003, june_scp_wy2004, june_scp_wy2005, 
                                june_scp_wy2006, june_scp_wy2007, june_scp_wy2008, june_scp_wy2009, june_scp_wy2010, 
                                june_scp_wy2011, june_scp_wy2012, june_scp_wy2013, june_scp_wy2014, june_scp_wy2015, 
                                june_scp_wy2016, june_scp_wy2017, june_scp_wy2018, june_scp_wy2019, june_mean_scp])

june_mean_scp_stack[june_mean_scp_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/june_mean_scp_stack.tif', june_mean_scp_stack, ReferencingMatrix, Projection)

'stacked_tiffs/june_mean_scp_stack.tif'

In [53]:
july_mean_scp = (july_scp_wy2001 + july_scp_wy2002 + july_scp_wy2003 + july_scp_wy2004 + july_scp_wy2005 + 
                    july_scp_wy2006 + july_scp_wy2007 + july_scp_wy2008 + july_scp_wy2009 + july_scp_wy2010 + 
                    july_scp_wy2011 + july_scp_wy2012 + july_scp_wy2013 + july_scp_wy2014 + july_scp_wy2015 + 
                    july_scp_wy2016 + july_scp_wy2017 + july_scp_wy2018 + july_scp_wy2019) / 19 

july_mean_scp_stack = array([july_scp_wy2001, july_scp_wy2002, july_scp_wy2003, july_scp_wy2004, july_scp_wy2005, 
                                july_scp_wy2006, july_scp_wy2007, july_scp_wy2008, july_scp_wy2009, july_scp_wy2010, 
                                july_scp_wy2011, july_scp_wy2012, july_scp_wy2013, july_scp_wy2014, july_scp_wy2015, 
                                july_scp_wy2016, july_scp_wy2017, july_scp_wy2018, july_scp_wy2019, july_mean_scp])

july_mean_scp_stack[july_mean_scp_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/july_mean_scp_stack.tif', july_mean_scp_stack, ReferencingMatrix, Projection)

'stacked_tiffs/july_mean_scp_stack.tif'

In [54]:
august_mean_scp = (august_scp_wy2001 + august_scp_wy2002 + august_scp_wy2003 + august_scp_wy2004 + august_scp_wy2005 + 
                    august_scp_wy2006 + august_scp_wy2007 + august_scp_wy2008 + august_scp_wy2009 + august_scp_wy2010 + 
                    august_scp_wy2011 + august_scp_wy2012 + august_scp_wy2013 + august_scp_wy2014 + august_scp_wy2015 + 
                    august_scp_wy2016 + august_scp_wy2017 + august_scp_wy2018 + august_scp_wy2019) / 19 

august_mean_scp_stack = array([august_scp_wy2001, august_scp_wy2002, august_scp_wy2003, august_scp_wy2004, august_scp_wy2005, 
                                august_scp_wy2006, august_scp_wy2007, august_scp_wy2008, august_scp_wy2009, august_scp_wy2010, 
                                august_scp_wy2011, august_scp_wy2012, august_scp_wy2013, august_scp_wy2014, august_scp_wy2015, 
                                august_scp_wy2016, august_scp_wy2017, august_scp_wy2018, august_scp_wy2019, august_mean_scp])

august_mean_scp_stack[august_mean_scp_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/august_mean_scp_stack.tif', august_mean_scp_stack, ReferencingMatrix, Projection)

'stacked_tiffs/august_mean_scp_stack.tif'

In [55]:
september_mean_scp = (september_scp_wy2001 + september_scp_wy2002 + september_scp_wy2003 + september_scp_wy2004 + september_scp_wy2005 + 
                    september_scp_wy2006 + september_scp_wy2007 + september_scp_wy2008 + september_scp_wy2009 + september_scp_wy2010 + 
                    september_scp_wy2011 + september_scp_wy2012 + september_scp_wy2013 + september_scp_wy2014 + september_scp_wy2015 + 
                    september_scp_wy2016 + september_scp_wy2017 + september_scp_wy2018 + september_scp_wy2019) / 19 

september_mean_scp_stack = array([september_scp_wy2001, september_scp_wy2002, september_scp_wy2003, september_scp_wy2004, september_scp_wy2005, 
                                september_scp_wy2006, september_scp_wy2007, september_scp_wy2008, september_scp_wy2009, september_scp_wy2010, 
                                september_scp_wy2011, september_scp_wy2012, september_scp_wy2013, september_scp_wy2014, september_scp_wy2015, 
                                september_scp_wy2016, september_scp_wy2017, september_scp_wy2018, september_scp_wy2019, september_mean_scp])

september_mean_scp_stack[september_mean_scp_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/september_mean_scp_stack.tif', september_mean_scp_stack, ReferencingMatrix, Projection)

'stacked_tiffs/september_mean_scp_stack.tif'

In [56]:
october_mean_scp = (october_scp_wy2001 + october_scp_wy2002 + october_scp_wy2003 + october_scp_wy2004 + october_scp_wy2005 + 
                    october_scp_wy2006 + october_scp_wy2007 + october_scp_wy2008 + october_scp_wy2009 + october_scp_wy2010 + 
                    october_scp_wy2011 + october_scp_wy2012 + october_scp_wy2013 + october_scp_wy2014 + october_scp_wy2015 + 
                    october_scp_wy2016 + october_scp_wy2017 + october_scp_wy2018 + october_scp_wy2019) / 19 

october_mean_scp_stack = array([october_scp_wy2001, october_scp_wy2002, october_scp_wy2003, october_scp_wy2004, october_scp_wy2005, 
                                october_scp_wy2006, october_scp_wy2007, october_scp_wy2008, october_scp_wy2009, october_scp_wy2010, 
                                october_scp_wy2011, october_scp_wy2012, october_scp_wy2013, october_scp_wy2014, october_scp_wy2015, 
                                october_scp_wy2016, october_scp_wy2017, october_scp_wy2018, october_scp_wy2019, october_mean_scp])

october_mean_scp_stack[october_mean_scp_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/october_mean_scp_stack.tif', october_mean_scp_stack, ReferencingMatrix, Projection)

'stacked_tiffs/october_mean_scp_stack.tif'

In [57]:
november_mean_scp = (november_scp_wy2001 + november_scp_wy2002 + november_scp_wy2003 + november_scp_wy2004 + november_scp_wy2005 + 
                    november_scp_wy2006 + november_scp_wy2007 + november_scp_wy2008 + november_scp_wy2009 + november_scp_wy2010 + 
                    november_scp_wy2011 + november_scp_wy2012 + november_scp_wy2013 + november_scp_wy2014 + november_scp_wy2015 + 
                    november_scp_wy2016 + november_scp_wy2017 + november_scp_wy2018 + november_scp_wy2019) / 19 

november_mean_scp_stack = array([november_scp_wy2001, november_scp_wy2002, november_scp_wy2003, november_scp_wy2004, november_scp_wy2005, 
                                november_scp_wy2006, november_scp_wy2007, november_scp_wy2008, november_scp_wy2009, november_scp_wy2010, 
                                november_scp_wy2011, november_scp_wy2012, november_scp_wy2013, november_scp_wy2014, november_scp_wy2015, 
                                november_scp_wy2016, november_scp_wy2017, november_scp_wy2018, november_scp_wy2019, november_mean_scp])

november_mean_scp_stack[november_mean_scp_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/november_mean_scp_stack.tif', november_mean_scp_stack, ReferencingMatrix, Projection)

'stacked_tiffs/november_mean_scp_stack.tif'

In [58]:
december_mean_scp = (december_scp_wy2001 + december_scp_wy2002 + december_scp_wy2003 + december_scp_wy2004 + december_scp_wy2005 + 
                    december_scp_wy2006 + december_scp_wy2007 + december_scp_wy2008 + december_scp_wy2009 + december_scp_wy2010 + 
                    december_scp_wy2011 + december_scp_wy2012 + december_scp_wy2013 + december_scp_wy2014 + december_scp_wy2015 + 
                    december_scp_wy2016 + december_scp_wy2017 + december_scp_wy2018 + december_scp_wy2019) / 19 

december_mean_scp_stack = array([december_scp_wy2001, december_scp_wy2002, december_scp_wy2003, december_scp_wy2004, december_scp_wy2005, 
                                december_scp_wy2006, december_scp_wy2007, december_scp_wy2008, december_scp_wy2009, december_scp_wy2010, 
                                december_scp_wy2011, december_scp_wy2012, december_scp_wy2013, december_scp_wy2014, december_scp_wy2015, 
                                december_scp_wy2016, december_scp_wy2017, december_scp_wy2018, december_scp_wy2019, december_mean_scp])

december_mean_scp_stack[december_mean_scp_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/december_mean_scp_stack.tif', december_mean_scp_stack, ReferencingMatrix, Projection)

'stacked_tiffs/december_mean_scp_stack.tif'

### Anomaly

In [59]:
# january snow cover percent anomaly
wy2001_january_scp_anomaly = january_scp_wy2001 - january_mean_scp
wy2002_january_scp_anomaly = january_scp_wy2002 - january_mean_scp
wy2003_january_scp_anomaly = january_scp_wy2003 - january_mean_scp
wy2004_january_scp_anomaly = january_scp_wy2004 - january_mean_scp
wy2005_january_scp_anomaly = january_scp_wy2005 - january_mean_scp
wy2006_january_scp_anomaly = january_scp_wy2006 - january_mean_scp
wy2007_january_scp_anomaly = january_scp_wy2007 - january_mean_scp
wy2008_january_scp_anomaly = january_scp_wy2008 - january_mean_scp
wy2009_january_scp_anomaly = january_scp_wy2009 - january_mean_scp
wy2010_january_scp_anomaly = january_scp_wy2010 - january_mean_scp
wy2011_january_scp_anomaly = january_scp_wy2011 - january_mean_scp
wy2012_january_scp_anomaly = january_scp_wy2012 - january_mean_scp
wy2013_january_scp_anomaly = january_scp_wy2013 - january_mean_scp
wy2014_january_scp_anomaly = january_scp_wy2014 - january_mean_scp
wy2015_january_scp_anomaly = january_scp_wy2015 - january_mean_scp
wy2016_january_scp_anomaly = january_scp_wy2016 - january_mean_scp
wy2017_january_scp_anomaly = january_scp_wy2017 - january_mean_scp
wy2018_january_scp_anomaly = january_scp_wy2018 - january_mean_scp
wy2019_january_scp_anomaly = january_scp_wy2019 - january_mean_scp

january_scp_anomaly_stack = array([wy2001_january_scp_anomaly, wy2002_january_scp_anomaly, wy2003_january_scp_anomaly, wy2004_january_scp_anomaly, wy2005_january_scp_anomaly,
                                   wy2006_january_scp_anomaly, wy2007_january_scp_anomaly, wy2008_january_scp_anomaly, wy2009_january_scp_anomaly, wy2010_january_scp_anomaly, 
                                   wy2011_january_scp_anomaly, wy2012_january_scp_anomaly, wy2013_january_scp_anomaly, wy2014_january_scp_anomaly, wy2015_january_scp_anomaly,
                                   wy2016_january_scp_anomaly, wy2017_january_scp_anomaly, wy2018_january_scp_anomaly, wy2019_january_scp_anomaly, january_mean_scp])

january_scp_anomaly_stack[january_scp_anomaly_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/january_scp_anomaly_stack.tif', january_scp_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/january_scp_anomaly_stack.tif'

In [257]:
np.nanmin(january_scp_anomaly_stack)

-91.51782682512734

In [256]:
np.nanmax(january_scp_anomaly_stack)

99.16977928692701

In [60]:
# february snow cover percent anomaly
wy2001_february_scp_anomaly = february_scp_wy2001 - february_mean_scp
wy2002_february_scp_anomaly = february_scp_wy2002 - february_mean_scp
wy2003_february_scp_anomaly = february_scp_wy2003 - february_mean_scp
wy2004_february_scp_anomaly = february_scp_wy2004 - february_mean_scp
wy2005_february_scp_anomaly = february_scp_wy2005 - february_mean_scp
wy2006_february_scp_anomaly = february_scp_wy2006 - february_mean_scp
wy2007_february_scp_anomaly = february_scp_wy2007 - february_mean_scp
wy2008_february_scp_anomaly = february_scp_wy2008 - february_mean_scp
wy2009_february_scp_anomaly = february_scp_wy2009 - february_mean_scp
wy2010_february_scp_anomaly = february_scp_wy2010 - february_mean_scp
wy2011_february_scp_anomaly = february_scp_wy2011 - february_mean_scp
wy2012_february_scp_anomaly = february_scp_wy2012 - february_mean_scp
wy2013_february_scp_anomaly = february_scp_wy2013 - february_mean_scp
wy2014_february_scp_anomaly = february_scp_wy2014 - february_mean_scp
wy2015_february_scp_anomaly = february_scp_wy2015 - february_mean_scp
wy2016_february_scp_anomaly = february_scp_wy2016 - february_mean_scp
wy2017_february_scp_anomaly = february_scp_wy2017 - february_mean_scp
wy2018_february_scp_anomaly = february_scp_wy2018 - february_mean_scp
wy2019_february_scp_anomaly = february_scp_wy2019 - february_mean_scp

february_scp_anomaly_stack = array([wy2001_february_scp_anomaly, wy2002_february_scp_anomaly, wy2003_february_scp_anomaly, wy2004_february_scp_anomaly, wy2005_february_scp_anomaly,
                                   wy2006_february_scp_anomaly, wy2007_february_scp_anomaly, wy2008_february_scp_anomaly, wy2009_february_scp_anomaly, wy2010_february_scp_anomaly, 
                                   wy2011_february_scp_anomaly, wy2012_february_scp_anomaly, wy2013_february_scp_anomaly, wy2014_february_scp_anomaly, wy2015_february_scp_anomaly,
                                   wy2016_february_scp_anomaly, wy2017_february_scp_anomaly, wy2018_february_scp_anomaly, wy2019_february_scp_anomaly, february_mean_scp])

february_scp_anomaly_stack[february_scp_anomaly_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/february_scp_anomaly_stack.tif', february_scp_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/february_scp_anomaly_stack.tif'

In [61]:
# march snow cover percent anomaly
wy2001_march_scp_anomaly = march_scp_wy2001 - march_mean_scp
wy2002_march_scp_anomaly = march_scp_wy2002 - march_mean_scp
wy2003_march_scp_anomaly = march_scp_wy2003 - march_mean_scp
wy2004_march_scp_anomaly = march_scp_wy2004 - march_mean_scp
wy2005_march_scp_anomaly = march_scp_wy2005 - march_mean_scp
wy2006_march_scp_anomaly = march_scp_wy2006 - march_mean_scp
wy2007_march_scp_anomaly = march_scp_wy2007 - march_mean_scp
wy2008_march_scp_anomaly = march_scp_wy2008 - march_mean_scp
wy2009_march_scp_anomaly = march_scp_wy2009 - march_mean_scp
wy2010_march_scp_anomaly = march_scp_wy2010 - march_mean_scp
wy2011_march_scp_anomaly = march_scp_wy2011 - march_mean_scp
wy2012_march_scp_anomaly = march_scp_wy2012 - march_mean_scp
wy2013_march_scp_anomaly = march_scp_wy2013 - march_mean_scp
wy2014_march_scp_anomaly = march_scp_wy2014 - march_mean_scp
wy2015_march_scp_anomaly = march_scp_wy2015 - march_mean_scp
wy2016_march_scp_anomaly = march_scp_wy2016 - march_mean_scp
wy2017_march_scp_anomaly = march_scp_wy2017 - march_mean_scp
wy2018_march_scp_anomaly = march_scp_wy2018 - march_mean_scp
wy2019_march_scp_anomaly = march_scp_wy2019 - march_mean_scp

march_scp_anomaly_stack = array([wy2001_march_scp_anomaly, wy2002_march_scp_anomaly, wy2003_march_scp_anomaly, wy2004_march_scp_anomaly, wy2005_march_scp_anomaly,
                                   wy2006_march_scp_anomaly, wy2007_march_scp_anomaly, wy2008_march_scp_anomaly, wy2009_march_scp_anomaly, wy2010_march_scp_anomaly, 
                                   wy2011_march_scp_anomaly, wy2012_march_scp_anomaly, wy2013_march_scp_anomaly, wy2014_march_scp_anomaly, wy2015_march_scp_anomaly,
                                   wy2016_march_scp_anomaly, wy2017_march_scp_anomaly, wy2018_march_scp_anomaly, wy2019_march_scp_anomaly, march_mean_scp])

march_scp_anomaly_stack[march_scp_anomaly_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/march_scp_anomaly_stack.tif', march_scp_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/march_scp_anomaly_stack.tif'

In [62]:
# april snow cover percent anomaly
wy2001_april_scp_anomaly = april_scp_wy2001 - april_mean_scp
wy2002_april_scp_anomaly = april_scp_wy2002 - april_mean_scp
wy2003_april_scp_anomaly = april_scp_wy2003 - april_mean_scp
wy2004_april_scp_anomaly = april_scp_wy2004 - april_mean_scp
wy2005_april_scp_anomaly = april_scp_wy2005 - april_mean_scp
wy2006_april_scp_anomaly = april_scp_wy2006 - april_mean_scp
wy2007_april_scp_anomaly = april_scp_wy2007 - april_mean_scp
wy2008_april_scp_anomaly = april_scp_wy2008 - april_mean_scp
wy2009_april_scp_anomaly = april_scp_wy2009 - april_mean_scp
wy2010_april_scp_anomaly = april_scp_wy2010 - april_mean_scp
wy2011_april_scp_anomaly = april_scp_wy2011 - april_mean_scp
wy2012_april_scp_anomaly = april_scp_wy2012 - april_mean_scp
wy2013_april_scp_anomaly = april_scp_wy2013 - april_mean_scp
wy2014_april_scp_anomaly = april_scp_wy2014 - april_mean_scp
wy2015_april_scp_anomaly = april_scp_wy2015 - april_mean_scp
wy2016_april_scp_anomaly = april_scp_wy2016 - april_mean_scp
wy2017_april_scp_anomaly = april_scp_wy2017 - april_mean_scp
wy2018_april_scp_anomaly = april_scp_wy2018 - april_mean_scp
wy2019_april_scp_anomaly = april_scp_wy2019 - april_mean_scp

april_scp_anomaly_stack = array([wy2001_april_scp_anomaly, wy2002_april_scp_anomaly, wy2003_april_scp_anomaly, wy2004_april_scp_anomaly, wy2005_april_scp_anomaly,
                                   wy2006_april_scp_anomaly, wy2007_april_scp_anomaly, wy2008_april_scp_anomaly, wy2009_april_scp_anomaly, wy2010_april_scp_anomaly, 
                                   wy2011_april_scp_anomaly, wy2012_april_scp_anomaly, wy2013_april_scp_anomaly, wy2014_april_scp_anomaly, wy2015_april_scp_anomaly,
                                   wy2016_april_scp_anomaly, wy2017_april_scp_anomaly, wy2018_april_scp_anomaly, wy2019_april_scp_anomaly, april_mean_scp])

april_scp_anomaly_stack[april_scp_anomaly_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/april_scp_anomaly_stack.tif', april_scp_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/april_scp_anomaly_stack.tif'

In [63]:
# may snow cover percent anomaly
wy2001_may_scp_anomaly = may_scp_wy2001 - may_mean_scp
wy2002_may_scp_anomaly = may_scp_wy2002 - may_mean_scp
wy2003_may_scp_anomaly = may_scp_wy2003 - may_mean_scp
wy2004_may_scp_anomaly = may_scp_wy2004 - may_mean_scp
wy2005_may_scp_anomaly = may_scp_wy2005 - may_mean_scp
wy2006_may_scp_anomaly = may_scp_wy2006 - may_mean_scp
wy2007_may_scp_anomaly = may_scp_wy2007 - may_mean_scp
wy2008_may_scp_anomaly = may_scp_wy2008 - may_mean_scp
wy2009_may_scp_anomaly = may_scp_wy2009 - may_mean_scp
wy2010_may_scp_anomaly = may_scp_wy2010 - may_mean_scp
wy2011_may_scp_anomaly = may_scp_wy2011 - may_mean_scp
wy2012_may_scp_anomaly = may_scp_wy2012 - may_mean_scp
wy2013_may_scp_anomaly = may_scp_wy2013 - may_mean_scp
wy2014_may_scp_anomaly = may_scp_wy2014 - may_mean_scp
wy2015_may_scp_anomaly = may_scp_wy2015 - may_mean_scp
wy2016_may_scp_anomaly = may_scp_wy2016 - may_mean_scp
wy2017_may_scp_anomaly = may_scp_wy2017 - may_mean_scp
wy2018_may_scp_anomaly = may_scp_wy2018 - may_mean_scp
wy2019_may_scp_anomaly = may_scp_wy2019 - may_mean_scp

may_scp_anomaly_stack = array([wy2001_may_scp_anomaly, wy2002_may_scp_anomaly, wy2003_may_scp_anomaly, wy2004_may_scp_anomaly, wy2005_may_scp_anomaly,
                                   wy2006_may_scp_anomaly, wy2007_may_scp_anomaly, wy2008_may_scp_anomaly, wy2009_may_scp_anomaly, wy2010_may_scp_anomaly, 
                                   wy2011_may_scp_anomaly, wy2012_may_scp_anomaly, wy2013_may_scp_anomaly, wy2014_may_scp_anomaly, wy2015_may_scp_anomaly,
                                   wy2016_may_scp_anomaly, wy2017_may_scp_anomaly, wy2018_may_scp_anomaly, wy2019_may_scp_anomaly, may_mean_scp])

may_scp_anomaly_stack[may_scp_anomaly_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/may_scp_anomaly_stack.tif', may_scp_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/may_scp_anomaly_stack.tif'

In [64]:
# june snow cover percent anomaly
wy2001_june_scp_anomaly = june_scp_wy2001 - june_mean_scp
wy2002_june_scp_anomaly = june_scp_wy2002 - june_mean_scp
wy2003_june_scp_anomaly = june_scp_wy2003 - june_mean_scp
wy2004_june_scp_anomaly = june_scp_wy2004 - june_mean_scp
wy2005_june_scp_anomaly = june_scp_wy2005 - june_mean_scp
wy2006_june_scp_anomaly = june_scp_wy2006 - june_mean_scp
wy2007_june_scp_anomaly = june_scp_wy2007 - june_mean_scp
wy2008_june_scp_anomaly = june_scp_wy2008 - june_mean_scp
wy2009_june_scp_anomaly = june_scp_wy2009 - june_mean_scp
wy2010_june_scp_anomaly = june_scp_wy2010 - june_mean_scp
wy2011_june_scp_anomaly = june_scp_wy2011 - june_mean_scp
wy2012_june_scp_anomaly = june_scp_wy2012 - june_mean_scp
wy2013_june_scp_anomaly = june_scp_wy2013 - june_mean_scp
wy2014_june_scp_anomaly = june_scp_wy2014 - june_mean_scp
wy2015_june_scp_anomaly = june_scp_wy2015 - june_mean_scp
wy2016_june_scp_anomaly = june_scp_wy2016 - june_mean_scp
wy2017_june_scp_anomaly = june_scp_wy2017 - june_mean_scp
wy2018_june_scp_anomaly = june_scp_wy2018 - june_mean_scp
wy2019_june_scp_anomaly = june_scp_wy2019 - june_mean_scp

june_scp_anomaly_stack = array([wy2001_june_scp_anomaly, wy2002_june_scp_anomaly, wy2003_june_scp_anomaly, wy2004_june_scp_anomaly, wy2005_june_scp_anomaly,
                                   wy2006_june_scp_anomaly, wy2007_june_scp_anomaly, wy2008_june_scp_anomaly, wy2009_june_scp_anomaly, wy2010_june_scp_anomaly, 
                                   wy2011_june_scp_anomaly, wy2012_june_scp_anomaly, wy2013_june_scp_anomaly, wy2014_june_scp_anomaly, wy2015_june_scp_anomaly,
                                   wy2016_june_scp_anomaly, wy2017_june_scp_anomaly, wy2018_june_scp_anomaly, wy2019_june_scp_anomaly, june_mean_scp])

june_scp_anomaly_stack[june_scp_anomaly_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/june_scp_anomaly_stack.tif', june_scp_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/june_scp_anomaly_stack.tif'

In [65]:
# july snow cover percent anomaly
wy2001_july_scp_anomaly = july_scp_wy2001 - july_mean_scp
wy2002_july_scp_anomaly = july_scp_wy2002 - july_mean_scp
wy2003_july_scp_anomaly = july_scp_wy2003 - july_mean_scp
wy2004_july_scp_anomaly = july_scp_wy2004 - july_mean_scp
wy2005_july_scp_anomaly = july_scp_wy2005 - july_mean_scp
wy2006_july_scp_anomaly = july_scp_wy2006 - july_mean_scp
wy2007_july_scp_anomaly = july_scp_wy2007 - july_mean_scp
wy2008_july_scp_anomaly = july_scp_wy2008 - july_mean_scp
wy2009_july_scp_anomaly = july_scp_wy2009 - july_mean_scp
wy2010_july_scp_anomaly = july_scp_wy2010 - july_mean_scp
wy2011_july_scp_anomaly = july_scp_wy2011 - july_mean_scp
wy2012_july_scp_anomaly = july_scp_wy2012 - july_mean_scp
wy2013_july_scp_anomaly = july_scp_wy2013 - july_mean_scp
wy2014_july_scp_anomaly = july_scp_wy2014 - july_mean_scp
wy2015_july_scp_anomaly = july_scp_wy2015 - july_mean_scp
wy2016_july_scp_anomaly = july_scp_wy2016 - july_mean_scp
wy2017_july_scp_anomaly = july_scp_wy2017 - july_mean_scp
wy2018_july_scp_anomaly = july_scp_wy2018 - july_mean_scp
wy2019_july_scp_anomaly = july_scp_wy2019 - july_mean_scp

july_scp_anomaly_stack = array([wy2001_july_scp_anomaly, wy2002_july_scp_anomaly, wy2003_july_scp_anomaly, wy2004_july_scp_anomaly, wy2005_july_scp_anomaly,
                                   wy2006_july_scp_anomaly, wy2007_july_scp_anomaly, wy2008_july_scp_anomaly, wy2009_july_scp_anomaly, wy2010_july_scp_anomaly, 
                                   wy2011_july_scp_anomaly, wy2012_july_scp_anomaly, wy2013_july_scp_anomaly, wy2014_july_scp_anomaly, wy2015_july_scp_anomaly,
                                   wy2016_july_scp_anomaly, wy2017_july_scp_anomaly, wy2018_july_scp_anomaly, wy2019_july_scp_anomaly, july_mean_scp])

july_scp_anomaly_stack[july_scp_anomaly_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/july_scp_anomaly_stack.tif', july_scp_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/july_scp_anomaly_stack.tif'

In [66]:
# august snow cover percent anomaly
wy2001_august_scp_anomaly = august_scp_wy2001 - august_mean_scp
wy2002_august_scp_anomaly = august_scp_wy2002 - august_mean_scp
wy2003_august_scp_anomaly = august_scp_wy2003 - august_mean_scp
wy2004_august_scp_anomaly = august_scp_wy2004 - august_mean_scp
wy2005_august_scp_anomaly = august_scp_wy2005 - august_mean_scp
wy2006_august_scp_anomaly = august_scp_wy2006 - august_mean_scp
wy2007_august_scp_anomaly = august_scp_wy2007 - august_mean_scp
wy2008_august_scp_anomaly = august_scp_wy2008 - august_mean_scp
wy2009_august_scp_anomaly = august_scp_wy2009 - august_mean_scp
wy2010_august_scp_anomaly = august_scp_wy2010 - august_mean_scp
wy2011_august_scp_anomaly = august_scp_wy2011 - august_mean_scp
wy2012_august_scp_anomaly = august_scp_wy2012 - august_mean_scp
wy2013_august_scp_anomaly = august_scp_wy2013 - august_mean_scp
wy2014_august_scp_anomaly = august_scp_wy2014 - august_mean_scp
wy2015_august_scp_anomaly = august_scp_wy2015 - august_mean_scp
wy2016_august_scp_anomaly = august_scp_wy2016 - august_mean_scp
wy2017_august_scp_anomaly = august_scp_wy2017 - august_mean_scp
wy2018_august_scp_anomaly = august_scp_wy2018 - august_mean_scp
wy2019_august_scp_anomaly = august_scp_wy2019 - august_mean_scp

august_scp_anomaly_stack = array([wy2001_august_scp_anomaly, wy2002_august_scp_anomaly, wy2003_august_scp_anomaly, wy2004_august_scp_anomaly, wy2005_august_scp_anomaly,
                                   wy2006_august_scp_anomaly, wy2007_august_scp_anomaly, wy2008_august_scp_anomaly, wy2009_august_scp_anomaly, wy2010_august_scp_anomaly, 
                                   wy2011_august_scp_anomaly, wy2012_august_scp_anomaly, wy2013_august_scp_anomaly, wy2014_august_scp_anomaly, wy2015_august_scp_anomaly,
                                   wy2016_august_scp_anomaly, wy2017_august_scp_anomaly, wy2018_august_scp_anomaly, wy2019_august_scp_anomaly, august_mean_scp])

august_scp_anomaly_stack[august_scp_anomaly_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/august_scp_anomaly_stack.tif', august_scp_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/august_scp_anomaly_stack.tif'

In [67]:
# september snow cover percent anomaly
wy2001_september_scp_anomaly = september_scp_wy2001 - september_mean_scp
wy2002_september_scp_anomaly = september_scp_wy2002 - september_mean_scp
wy2003_september_scp_anomaly = september_scp_wy2003 - september_mean_scp
wy2004_september_scp_anomaly = september_scp_wy2004 - september_mean_scp
wy2005_september_scp_anomaly = september_scp_wy2005 - september_mean_scp
wy2006_september_scp_anomaly = september_scp_wy2006 - september_mean_scp
wy2007_september_scp_anomaly = september_scp_wy2007 - september_mean_scp
wy2008_september_scp_anomaly = september_scp_wy2008 - september_mean_scp
wy2009_september_scp_anomaly = september_scp_wy2009 - september_mean_scp
wy2010_september_scp_anomaly = september_scp_wy2010 - september_mean_scp
wy2011_september_scp_anomaly = september_scp_wy2011 - september_mean_scp
wy2012_september_scp_anomaly = september_scp_wy2012 - september_mean_scp
wy2013_september_scp_anomaly = september_scp_wy2013 - september_mean_scp
wy2014_september_scp_anomaly = september_scp_wy2014 - september_mean_scp
wy2015_september_scp_anomaly = september_scp_wy2015 - september_mean_scp
wy2016_september_scp_anomaly = september_scp_wy2016 - september_mean_scp
wy2017_september_scp_anomaly = september_scp_wy2017 - september_mean_scp
wy2018_september_scp_anomaly = september_scp_wy2018 - september_mean_scp
wy2019_september_scp_anomaly = september_scp_wy2019 - september_mean_scp

september_scp_anomaly_stack = array([wy2001_september_scp_anomaly, wy2002_september_scp_anomaly, wy2003_september_scp_anomaly, wy2004_september_scp_anomaly, wy2005_september_scp_anomaly,
                                   wy2006_september_scp_anomaly, wy2007_september_scp_anomaly, wy2008_september_scp_anomaly, wy2009_september_scp_anomaly, wy2010_september_scp_anomaly, 
                                   wy2011_september_scp_anomaly, wy2012_september_scp_anomaly, wy2013_september_scp_anomaly, wy2014_september_scp_anomaly, wy2015_september_scp_anomaly,
                                   wy2016_september_scp_anomaly, wy2017_september_scp_anomaly, wy2018_september_scp_anomaly, wy2019_september_scp_anomaly, september_mean_scp])

september_scp_anomaly_stack[september_scp_anomaly_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/september_scp_anomaly_stack.tif', september_scp_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/september_scp_anomaly_stack.tif'

In [68]:
# october snow cover percent anomaly
wy2001_october_scp_anomaly = october_scp_wy2001 - october_mean_scp
wy2002_october_scp_anomaly = october_scp_wy2002 - october_mean_scp
wy2003_october_scp_anomaly = october_scp_wy2003 - october_mean_scp
wy2004_october_scp_anomaly = october_scp_wy2004 - october_mean_scp
wy2005_october_scp_anomaly = october_scp_wy2005 - october_mean_scp
wy2006_october_scp_anomaly = october_scp_wy2006 - october_mean_scp
wy2007_october_scp_anomaly = october_scp_wy2007 - october_mean_scp
wy2008_october_scp_anomaly = october_scp_wy2008 - october_mean_scp
wy2009_october_scp_anomaly = october_scp_wy2009 - october_mean_scp
wy2010_october_scp_anomaly = october_scp_wy2010 - october_mean_scp
wy2011_october_scp_anomaly = october_scp_wy2011 - october_mean_scp
wy2012_october_scp_anomaly = october_scp_wy2012 - october_mean_scp
wy2013_october_scp_anomaly = october_scp_wy2013 - october_mean_scp
wy2014_october_scp_anomaly = october_scp_wy2014 - october_mean_scp
wy2015_october_scp_anomaly = october_scp_wy2015 - october_mean_scp
wy2016_october_scp_anomaly = october_scp_wy2016 - october_mean_scp
wy2017_october_scp_anomaly = october_scp_wy2017 - october_mean_scp
wy2018_october_scp_anomaly = october_scp_wy2018 - october_mean_scp
wy2019_october_scp_anomaly = october_scp_wy2019 - october_mean_scp

october_scp_anomaly_stack = array([wy2001_october_scp_anomaly, wy2002_october_scp_anomaly, wy2003_october_scp_anomaly, wy2004_october_scp_anomaly, wy2005_october_scp_anomaly,
                                   wy2006_october_scp_anomaly, wy2007_october_scp_anomaly, wy2008_october_scp_anomaly, wy2009_october_scp_anomaly, wy2010_october_scp_anomaly, 
                                   wy2011_october_scp_anomaly, wy2012_october_scp_anomaly, wy2013_october_scp_anomaly, wy2014_october_scp_anomaly, wy2015_october_scp_anomaly,
                                   wy2016_october_scp_anomaly, wy2017_october_scp_anomaly, wy2018_october_scp_anomaly, wy2019_october_scp_anomaly, october_mean_scp])

october_scp_anomaly_stack[october_scp_anomaly_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/october_scp_anomaly_stack.tif', october_scp_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/october_scp_anomaly_stack.tif'

In [69]:
# november snow cover percent anomaly
wy2001_november_scp_anomaly = november_scp_wy2001 - november_mean_scp
wy2002_november_scp_anomaly = november_scp_wy2002 - november_mean_scp
wy2003_november_scp_anomaly = november_scp_wy2003 - november_mean_scp
wy2004_november_scp_anomaly = november_scp_wy2004 - november_mean_scp
wy2005_november_scp_anomaly = november_scp_wy2005 - november_mean_scp
wy2006_november_scp_anomaly = november_scp_wy2006 - november_mean_scp
wy2007_november_scp_anomaly = november_scp_wy2007 - november_mean_scp
wy2008_november_scp_anomaly = november_scp_wy2008 - november_mean_scp
wy2009_november_scp_anomaly = november_scp_wy2009 - november_mean_scp
wy2010_november_scp_anomaly = november_scp_wy2010 - november_mean_scp
wy2011_november_scp_anomaly = november_scp_wy2011 - november_mean_scp
wy2012_november_scp_anomaly = november_scp_wy2012 - november_mean_scp
wy2013_november_scp_anomaly = november_scp_wy2013 - november_mean_scp
wy2014_november_scp_anomaly = november_scp_wy2014 - november_mean_scp
wy2015_november_scp_anomaly = november_scp_wy2015 - november_mean_scp
wy2016_november_scp_anomaly = november_scp_wy2016 - november_mean_scp
wy2017_november_scp_anomaly = november_scp_wy2017 - november_mean_scp
wy2018_november_scp_anomaly = november_scp_wy2018 - november_mean_scp
wy2019_november_scp_anomaly = november_scp_wy2019 - november_mean_scp

november_scp_anomaly_stack = array([wy2001_november_scp_anomaly, wy2002_november_scp_anomaly, wy2003_november_scp_anomaly, wy2004_november_scp_anomaly, wy2005_november_scp_anomaly,
                                   wy2006_november_scp_anomaly, wy2007_november_scp_anomaly, wy2008_november_scp_anomaly, wy2009_november_scp_anomaly, wy2010_november_scp_anomaly, 
                                   wy2011_november_scp_anomaly, wy2012_november_scp_anomaly, wy2013_november_scp_anomaly, wy2014_november_scp_anomaly, wy2015_november_scp_anomaly,
                                   wy2016_november_scp_anomaly, wy2017_november_scp_anomaly, wy2018_november_scp_anomaly, wy2019_november_scp_anomaly, november_mean_scp])

november_scp_anomaly_stack[november_scp_anomaly_stack == 0] = np.nan

CreateGeoTiffStack('stacked_tiffs/november_scp_anomaly_stack.tif', november_scp_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/november_scp_anomaly_stack.tif'

In [70]:
# december snow cover percent anomaly
wy2001_december_scp_anomaly = december_scp_wy2001 - december_mean_scp
wy2002_december_scp_anomaly = december_scp_wy2002 - december_mean_scp
wy2003_december_scp_anomaly = december_scp_wy2003 - december_mean_scp
wy2004_december_scp_anomaly = december_scp_wy2004 - december_mean_scp
wy2005_december_scp_anomaly = december_scp_wy2005 - december_mean_scp
wy2006_december_scp_anomaly = december_scp_wy2006 - december_mean_scp
wy2007_december_scp_anomaly = december_scp_wy2007 - december_mean_scp
wy2008_december_scp_anomaly = december_scp_wy2008 - december_mean_scp
wy2009_december_scp_anomaly = december_scp_wy2009 - december_mean_scp
wy2010_december_scp_anomaly = december_scp_wy2010 - december_mean_scp
wy2011_december_scp_anomaly = december_scp_wy2011 - december_mean_scp
wy2012_december_scp_anomaly = december_scp_wy2012 - december_mean_scp
wy2013_december_scp_anomaly = december_scp_wy2013 - december_mean_scp
wy2014_december_scp_anomaly = december_scp_wy2014 - december_mean_scp
wy2015_december_scp_anomaly = december_scp_wy2015 - december_mean_scp
wy2016_december_scp_anomaly = december_scp_wy2016 - december_mean_scp
wy2017_december_scp_anomaly = december_scp_wy2017 - december_mean_scp
wy2018_december_scp_anomaly = december_scp_wy2018 - december_mean_scp
wy2019_december_scp_anomaly = december_scp_wy2019 - december_mean_scp

december_scp_anomaly_stack = array([wy2001_december_scp_anomaly, wy2002_december_scp_anomaly, wy2003_december_scp_anomaly, wy2004_december_scp_anomaly, wy2005_december_scp_anomaly,
                                   wy2006_december_scp_anomaly, wy2007_december_scp_anomaly, wy2008_december_scp_anomaly, wy2009_december_scp_anomaly, wy2010_december_scp_anomaly, 
                                   wy2011_december_scp_anomaly, wy2012_december_scp_anomaly, wy2013_december_scp_anomaly, wy2014_december_scp_anomaly, wy2015_december_scp_anomaly,
                                   wy2016_december_scp_anomaly, wy2017_december_scp_anomaly, wy2018_december_scp_anomaly, wy2019_december_scp_anomaly, december_mean_scp])

december_scp_anomaly_stack[december_scp_anomaly_stack == 0] = np.nan
CreateGeoTiffStack('stacked_tiffs/december_scp_anomaly_stack.tif', december_scp_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/december_scp_anomaly_stack.tif'

In [71]:
# wy2001_monthly_scp_anomaly_stack = array([wy2001_january_scp_anomaly, wy2001_february_scp_anomaly, wy2001_march_scp_anomaly, wy2001_april_scp_anomaly,
#                                          wy2001_may_scp_anomaly, wy2001_june_scp_anomaly, wy2001_july_scp_anomaly, wy2001_august_scp_anomaly, wy2001_september_scp_anomaly, 
#                                          wy2001_october_scp_anomaly, wy2001_november_scp_anomaly, wy2001_december_scp_anomaly])

# wy2001_monthly_scp_anomaly_stack[wy2001_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2001_monthly_scp_anomaly_stack.tif', wy2001_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [72]:
# wy2002_monthly_scp_anomaly_stack = array([wy2002_january_scp_anomaly, wy2002_february_scp_anomaly, wy2002_march_scp_anomaly, wy2002_april_scp_anomaly,
#                                          wy2002_may_scp_anomaly, wy2002_june_scp_anomaly, wy2002_july_scp_anomaly, wy2002_august_scp_anomaly, wy2002_september_scp_anomaly, 
#                                          wy2002_october_scp_anomaly, wy2002_november_scp_anomaly, wy2002_december_scp_anomaly])

# wy2002_monthly_scp_anomaly_stack[wy2002_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2002_monthly_scp_anomaly_stack.tif', wy2002_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [73]:
# wy2003_monthly_scp_anomaly_stack = array([wy2003_january_scp_anomaly, wy2003_february_scp_anomaly, wy2003_march_scp_anomaly, wy2003_april_scp_anomaly,
#                                          wy2003_may_scp_anomaly, wy2003_june_scp_anomaly, wy2003_july_scp_anomaly, wy2003_august_scp_anomaly, wy2003_september_scp_anomaly, 
#                                          wy2003_october_scp_anomaly, wy2003_november_scp_anomaly, wy2003_december_scp_anomaly])

# wy2003_monthly_scp_anomaly_stack[wy2003_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2003_monthly_scp_anomaly_stack.tif', wy2003_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [74]:
# wy2004_monthly_scp_anomaly_stack = array([wy2004_january_scp_anomaly, wy2004_february_scp_anomaly, wy2004_march_scp_anomaly, wy2004_april_scp_anomaly,
#                                          wy2004_may_scp_anomaly, wy2004_june_scp_anomaly, wy2004_july_scp_anomaly, wy2004_august_scp_anomaly, wy2004_september_scp_anomaly, 
#                                          wy2004_october_scp_anomaly, wy2004_november_scp_anomaly, wy2004_december_scp_anomaly])

# wy2004_monthly_scp_anomaly_stack[wy2004_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2004_monthly_scp_anomaly_stack.tif', wy2004_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [75]:
# wy2005_monthly_scp_anomaly_stack = array([wy2005_january_scp_anomaly, wy2005_february_scp_anomaly, wy2005_march_scp_anomaly, wy2005_april_scp_anomaly,
#                                          wy2005_may_scp_anomaly, wy2005_june_scp_anomaly, wy2005_july_scp_anomaly, wy2005_august_scp_anomaly, wy2005_september_scp_anomaly, 
#                                          wy2005_october_scp_anomaly, wy2005_november_scp_anomaly, wy2005_december_scp_anomaly])

# wy2005_monthly_scp_anomaly_stack[wy2005_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2005_monthly_scp_anomaly_stack.tif', wy2005_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [163]:
# wy2006_monthly_scp_anomaly_stack = array([wy2006_january_scp_anomaly, wy2006_february_scp_anomaly, wy2006_march_scp_anomaly, wy2006_april_scp_anomaly,
#                                          wy2006_may_scp_anomaly, wy2006_june_scp_anomaly, wy2006_july_scp_anomaly, wy2006_august_scp_anomaly, wy2006_september_scp_anomaly, 
#                                          wy2006_october_scp_anomaly, wy2006_november_scp_anomaly, wy2006_december_scp_anomaly])

# wy2006_monthly_scp_anomaly_stack[wy2006_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2006_monthly_scp_anomaly_stack.tif', wy2006_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [77]:
# wy2007_monthly_scp_anomaly_stack = array([wy2007_january_scp_anomaly, wy2007_february_scp_anomaly, wy2007_march_scp_anomaly, wy2007_april_scp_anomaly,
#                                          wy2007_may_scp_anomaly, wy2007_june_scp_anomaly, wy2007_july_scp_anomaly, wy2007_august_scp_anomaly, wy2007_september_scp_anomaly, 
#                                          wy2007_october_scp_anomaly, wy2007_november_scp_anomaly, wy2007_december_scp_anomaly])

# wy2006_monthly_scp_anomaly_stack[wy2006_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2007_monthly_scp_anomaly_stack.tif', wy2007_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [78]:
# wy2008_monthly_scp_anomaly_stack = array([wy2008_january_scp_anomaly, wy2008_february_scp_anomaly, wy2008_march_scp_anomaly, wy2008_april_scp_anomaly,
#                                          wy2008_may_scp_anomaly, wy2008_june_scp_anomaly, wy2008_july_scp_anomaly, wy2008_august_scp_anomaly, wy2008_september_scp_anomaly, 
#                                          wy2008_october_scp_anomaly, wy2008_november_scp_anomaly, wy2008_december_scp_anomaly])

# wy2008_monthly_scp_anomaly_stack[wy2008_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2008_monthly_scp_anomaly_stack.tif', wy2008_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [79]:
# wy2009_monthly_scp_anomaly_stack = array([wy2009_january_scp_anomaly, wy2009_february_scp_anomaly, wy2009_march_scp_anomaly, wy2009_april_scp_anomaly,
#                                          wy2009_may_scp_anomaly, wy2009_june_scp_anomaly, wy2009_july_scp_anomaly, wy2009_august_scp_anomaly, wy2009_september_scp_anomaly, 
#                                          wy2009_october_scp_anomaly, wy2009_november_scp_anomaly, wy2009_december_scp_anomaly])

# wy2009_monthly_scp_anomaly_stack[wy2009_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2009_monthly_scp_anomaly_stack.tif', wy2009_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [80]:
# wy2010_monthly_scp_anomaly_stack = array([wy2010_january_scp_anomaly, wy2010_february_scp_anomaly, wy2010_march_scp_anomaly, wy2010_april_scp_anomaly,
#                                          wy2010_may_scp_anomaly, wy2010_june_scp_anomaly, wy2010_july_scp_anomaly, wy2010_august_scp_anomaly, wy2010_september_scp_anomaly, 
#                                          wy2010_october_scp_anomaly, wy2010_november_scp_anomaly, wy2010_december_scp_anomaly])

# wy2010_monthly_scp_anomaly_stack[wy2010_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2010_monthly_scp_anomaly_stack.tif', wy2010_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [81]:
# wy2011_monthly_scp_anomaly_stack = array([wy2011_january_scp_anomaly, wy2011_february_scp_anomaly, wy2011_march_scp_anomaly, wy2011_april_scp_anomaly,
#                                          wy2011_may_scp_anomaly, wy2011_june_scp_anomaly, wy2011_july_scp_anomaly, wy2011_august_scp_anomaly, wy2011_september_scp_anomaly, 
#                                          wy2011_october_scp_anomaly, wy2011_november_scp_anomaly, wy2011_december_scp_anomaly])

# wy2011_monthly_scp_anomaly_stack[wy2011_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2011_monthly_scp_anomaly_stack.tif', wy2011_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [82]:
# wy2012_monthly_scp_anomaly_stack = array([wy2012_january_scp_anomaly, wy2012_february_scp_anomaly, wy2012_march_scp_anomaly, wy2012_april_scp_anomaly,
#                                          wy2012_may_scp_anomaly, wy2012_june_scp_anomaly, wy2012_july_scp_anomaly, wy2012_august_scp_anomaly, wy2012_september_scp_anomaly, 
#                                          wy2012_october_scp_anomaly, wy2012_november_scp_anomaly, wy2012_december_scp_anomaly])

# wy2012_monthly_scp_anomaly_stack[wy2012_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2012_monthly_scp_anomaly_stack.tif', wy2012_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [83]:
# wy2013_monthly_scp_anomaly_stack = array([wy2013_january_scp_anomaly, wy2013_february_scp_anomaly, wy2013_march_scp_anomaly, wy2013_april_scp_anomaly,
#                                          wy2013_may_scp_anomaly, wy2013_june_scp_anomaly, wy2013_july_scp_anomaly, wy2013_august_scp_anomaly, wy2013_september_scp_anomaly, 
#                                          wy2013_october_scp_anomaly, wy2013_november_scp_anomaly, wy2013_december_scp_anomaly])

# wy2013_monthly_scp_anomaly_stack[wy2013_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2013_monthly_scp_anomaly_stack.tif', wy2013_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [84]:
# wy2014_monthly_scp_anomaly_stack = array([wy2014_january_scp_anomaly, wy2014_february_scp_anomaly, wy2014_march_scp_anomaly, wy2014_april_scp_anomaly,
#                                          wy2014_may_scp_anomaly, wy2014_june_scp_anomaly, wy2014_july_scp_anomaly, wy2014_august_scp_anomaly, wy2014_september_scp_anomaly, 
#                                          wy2014_october_scp_anomaly, wy2014_november_scp_anomaly, wy2014_december_scp_anomaly])

# wy2014_monthly_scp_anomaly_stack[wy2014_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2014_monthly_scp_anomaly_stack.tif', wy2014_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [85]:
# wy2015_monthly_scp_anomaly_stack = array([wy2015_january_scp_anomaly, wy2015_february_scp_anomaly, wy2015_march_scp_anomaly, wy2015_april_scp_anomaly,
#                                          wy2015_may_scp_anomaly, wy2015_june_scp_anomaly, wy2015_july_scp_anomaly, wy2015_august_scp_anomaly, wy2015_september_scp_anomaly, 
#                                          wy2015_october_scp_anomaly, wy2015_november_scp_anomaly, wy2015_december_scp_anomaly])

# wy2015_monthly_scp_anomaly_stack[wy2015_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2015_monthly_scp_anomaly_stack.tif', wy2015_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [86]:
# wy2016_monthly_scp_anomaly_stack = array([wy2016_january_scp_anomaly, wy2016_february_scp_anomaly, wy2016_march_scp_anomaly, wy2016_april_scp_anomaly,
#                                          wy2016_may_scp_anomaly, wy2016_june_scp_anomaly, wy2016_july_scp_anomaly, wy2016_august_scp_anomaly, wy2016_september_scp_anomaly, 
#                                          wy2016_october_scp_anomaly, wy2016_november_scp_anomaly, wy2016_december_scp_anomaly])

# wy2016_monthly_scp_anomaly_stack[wy2016_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2016_monthly_scp_anomaly_stack.tif', wy2016_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [87]:
# wy2017_monthly_scp_anomaly_stack = array([wy2017_january_scp_anomaly, wy2017_february_scp_anomaly, wy2017_march_scp_anomaly, wy2017_april_scp_anomaly,
#                                          wy2017_may_scp_anomaly, wy2017_june_scp_anomaly, wy2017_july_scp_anomaly, wy2017_august_scp_anomaly, wy2017_september_scp_anomaly, 
#                                          wy2017_october_scp_anomaly, wy2017_november_scp_anomaly, wy2017_december_scp_anomaly])

# wy2017_monthly_scp_anomaly_stack[wy2017_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2017_monthly_scp_anomaly_stack.tif', wy2017_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [88]:
# wy2018_monthly_scp_anomaly_stack = array([wy2018_january_scp_anomaly, wy2018_february_scp_anomaly, wy2018_march_scp_anomaly, wy2018_april_scp_anomaly,
#                                          wy2018_may_scp_anomaly, wy2018_june_scp_anomaly, wy2018_july_scp_anomaly, wy2018_august_scp_anomaly, wy2018_september_scp_anomaly, 
#                                          wy2018_october_scp_anomaly, wy2018_november_scp_anomaly, wy2018_december_scp_anomaly])

# wy2018_monthly_scp_anomaly_stack[wy2018_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2018_monthly_scp_anomaly_stack.tif', wy2018_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

In [89]:
# wy2019_monthly_scp_anomaly_stack = array([wy2019_january_scp_anomaly, wy2019_february_scp_anomaly, wy2019_march_scp_anomaly, wy2019_april_scp_anomaly,
#                                          wy2019_may_scp_anomaly, wy2019_june_scp_anomaly, wy2019_july_scp_anomaly, wy2019_august_scp_anomaly, wy2019_september_scp_anomaly, 
#                                          wy2019_october_scp_anomaly, wy2019_november_scp_anomaly, wy2019_december_scp_anomaly])

# wy2019_monthly_scp_anomaly_stack[wy2019_monthly_scp_anomaly_stack == 0] = np.nan

# CreateGeoTiffStack('stacked_tiffs/wy2019_monthly_scp_anomaly_stack.tif', wy2019_monthly_scp_anomaly_stack, ReferencingMatrix, Projection)

# Albedo
## Annual
### Mean

In [90]:
# Trying to remove fill values before averaging
def AnnualMeanAlbedo(filepath):
    albedo = rioxarray.open_rasterio(filepath)
    albedo = albedo.where(albedo != 65535)
    annual_mean_albedo = albedo.isel(band=slice(0, 364)).mean(dim="band")
    annual_mean_albedo_numpy = annual_mean_albedo.to_numpy()
    annual_mean_albedo_numpy = np.transpose(annual_mean_albedo_numpy)
    annual_mean_albedo_numpy = annual_mean_albedo_numpy / 10000
    
    return(annual_mean_albedo_numpy)

In [91]:
warnings.filterwarnings('ignore')

# these are numpy arrays
annual_mean_albedo_2001 = AnnualMeanAlbedo(wy2001_albedo)
annual_mean_albedo_2002 = AnnualMeanAlbedo(wy2002_albedo)
annual_mean_albedo_2003 = AnnualMeanAlbedo(wy2003_albedo)
annual_mean_albedo_2004 = AnnualMeanAlbedo(wy2004_albedo)
annual_mean_albedo_2005 = AnnualMeanAlbedo(wy2005_albedo)
annual_mean_albedo_2006 = AnnualMeanAlbedo(wy2006_albedo)
annual_mean_albedo_2007 = AnnualMeanAlbedo(wy2007_albedo)
annual_mean_albedo_2008 = AnnualMeanAlbedo(wy2008_albedo)
annual_mean_albedo_2009 = AnnualMeanAlbedo(wy2009_albedo)
annual_mean_albedo_2010 = AnnualMeanAlbedo(wy2010_albedo)
annual_mean_albedo_2011 = AnnualMeanAlbedo(wy2011_albedo)
annual_mean_albedo_2012 = AnnualMeanAlbedo(wy2012_albedo)
annual_mean_albedo_2013 = AnnualMeanAlbedo(wy2013_albedo)
annual_mean_albedo_2014 = AnnualMeanAlbedo(wy2014_albedo)
annual_mean_albedo_2015 = AnnualMeanAlbedo(wy2015_albedo)
annual_mean_albedo_2016 = AnnualMeanAlbedo(wy2016_albedo)
annual_mean_albedo_2017 = AnnualMeanAlbedo(wy2017_albedo)
annual_mean_albedo_2018 = AnnualMeanAlbedo(wy2018_albedo)
annual_mean_albedo_2019 = AnnualMeanAlbedo(wy2019_albedo)

In [230]:
mean_annual_albedo_array = np.array([annual_mean_albedo_2001, annual_mean_albedo_2002, annual_mean_albedo_2003, annual_mean_albedo_2004, annual_mean_albedo_2005, 
                      annual_mean_albedo_2006, annual_mean_albedo_2007, annual_mean_albedo_2008, annual_mean_albedo_2009, annual_mean_albedo_2010, 
                      annual_mean_albedo_2011, annual_mean_albedo_2012, annual_mean_albedo_2013, annual_mean_albedo_2014, annual_mean_albedo_2015, 
                      annual_mean_albedo_2016, annual_mean_albedo_2017, annual_mean_albedo_2018, annual_mean_albedo_2010])

mean_annual_albedo = np.nanmean(mean_annual_albedo_array, axis = 0)

In [231]:
annual_mean_albedo_stack = array([annual_mean_albedo_2001, annual_mean_albedo_2002, annual_mean_albedo_2003, annual_mean_albedo_2004, annual_mean_albedo_2005,
                             annual_mean_albedo_2006, annual_mean_albedo_2007, annual_mean_albedo_2008, annual_mean_albedo_2009, annual_mean_albedo_2010, annual_mean_albedo_2011,
                             annual_mean_albedo_2012, annual_mean_albedo_2013, annual_mean_albedo_2014, annual_mean_albedo_2015, annual_mean_albedo_2016, annual_mean_albedo_2017,
                             annual_mean_albedo_2018, annual_mean_albedo_2019, mean_annual_albedo])

CreateGeoTiffStack('stacked_tiffs/annual_mean_albedo_stack.tif', annual_mean_albedo_stack, ReferencingMatrix, Projection)

'stacked_tiffs/annual_mean_albedo_stack.tif'

In [232]:
np.nanmin(annual_mean_albedo_stack)

0.4731642857142857

In [233]:
np.nanmax(annual_mean_albedo_stack)

0.86088

### Anomaly

In [234]:
wy2001_albedo_anomaly = annual_mean_albedo_2001 - mean_annual_albedo
wy2002_albedo_anomaly = annual_mean_albedo_2002 - mean_annual_albedo
wy2003_albedo_anomaly = annual_mean_albedo_2003 - mean_annual_albedo
wy2004_albedo_anomaly = annual_mean_albedo_2004 - mean_annual_albedo
wy2005_albedo_anomaly = annual_mean_albedo_2005 - mean_annual_albedo
wy2006_albedo_anomaly = annual_mean_albedo_2006 - mean_annual_albedo
wy2007_albedo_anomaly = annual_mean_albedo_2007 - mean_annual_albedo
wy2008_albedo_anomaly = annual_mean_albedo_2008 - mean_annual_albedo
wy2009_albedo_anomaly = annual_mean_albedo_2009 - mean_annual_albedo
wy2010_albedo_anomaly = annual_mean_albedo_2010 - mean_annual_albedo
wy2011_albedo_anomaly = annual_mean_albedo_2011 - mean_annual_albedo
wy2012_albedo_anomaly = annual_mean_albedo_2012 - mean_annual_albedo
wy2013_albedo_anomaly = annual_mean_albedo_2013 - mean_annual_albedo
wy2014_albedo_anomaly = annual_mean_albedo_2014 - mean_annual_albedo
wy2015_albedo_anomaly = annual_mean_albedo_2015 - mean_annual_albedo
wy2016_albedo_anomaly = annual_mean_albedo_2016 - mean_annual_albedo
wy2017_albedo_anomaly = annual_mean_albedo_2017 - mean_annual_albedo
wy2018_albedo_anomaly = annual_mean_albedo_2018 - mean_annual_albedo
wy2019_albedo_anomaly = annual_mean_albedo_2019 - mean_annual_albedo

annual_albedo_anomaly_stack = array([wy2001_albedo_anomaly, wy2002_albedo_anomaly, wy2003_albedo_anomaly, wy2004_albedo_anomaly, wy2005_albedo_anomaly,
                                 wy2006_albedo_anomaly, wy2007_albedo_anomaly, wy2008_albedo_anomaly, wy2009_albedo_anomaly, wy2010_albedo_anomaly,
                                 wy2011_albedo_anomaly, wy2012_albedo_anomaly, wy2013_albedo_anomaly, wy2014_albedo_anomaly, wy2015_albedo_anomaly,
                                 wy2016_albedo_anomaly, wy2017_albedo_anomaly, wy2018_albedo_anomaly, wy2019_albedo_anomaly])

CreateGeoTiffStack('stacked_tiffs/annual_albedo_anomaly_stack.tif', annual_albedo_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/annual_albedo_anomaly_stack.tif'

In [235]:
np.nanmin(annual_albedo_anomaly_stack)

-0.25539917611007645

In [236]:
np.nanmax(annual_albedo_anomaly_stack)

0.16670000000000007

## Monthly
### Mean

In [265]:
def MonthlyAlbedo(filepath, start_index, end_index):
    albedo = rioxarray.open_rasterio(filepath)
    albedo = albedo.where(albedo != 65535)
    monthly_mean_albedo = albedo.isel(band=slice(start_index, end_index)).mean(dim="band")
    monthly_mean_albedo_numpy = monthly_mean_albedo.to_numpy()
    monthly_mean_albedo_numpy = np.transpose(monthly_mean_albedo_numpy)
    monthly_mean_albedo_numpy = monthly_mean_albedo_numpy / 10000
    
    return(monthly_mean_albedo_numpy)

In [ ]:
#warnings.filterwarnings('ignore')

# wy2001
january_albedo_wy2001 = MonthlyAlbedo(wy2001_albedo, january_start_index, january_end_index)
february_albedo_wy2001 = MonthlyAlbedo(wy2001_albedo, february_start_index, february_end_index)
march_albedo_wy2001 = MonthlyAlbedo(wy2001_albedo, march_start_index, march_end_index)
april_albedo_wy2001 = MonthlyAlbedo(wy2001_albedo, april_start_index, april_end_index)
may_albedo_wy2001 = MonthlyAlbedo(wy2001_albedo, may_start_index, may_end_index)
june_albedo_wy2001 = MonthlyAlbedo(wy2001_albedo, june_start_index, june_end_index)
july_albedo_wy2001 = MonthlyAlbedo(wy2001_albedo, july_start_index, july_end_index)
august_albedo_wy2001 = MonthlyAlbedo(wy2001_albedo, august_start_index, august_end_index)
september_albedo_wy2001 = MonthlyAlbedo(wy2001_albedo, september_start_index, september_end_index)
october_albedo_wy2001 = MonthlyAlbedo(wy2001_albedo, october_start_index, october_end_index)
november_albedo_wy2001 = MonthlyAlbedo(wy2001_albedo, november_start_index, november_end_index)
december_albedo_wy2001 = MonthlyAlbedo(wy2001_albedo, december_start_index, december_end_index)

# wy2001_monthly_mean_albedo_stack = array([january_albedo_wy2001, february_albedo_wy2001, march_albedo_wy2001, april_albedo_wy2001,
#                                       may_albedo_wy2001, june_albedo_wy2001, july_albedo_wy2001, august_albedo_wy2001, 
#                                        september_albedo_wy2001, october_albedo_wy2001, november_albedo_wy2001, december_albedo_wy2001])

# CreateGeoTiffStack('stacked_tiffs/wy2001_monthly_mean_albedo_stack.tif', wy2001_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [97]:
# wy2002
january_albedo_wy2002 = MonthlyAlbedo(wy2002_albedo, january_start_index, january_end_index)
february_albedo_wy2002 = MonthlyAlbedo(wy2002_albedo, february_start_index, february_end_index)
march_albedo_wy2002 = MonthlyAlbedo(wy2002_albedo, march_start_index, march_end_index)
april_albedo_wy2002 = MonthlyAlbedo(wy2002_albedo, april_start_index, april_end_index)
may_albedo_wy2002 = MonthlyAlbedo(wy2002_albedo, may_start_index, may_end_index)
june_albedo_wy2002 = MonthlyAlbedo(wy2002_albedo, june_start_index, june_end_index)
july_albedo_wy2002 = MonthlyAlbedo(wy2002_albedo, july_start_index, july_end_index)
august_albedo_wy2002 = MonthlyAlbedo(wy2002_albedo, august_start_index, august_end_index)
september_albedo_wy2002 = MonthlyAlbedo(wy2002_albedo, september_start_index, september_end_index)
october_albedo_wy2002 = MonthlyAlbedo(wy2002_albedo, october_start_index, october_end_index)
november_albedo_wy2002 = MonthlyAlbedo(wy2002_albedo, november_start_index, november_end_index)
december_albedo_wy2002 = MonthlyAlbedo(wy2002_albedo, december_start_index, december_end_index)

# wy2002_monthly_mean_albedo_stack = array([january_albedo_wy2002, february_albedo_wy2002, march_albedo_wy2002, april_albedo_wy2002,
#                                       may_albedo_wy2002, june_albedo_wy2002, july_albedo_wy2002, august_albedo_wy2002, 
#                                        september_albedo_wy2002, october_albedo_wy2002, november_albedo_wy2002, december_albedo_wy2002])

# CreateGeoTiffStack('stacked_tiffs/wy2002_monthly_mean_albedo_stack.tif', wy2002_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [98]:
# wy2003
january_albedo_wy2003 = MonthlyAlbedo(wy2003_albedo, january_start_index, january_end_index)
february_albedo_wy2003 = MonthlyAlbedo(wy2003_albedo, february_start_index, february_end_index)
march_albedo_wy2003 = MonthlyAlbedo(wy2003_albedo, march_start_index, march_end_index)
april_albedo_wy2003 = MonthlyAlbedo(wy2003_albedo, april_start_index, april_end_index)
may_albedo_wy2003 = MonthlyAlbedo(wy2003_albedo, may_start_index, may_end_index)
june_albedo_wy2003 = MonthlyAlbedo(wy2003_albedo, june_start_index, june_end_index)
july_albedo_wy2003 = MonthlyAlbedo(wy2003_albedo, july_start_index, july_end_index)
august_albedo_wy2003 = MonthlyAlbedo(wy2003_albedo, august_start_index, august_end_index)
september_albedo_wy2003 = MonthlyAlbedo(wy2003_albedo, september_start_index, september_end_index)
october_albedo_wy2003 = MonthlyAlbedo(wy2003_albedo, october_start_index, october_end_index)
november_albedo_wy2003 = MonthlyAlbedo(wy2003_albedo, november_start_index, november_end_index)
december_albedo_wy2003 = MonthlyAlbedo(wy2003_albedo, december_start_index, december_end_index)

# wy2003_monthly_mean_albedo_stack = array([january_albedo_wy2003, february_albedo_wy2003, march_albedo_wy2003, april_albedo_wy2003,
#                                       may_albedo_wy2003, june_albedo_wy2003, july_albedo_wy2003, august_albedo_wy2003, 
#                                        september_albedo_wy2003, october_albedo_wy2003, november_albedo_wy2003, december_albedo_wy2003])

# CreateGeoTiffStack('stacked_tiffs/wy2003_monthly_mean_albedo_stack.tif', wy2003_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [99]:
#warnings.filterwarnings('ignore')

# wy2004
january_albedo_wy2004 = MonthlyAlbedo(wy2004_albedo, january_start_index, january_end_index)
february_albedo_wy2004 = MonthlyAlbedo(wy2004_albedo, february_leapyear_start_index, february_leapyear_end_index)
march_albedo_wy2004 = MonthlyAlbedo(wy2004_albedo, march_leapyear_start_index, march_leapyear_end_index)
april_albedo_wy2004 = MonthlyAlbedo(wy2004_albedo, april_leapyear_start_index, april_leapyear_end_index)
may_albedo_wy2004 = MonthlyAlbedo(wy2004_albedo, may_leapyear_start_index, may_leapyear_end_index)
june_albedo_wy2004 = MonthlyAlbedo(wy2004_albedo, june_leapyear_start_index, june_leapyear_end_index)
july_albedo_wy2004 = MonthlyAlbedo(wy2004_albedo, july_leapyear_start_index, july_leapyear_end_index)
august_albedo_wy2004 = MonthlyAlbedo(wy2004_albedo, august_leapyear_start_index, august_leapyear_end_index)
september_albedo_wy2004 = MonthlyAlbedo(wy2004_albedo, september_leapyear_start_index, september_leapyear_end_index)
october_albedo_wy2004 = MonthlyAlbedo(wy2004_albedo, october_start_index, october_end_index)
november_albedo_wy2004 = MonthlyAlbedo(wy2004_albedo, november_start_index, november_end_index)
december_albedo_wy2004 = MonthlyAlbedo(wy2004_albedo, december_start_index, december_end_index)

# wy2004_monthly_mean_albedo_stack = array([january_albedo_wy2004, february_albedo_wy2004, march_albedo_wy2004, april_albedo_wy2004,
#                                       may_albedo_wy2004, june_albedo_wy2004, july_albedo_wy2004, august_albedo_wy2004, 
#                                        september_albedo_wy2004, october_albedo_wy2004, november_albedo_wy2004, december_albedo_wy2004])

# CreateGeoTiffStack('stacked_tiffs/wy2004_monthly_mean_albedo_stack.tif', wy2004_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [100]:
# wy2005
january_albedo_wy2005 = MonthlyAlbedo(wy2005_albedo, january_start_index, january_end_index)
february_albedo_wy2005 = MonthlyAlbedo(wy2005_albedo, february_start_index, february_end_index)
march_albedo_wy2005 = MonthlyAlbedo(wy2005_albedo, march_start_index, march_end_index)
april_albedo_wy2005 = MonthlyAlbedo(wy2005_albedo, april_start_index, april_end_index)
may_albedo_wy2005 = MonthlyAlbedo(wy2005_albedo, may_start_index, may_end_index)
june_albedo_wy2005 = MonthlyAlbedo(wy2005_albedo, june_start_index, june_end_index)
july_albedo_wy2005 = MonthlyAlbedo(wy2005_albedo, july_start_index, july_end_index)
august_albedo_wy2005 = MonthlyAlbedo(wy2005_albedo, august_start_index, august_end_index)
september_albedo_wy2005 = MonthlyAlbedo(wy2005_albedo, september_start_index, september_end_index)
october_albedo_wy2005 = MonthlyAlbedo(wy2005_albedo, october_start_index, october_end_index)
november_albedo_wy2005 = MonthlyAlbedo(wy2005_albedo, november_start_index, november_end_index)
december_albedo_wy2005 = MonthlyAlbedo(wy2005_albedo, december_start_index, december_end_index)

# wy2005_monthly_mean_albedo_stack = array([january_albedo_wy2005, february_albedo_wy2005, march_albedo_wy2005, april_albedo_wy2005,
#                                       may_albedo_wy2005, june_albedo_wy2005, july_albedo_wy2005, august_albedo_wy2005, 
#                                        september_albedo_wy2005, october_albedo_wy2005, november_albedo_wy2005, december_albedo_wy2005])

# CreateGeoTiffStack('stacked_tiffs/wy2005_monthly_mean_albedo_stack.tif', wy2005_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [101]:
# wy2006
january_albedo_wy2006 = MonthlyAlbedo(wy2006_albedo, january_start_index, january_end_index)
february_albedo_wy2006 = MonthlyAlbedo(wy2006_albedo, february_start_index, february_end_index)
march_albedo_wy2006 = MonthlyAlbedo(wy2006_albedo, march_start_index, march_end_index)
april_albedo_wy2006 = MonthlyAlbedo(wy2006_albedo, april_start_index, april_end_index)
may_albedo_wy2006 = MonthlyAlbedo(wy2006_albedo, may_start_index, may_end_index)
june_albedo_wy2006 = MonthlyAlbedo(wy2006_albedo, june_start_index, june_end_index)
july_albedo_wy2006 = MonthlyAlbedo(wy2006_albedo, july_start_index, july_end_index)
august_albedo_wy2006 = MonthlyAlbedo(wy2006_albedo, august_start_index, august_end_index)
september_albedo_wy2006 = MonthlyAlbedo(wy2006_albedo, september_start_index, september_end_index)
october_albedo_wy2006 = MonthlyAlbedo(wy2006_albedo, october_start_index, october_end_index)
november_albedo_wy2006 = MonthlyAlbedo(wy2006_albedo, november_start_index, november_end_index)
december_albedo_wy2006 = MonthlyAlbedo(wy2006_albedo, december_start_index, december_end_index)

# wy2006_monthly_mean_albedo_stack = array([january_albedo_wy2006, february_albedo_wy2006, march_albedo_wy2006, april_albedo_wy2006,
#                                       may_albedo_wy2006, june_albedo_wy2006, july_albedo_wy2006, august_albedo_wy2006, 
#                                        september_albedo_wy2006, october_albedo_wy2006, november_albedo_wy2006, december_albedo_wy2006])

# CreateGeoTiffStack('stacked_tiffs/wy2006_monthly_mean_albedo_stack.tif', wy2006_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [102]:
# wy2007
january_albedo_wy2007 = MonthlyAlbedo(wy2007_albedo, january_start_index, january_end_index)
february_albedo_wy2007 = MonthlyAlbedo(wy2007_albedo, february_start_index, february_end_index)
march_albedo_wy2007 = MonthlyAlbedo(wy2007_albedo, march_start_index, march_end_index)
april_albedo_wy2007 = MonthlyAlbedo(wy2007_albedo, april_start_index, april_end_index)
may_albedo_wy2007 = MonthlyAlbedo(wy2007_albedo, may_start_index, may_end_index)
june_albedo_wy2007 = MonthlyAlbedo(wy2007_albedo, june_start_index, june_end_index)
july_albedo_wy2007 = MonthlyAlbedo(wy2007_albedo, july_start_index, july_end_index)
august_albedo_wy2007 = MonthlyAlbedo(wy2007_albedo, august_start_index, august_end_index)
september_albedo_wy2007 = MonthlyAlbedo(wy2007_albedo, september_start_index, september_end_index)
october_albedo_wy2007 = MonthlyAlbedo(wy2007_albedo, october_start_index, october_end_index)
november_albedo_wy2007 = MonthlyAlbedo(wy2007_albedo, november_start_index, november_end_index)
december_albedo_wy2007 = MonthlyAlbedo(wy2007_albedo, december_start_index, december_end_index)

# wy2007_monthly_mean_albedo_stack = array([january_albedo_wy2007, february_albedo_wy2007, march_albedo_wy2007, april_albedo_wy2007,
#                                       may_albedo_wy2007, june_albedo_wy2007, july_albedo_wy2007, august_albedo_wy2007, 
#                                        september_albedo_wy2007, october_albedo_wy2007, november_albedo_wy2007, december_albedo_wy2007])

# CreateGeoTiffStack('stacked_tiffs/wy2007_monthly_mean_albedo_stack.tif', wy2007_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [103]:
# wy2008
january_albedo_wy2008 = MonthlyAlbedo(wy2008_albedo, january_start_index, january_end_index)
february_albedo_wy2008 = MonthlyAlbedo(wy2008_albedo, february_leapyear_start_index, february_leapyear_end_index)
march_albedo_wy2008 = MonthlyAlbedo(wy2008_albedo, march_leapyear_start_index, march_leapyear_end_index)
april_albedo_wy2008 = MonthlyAlbedo(wy2008_albedo, april_leapyear_start_index, april_leapyear_end_index)
may_albedo_wy2008 = MonthlyAlbedo(wy2008_albedo, may_leapyear_start_index, may_leapyear_end_index)
june_albedo_wy2008 = MonthlyAlbedo(wy2008_albedo, june_leapyear_start_index, june_leapyear_end_index)
july_albedo_wy2008 = MonthlyAlbedo(wy2008_albedo, july_leapyear_start_index, july_leapyear_end_index)
august_albedo_wy2008 = MonthlyAlbedo(wy2008_albedo, august_leapyear_start_index, august_leapyear_end_index)
september_albedo_wy2008 = MonthlyAlbedo(wy2008_albedo, september_leapyear_start_index, september_leapyear_end_index)
october_albedo_wy2008 = MonthlyAlbedo(wy2008_albedo, october_start_index, october_end_index)
november_albedo_wy2008 = MonthlyAlbedo(wy2008_albedo, november_start_index, november_end_index)
december_albedo_wy2008 = MonthlyAlbedo(wy2008_albedo, december_start_index, december_end_index)

# wy2008_monthly_mean_albedo_stack = array([january_albedo_wy2008, february_albedo_wy2008, march_albedo_wy2008, april_albedo_wy2008,
#                                       may_albedo_wy2008, june_albedo_wy2008, july_albedo_wy2008, august_albedo_wy2008, 
#                                        september_albedo_wy2008, october_albedo_wy2008, november_albedo_wy2008, december_albedo_wy2008])

# CreateGeoTiffStack('stacked_tiffs/wy2008_monthly_mean_albedo_stack.tif', wy2008_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [104]:
# wy2009
january_albedo_wy2009 = MonthlyAlbedo(wy2009_albedo, january_start_index, january_end_index)
february_albedo_wy2009 = MonthlyAlbedo(wy2009_albedo, february_start_index, february_end_index)
march_albedo_wy2009 = MonthlyAlbedo(wy2009_albedo, march_start_index, march_end_index)
april_albedo_wy2009 = MonthlyAlbedo(wy2009_albedo, april_start_index, april_end_index)
may_albedo_wy2009 = MonthlyAlbedo(wy2009_albedo, may_start_index, may_end_index)
june_albedo_wy2009 = MonthlyAlbedo(wy2009_albedo, june_start_index, june_end_index)
july_albedo_wy2009 = MonthlyAlbedo(wy2009_albedo, july_start_index, july_end_index)
august_albedo_wy2009 = MonthlyAlbedo(wy2009_albedo, august_start_index, august_end_index)
september_albedo_wy2009 = MonthlyAlbedo(wy2009_albedo, september_start_index, september_end_index)
october_albedo_wy2009 = MonthlyAlbedo(wy2009_albedo, october_start_index, october_end_index)
november_albedo_wy2009 = MonthlyAlbedo(wy2009_albedo, november_start_index, november_end_index)
december_albedo_wy2009 = MonthlyAlbedo(wy2009_albedo, december_start_index, december_end_index)

# wy2009_monthly_mean_albedo_stack = array([january_albedo_wy2009, february_albedo_wy2009, march_albedo_wy2009, april_albedo_wy2009,
#                                       may_albedo_wy2009, june_albedo_wy2009, july_albedo_wy2009, august_albedo_wy2009, 
#                                        september_albedo_wy2009, october_albedo_wy2009, november_albedo_wy2009, december_albedo_wy2009])

# CreateGeoTiffStack('stacked_tiffs/wy2009_monthly_mean_albedo_stack.tif', wy2009_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [105]:
# wy2010
january_albedo_wy2010 = MonthlyAlbedo(wy2010_albedo, january_start_index, january_end_index)
february_albedo_wy2010 = MonthlyAlbedo(wy2010_albedo, february_start_index, february_end_index)
march_albedo_wy2010 = MonthlyAlbedo(wy2010_albedo, march_start_index, march_end_index)
april_albedo_wy2010 = MonthlyAlbedo(wy2010_albedo, april_start_index, april_end_index)
may_albedo_wy2010 = MonthlyAlbedo(wy2010_albedo, may_start_index, may_end_index)
june_albedo_wy2010 = MonthlyAlbedo(wy2010_albedo, june_start_index, june_end_index)
july_albedo_wy2010 = MonthlyAlbedo(wy2010_albedo, july_start_index, july_end_index)
august_albedo_wy2010 = MonthlyAlbedo(wy2010_albedo, august_start_index, august_end_index)
september_albedo_wy2010 = MonthlyAlbedo(wy2010_albedo, september_start_index, september_end_index)
october_albedo_wy2010 = MonthlyAlbedo(wy2010_albedo, october_start_index, october_end_index)
november_albedo_wy2010 = MonthlyAlbedo(wy2010_albedo, november_start_index, november_end_index)
december_albedo_wy2010 = MonthlyAlbedo(wy2010_albedo, december_start_index, december_end_index)

# wy2010_monthly_mean_albedo_stack = array([january_albedo_wy2010, february_albedo_wy2010, march_albedo_wy2010, april_albedo_wy2010,
#                                       may_albedo_wy2010, june_albedo_wy2010, july_albedo_wy2010, august_albedo_wy2010, 
#                                        september_albedo_wy2010, october_albedo_wy2010, november_albedo_wy2010, december_albedo_wy2010])

# CreateGeoTiffStack('stacked_tiffs/wy2010_monthly_mean_albedo_stack.tif', wy2010_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [106]:
# wy2011
january_albedo_wy2011 = MonthlyAlbedo(wy2011_albedo, january_start_index, january_end_index)
february_albedo_wy2011 = MonthlyAlbedo(wy2011_albedo, february_start_index, february_end_index)
march_albedo_wy2011 = MonthlyAlbedo(wy2011_albedo, march_start_index, march_end_index)
april_albedo_wy2011 = MonthlyAlbedo(wy2011_albedo, april_start_index, april_end_index)
may_albedo_wy2011 = MonthlyAlbedo(wy2011_albedo, may_start_index, may_end_index)
june_albedo_wy2011 = MonthlyAlbedo(wy2011_albedo, june_start_index, june_end_index)
july_albedo_wy2011 = MonthlyAlbedo(wy2011_albedo, july_start_index, july_end_index)
august_albedo_wy2011 = MonthlyAlbedo(wy2011_albedo, august_start_index, august_end_index)
september_albedo_wy2011 = MonthlyAlbedo(wy2011_albedo, september_start_index, september_end_index)
october_albedo_wy2011 = MonthlyAlbedo(wy2011_albedo, october_start_index, october_end_index)
november_albedo_wy2011 = MonthlyAlbedo(wy2011_albedo, november_start_index, november_end_index)
december_albedo_wy2011 = MonthlyAlbedo(wy2011_albedo, december_start_index, december_end_index)

# wy2011_monthly_mean_albedo_stack = array([january_albedo_wy2011, february_albedo_wy2011, march_albedo_wy2011, april_albedo_wy2011,
#                                       may_albedo_wy2011, june_albedo_wy2011, july_albedo_wy2011, august_albedo_wy2011, 
#                                        september_albedo_wy2011, october_albedo_wy2011, november_albedo_wy2011, december_albedo_wy2011])

# CreateGeoTiffStack('stacked_tiffs/wy2011_monthly_mean_albedo_stack.tif', wy2011_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [107]:
# wy2012
january_albedo_wy2012 = MonthlyAlbedo(wy2012_albedo, january_start_index, january_end_index)
february_albedo_wy2012 = MonthlyAlbedo(wy2012_albedo, february_leapyear_start_index, february_leapyear_end_index)
march_albedo_wy2012 = MonthlyAlbedo(wy2012_albedo, march_leapyear_start_index, march_leapyear_end_index)
april_albedo_wy2012 = MonthlyAlbedo(wy2012_albedo, april_leapyear_start_index, april_leapyear_end_index)
may_albedo_wy2012 = MonthlyAlbedo(wy2012_albedo, may_leapyear_start_index, may_leapyear_end_index)
june_albedo_wy2012 = MonthlyAlbedo(wy2012_albedo, june_leapyear_start_index, june_leapyear_end_index)
july_albedo_wy2012 = MonthlyAlbedo(wy2012_albedo, july_leapyear_start_index, july_leapyear_end_index)
august_albedo_wy2012 = MonthlyAlbedo(wy2012_albedo, august_leapyear_start_index, august_leapyear_end_index)
september_albedo_wy2012 = MonthlyAlbedo(wy2012_albedo, september_leapyear_start_index, september_leapyear_end_index)
october_albedo_wy2012 = MonthlyAlbedo(wy2012_albedo, october_start_index, october_end_index)
november_albedo_wy2012 = MonthlyAlbedo(wy2012_albedo, november_start_index, november_end_index)
december_albedo_wy2012 = MonthlyAlbedo(wy2012_albedo, december_start_index, december_end_index)

# wy2012_monthly_mean_albedo_stack = array([january_albedo_wy2012, february_albedo_wy2012, march_albedo_wy2012, april_albedo_wy2012,
#                                       may_albedo_wy2012, june_albedo_wy2012, july_albedo_wy2012, august_albedo_wy2012, 
#                                        september_albedo_wy2012, october_albedo_wy2012, november_albedo_wy2012, december_albedo_wy2012])

# CreateGeoTiffStack('stacked_tiffs/wy2012_monthly_mean_albedo_stack.tif', wy2012_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [108]:
# wy2013
january_albedo_wy2013 = MonthlyAlbedo(wy2013_albedo, january_start_index, january_end_index)
february_albedo_wy2013 = MonthlyAlbedo(wy2013_albedo, february_start_index, february_end_index)
march_albedo_wy2013 = MonthlyAlbedo(wy2013_albedo, march_start_index, march_end_index)
april_albedo_wy2013 = MonthlyAlbedo(wy2013_albedo, april_start_index, april_end_index)
may_albedo_wy2013 = MonthlyAlbedo(wy2013_albedo, may_start_index, may_end_index)
june_albedo_wy2013 = MonthlyAlbedo(wy2013_albedo, june_start_index, june_end_index)
july_albedo_wy2013 = MonthlyAlbedo(wy2013_albedo, july_start_index, july_end_index)
august_albedo_wy2013 = MonthlyAlbedo(wy2013_albedo, august_start_index, august_end_index)
september_albedo_wy2013 = MonthlyAlbedo(wy2013_albedo, september_start_index, september_end_index)
october_albedo_wy2013 = MonthlyAlbedo(wy2013_albedo, october_start_index, october_end_index)
november_albedo_wy2013 = MonthlyAlbedo(wy2013_albedo, november_start_index, november_end_index)
december_albedo_wy2013 = MonthlyAlbedo(wy2013_albedo, december_start_index, december_end_index)

# wy2013_monthly_mean_albedo_stack = array([january_albedo_wy2013, february_albedo_wy2013, march_albedo_wy2013, april_albedo_wy2013,
#                                       may_albedo_wy2013, june_albedo_wy2013, july_albedo_wy2013, august_albedo_wy2013, 
#                                        september_albedo_wy2013, october_albedo_wy2013, november_albedo_wy2013, december_albedo_wy2013])

# CreateGeoTiffStack('stacked_tiffs/wy2013_monthly_mean_albedo_stack.tif', wy2013_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [109]:
# wy2014
january_albedo_wy2014 = MonthlyAlbedo(wy2014_albedo, january_start_index, january_end_index)
february_albedo_wy2014 = MonthlyAlbedo(wy2014_albedo, february_start_index, february_end_index)
march_albedo_wy2014 = MonthlyAlbedo(wy2014_albedo, march_start_index, march_end_index)
april_albedo_wy2014 = MonthlyAlbedo(wy2014_albedo, april_start_index, april_end_index)
may_albedo_wy2014 = MonthlyAlbedo(wy2014_albedo, may_start_index, may_end_index)
june_albedo_wy2014 = MonthlyAlbedo(wy2014_albedo, june_start_index, june_end_index)
july_albedo_wy2014 = MonthlyAlbedo(wy2014_albedo, july_start_index, july_end_index)
august_albedo_wy2014 = MonthlyAlbedo(wy2014_albedo, august_start_index, august_end_index)
september_albedo_wy2014 = MonthlyAlbedo(wy2014_albedo, september_start_index, september_end_index)
october_albedo_wy2014 = MonthlyAlbedo(wy2014_albedo, october_start_index, october_end_index)
november_albedo_wy2014 = MonthlyAlbedo(wy2014_albedo, november_start_index, november_end_index)
december_albedo_wy2014 = MonthlyAlbedo(wy2014_albedo, december_start_index, december_end_index)

# wy2014_monthly_mean_albedo_stack = array([january_albedo_wy2014, february_albedo_wy2014, march_albedo_wy2014, april_albedo_wy2014,
#                                       may_albedo_wy2014, june_albedo_wy2014, july_albedo_wy2014, august_albedo_wy2014, 
#                                        september_albedo_wy2014, october_albedo_wy2014, november_albedo_wy2014, december_albedo_wy2014])

# CreateGeoTiffStack('stacked_tiffs/wy2014_monthly_mean_albedo_stack.tif', wy2014_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [110]:
# wy2015
january_albedo_wy2015 = MonthlyAlbedo(wy2015_albedo, january_start_index, january_end_index)
february_albedo_wy2015 = MonthlyAlbedo(wy2015_albedo, february_start_index, february_end_index)
march_albedo_wy2015 = MonthlyAlbedo(wy2015_albedo, march_start_index, march_end_index)
april_albedo_wy2015 = MonthlyAlbedo(wy2015_albedo, april_start_index, april_end_index)
may_albedo_wy2015 = MonthlyAlbedo(wy2015_albedo, may_start_index, may_end_index)
june_albedo_wy2015 = MonthlyAlbedo(wy2015_albedo, june_start_index, june_end_index)
july_albedo_wy2015 = MonthlyAlbedo(wy2015_albedo, july_start_index, july_end_index)
august_albedo_wy2015 = MonthlyAlbedo(wy2015_albedo, august_start_index, august_end_index)
september_albedo_wy2015 = MonthlyAlbedo(wy2015_albedo, september_start_index, september_end_index)
october_albedo_wy2015 = MonthlyAlbedo(wy2015_albedo, october_start_index, october_end_index)
november_albedo_wy2015 = MonthlyAlbedo(wy2015_albedo, november_start_index, november_end_index)
december_albedo_wy2015 = MonthlyAlbedo(wy2015_albedo, december_start_index, december_end_index)

# wy2015_monthly_mean_albedo_stack = array([january_albedo_wy2015, february_albedo_wy2015, march_albedo_wy2015, april_albedo_wy2015,
#                                       may_albedo_wy2015, june_albedo_wy2015, july_albedo_wy2015, august_albedo_wy2015, 
#                                        september_albedo_wy2015, october_albedo_wy2015, november_albedo_wy2015, december_albedo_wy2015])

# CreateGeoTiffStack('stacked_tiffs/wy2015_monthly_mean_albedo_stack.tif', wy2015_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [111]:
# wy2016
january_albedo_wy2016 = MonthlyAlbedo(wy2016_albedo, january_start_index, january_end_index)
february_albedo_wy2016 = MonthlyAlbedo(wy2016_albedo, february_leapyear_start_index, february_leapyear_end_index)
march_albedo_wy2016 = MonthlyAlbedo(wy2016_albedo, march_leapyear_start_index, march_leapyear_end_index)
april_albedo_wy2016 = MonthlyAlbedo(wy2016_albedo, april_leapyear_start_index, april_leapyear_end_index)
may_albedo_wy2016 = MonthlyAlbedo(wy2016_albedo, may_leapyear_start_index, may_leapyear_end_index)
june_albedo_wy2016 = MonthlyAlbedo(wy2016_albedo, june_leapyear_start_index, june_leapyear_end_index)
july_albedo_wy2016 = MonthlyAlbedo(wy2016_albedo, july_leapyear_start_index, july_leapyear_end_index)
august_albedo_wy2016 = MonthlyAlbedo(wy2016_albedo, august_leapyear_start_index, august_leapyear_end_index)
september_albedo_wy2016 = MonthlyAlbedo(wy2016_albedo, september_leapyear_start_index, september_leapyear_end_index)
october_albedo_wy2016 = MonthlyAlbedo(wy2016_albedo, october_start_index, october_end_index)
november_albedo_wy2016 = MonthlyAlbedo(wy2016_albedo, november_start_index, november_end_index)
december_albedo_wy2016 = MonthlyAlbedo(wy2016_albedo, december_start_index, december_end_index)

# wy2016_monthly_mean_albedo_stack = array([january_albedo_wy2016, february_albedo_wy2016, march_albedo_wy2016, april_albedo_wy2016,
#                                       may_albedo_wy2016, june_albedo_wy2016, july_albedo_wy2016, august_albedo_wy2016, 
#                                        september_albedo_wy2016, october_albedo_wy2016, november_albedo_wy2016, december_albedo_wy2016])

# CreateGeoTiffStack('stacked_tiffs/wy2016_monthly_mean_albedo_stack.tif', wy2016_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [112]:
# wy2017
january_albedo_wy2017 = MonthlyAlbedo(wy2017_albedo, january_start_index, january_end_index)
february_albedo_wy2017 = MonthlyAlbedo(wy2017_albedo, february_start_index, february_end_index)
march_albedo_wy2017 = MonthlyAlbedo(wy2017_albedo, march_start_index, march_end_index)
april_albedo_wy2017 = MonthlyAlbedo(wy2017_albedo, april_start_index, april_end_index)
may_albedo_wy2017 = MonthlyAlbedo(wy2017_albedo, may_start_index, may_end_index)
june_albedo_wy2017 = MonthlyAlbedo(wy2017_albedo, june_start_index, june_end_index)
july_albedo_wy2017 = MonthlyAlbedo(wy2017_albedo, july_start_index, july_end_index)
august_albedo_wy2017 = MonthlyAlbedo(wy2017_albedo, august_start_index, august_end_index)
september_albedo_wy2017 = MonthlyAlbedo(wy2017_albedo, september_start_index, september_end_index)
october_albedo_wy2017 = MonthlyAlbedo(wy2017_albedo, october_start_index, october_end_index)
november_albedo_wy2017 = MonthlyAlbedo(wy2017_albedo, november_start_index, november_end_index)
december_albedo_wy2017 = MonthlyAlbedo(wy2017_albedo, december_start_index, december_end_index)

# wy2017_monthly_mean_albedo_stack = array([january_albedo_wy2017, february_albedo_wy2017, march_albedo_wy2017, april_albedo_wy2017,
#                                       may_albedo_wy2017, june_albedo_wy2017, july_albedo_wy2017, august_albedo_wy2017, 
#                                        september_albedo_wy2017, october_albedo_wy2017, november_albedo_wy2017, december_albedo_wy2017])

# CreateGeoTiffStack('stacked_tiffs/wy2017_monthly_mean_albedo_stack.tif', wy2017_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [113]:
# wy2018
january_albedo_wy2018 = MonthlyAlbedo(wy2018_albedo, january_start_index, january_end_index)
february_albedo_wy2018 = MonthlyAlbedo(wy2018_albedo, february_start_index, february_end_index)
march_albedo_wy2018 = MonthlyAlbedo(wy2018_albedo, march_start_index, march_end_index)
april_albedo_wy2018 = MonthlyAlbedo(wy2018_albedo, april_start_index, april_end_index)
may_albedo_wy2018 = MonthlyAlbedo(wy2018_albedo, may_start_index, may_end_index)
june_albedo_wy2018 = MonthlyAlbedo(wy2018_albedo, june_start_index, june_end_index)
july_albedo_wy2018 = MonthlyAlbedo(wy2018_albedo, july_start_index, july_end_index)
august_albedo_wy2018 = MonthlyAlbedo(wy2018_albedo, august_start_index, august_end_index)
september_albedo_wy2018 = MonthlyAlbedo(wy2018_albedo, september_start_index, september_end_index)
october_albedo_wy2018 = MonthlyAlbedo(wy2018_albedo, october_start_index, october_end_index)
november_albedo_wy2018 = MonthlyAlbedo(wy2018_albedo, november_start_index, november_end_index)
december_albedo_wy2018 = MonthlyAlbedo(wy2018_albedo, december_start_index, december_end_index)

# wy2018_monthly_mean_albedo_stack = array([january_albedo_wy2018, february_albedo_wy2018, march_albedo_wy2018, april_albedo_wy2018,
#                                       may_albedo_wy2018, june_albedo_wy2018, july_albedo_wy2018, august_albedo_wy2018, 
#                                        september_albedo_wy2018, october_albedo_wy2018, november_albedo_wy2018, december_albedo_wy2018])

# CreateGeoTiffStack('stacked_tiffs/wy2018_monthly_mean_albedo_stack.tif', wy2018_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [114]:
# wy2019
january_albedo_wy2019 = MonthlyAlbedo(wy2019_albedo, january_start_index, january_end_index)
february_albedo_wy2019 = MonthlyAlbedo(wy2019_albedo, february_start_index, february_end_index)
march_albedo_wy2019 = MonthlyAlbedo(wy2019_albedo, march_start_index, march_end_index)
april_albedo_wy2019 = MonthlyAlbedo(wy2019_albedo, april_start_index, april_end_index)
may_albedo_wy2019 = MonthlyAlbedo(wy2019_albedo, may_start_index, may_end_index)
june_albedo_wy2019 = MonthlyAlbedo(wy2019_albedo, june_start_index, june_end_index)
july_albedo_wy2019 = MonthlyAlbedo(wy2019_albedo, july_start_index, july_end_index)
august_albedo_wy2019 = MonthlyAlbedo(wy2019_albedo, august_start_index, august_end_index)
september_albedo_wy2019 = MonthlyAlbedo(wy2019_albedo, september_start_index, september_end_index)
october_albedo_wy2019 = MonthlyAlbedo(wy2019_albedo, october_start_index, october_end_index)
november_albedo_wy2019 = MonthlyAlbedo(wy2019_albedo, november_start_index, november_end_index)
december_albedo_wy2019 = MonthlyAlbedo(wy2019_albedo, december_start_index, december_end_index)

# wy2019_monthly_mean_albedo_stack = array([january_albedo_wy2019, february_albedo_wy2019, march_albedo_wy2019, april_albedo_wy2019,
#                                       may_albedo_wy2019, june_albedo_wy2019, july_albedo_wy2019, august_albedo_wy2019, 
#                                        september_albedo_wy2019, october_albedo_wy2019, november_albedo_wy2019, december_albedo_wy2019])

# CreateGeoTiffStack('stacked_tiffs/wy2019_monthly_mean_albedo_stack.tif', wy2019_monthly_mean_albedo_stack, ReferencingMatrix, Projection)

In [115]:
#type(wy2019_monthly_mean_albedo_stack)

In [189]:
january_mean_albedo_array = np.array([january_albedo_wy2001, january_albedo_wy2002, january_albedo_wy2003, january_albedo_wy2004, january_albedo_wy2005, 
                                   january_albedo_wy2006, january_albedo_wy2007, january_albedo_wy2008, january_albedo_wy2009, january_albedo_wy2010, 
                                   january_albedo_wy2011, january_albedo_wy2012, january_albedo_wy2013, january_albedo_wy2014, january_albedo_wy2015, 
                                   january_albedo_wy2016, january_albedo_wy2017, january_albedo_wy2018, january_albedo_wy2019])

january_mean_albedo = np.nanmean(january_mean_albedo_array, axis = 0)

january_mean_albedo_stack = array([january_albedo_wy2001, january_albedo_wy2002, january_albedo_wy2003, january_albedo_wy2004, january_albedo_wy2005,
                                   january_albedo_wy2006, january_albedo_wy2007, january_albedo_wy2008, january_albedo_wy2009, january_albedo_wy2010,
                                   january_albedo_wy2011, january_albedo_wy2012, january_albedo_wy2013, january_albedo_wy2014, january_albedo_wy2015,
                                   january_albedo_wy2016, january_albedo_wy2017, january_albedo_wy2018, january_albedo_wy2019, january_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/january_mean_albedo_stack.tif', january_mean_albedo_stack, ReferencingMatrix, Projection)

'stacked_tiffs/january_mean_albedo_stack.tif'

In [190]:
february_mean_albedo_array = np.array([february_albedo_wy2001, february_albedo_wy2002, february_albedo_wy2003, february_albedo_wy2004, february_albedo_wy2005, 
                                   february_albedo_wy2006, february_albedo_wy2007, february_albedo_wy2008, february_albedo_wy2009, february_albedo_wy2010, 
                                   february_albedo_wy2011, february_albedo_wy2012, february_albedo_wy2013, february_albedo_wy2014, february_albedo_wy2015, 
                                   february_albedo_wy2016, february_albedo_wy2017, february_albedo_wy2018, february_albedo_wy2019])

february_mean_albedo = np.nanmean(february_mean_albedo_array, axis = 0)

february_mean_albedo_stack = array([february_albedo_wy2001, february_albedo_wy2002, february_albedo_wy2003, february_albedo_wy2004, february_albedo_wy2005,
                                   february_albedo_wy2006, february_albedo_wy2007, february_albedo_wy2008, february_albedo_wy2009, february_albedo_wy2010,
                                   february_albedo_wy2011, february_albedo_wy2012, february_albedo_wy2013, february_albedo_wy2014, february_albedo_wy2015,
                                   february_albedo_wy2016, february_albedo_wy2017, february_albedo_wy2018, february_albedo_wy2019, february_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/february_mean_albedo_stack.tif', february_mean_albedo_stack, ReferencingMatrix, Projection)

'stacked_tiffs/february_mean_albedo_stack.tif'

In [191]:
march_mean_albedo_array = np.array([march_albedo_wy2001, march_albedo_wy2002, march_albedo_wy2003, march_albedo_wy2004, march_albedo_wy2005, 
                                   march_albedo_wy2006, march_albedo_wy2007, march_albedo_wy2008, march_albedo_wy2009, march_albedo_wy2010, 
                                   march_albedo_wy2011, march_albedo_wy2012, march_albedo_wy2013, march_albedo_wy2014, march_albedo_wy2015, 
                                   march_albedo_wy2016, march_albedo_wy2017, march_albedo_wy2018, march_albedo_wy2019])

march_mean_albedo = np.nanmean(march_mean_albedo_array, axis = 0)

march_mean_albedo_stack = array([march_albedo_wy2001, march_albedo_wy2002, march_albedo_wy2003, march_albedo_wy2004, march_albedo_wy2005,
                                   march_albedo_wy2006, march_albedo_wy2007, march_albedo_wy2008, march_albedo_wy2009, march_albedo_wy2010,
                                   march_albedo_wy2011, march_albedo_wy2012, march_albedo_wy2013, march_albedo_wy2014, march_albedo_wy2015,
                                   march_albedo_wy2016, march_albedo_wy2017, march_albedo_wy2018, march_albedo_wy2019, march_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/march_mean_albedo_stack.tif', march_mean_albedo_stack, ReferencingMatrix, Projection)

'stacked_tiffs/march_mean_albedo_stack.tif'

In [192]:
april_mean_albedo_array = np.array([april_albedo_wy2001, april_albedo_wy2002, april_albedo_wy2003, april_albedo_wy2004, april_albedo_wy2005, 
                                   april_albedo_wy2006, april_albedo_wy2007, april_albedo_wy2008, april_albedo_wy2009, april_albedo_wy2010, 
                                   april_albedo_wy2011, april_albedo_wy2012, april_albedo_wy2013, april_albedo_wy2014, april_albedo_wy2015, 
                                   april_albedo_wy2016, april_albedo_wy2017, april_albedo_wy2018, april_albedo_wy2019])

april_mean_albedo = np.nanmean(april_mean_albedo_array, axis = 0)

april_mean_albedo_stack = array([april_albedo_wy2001, april_albedo_wy2002, april_albedo_wy2003, april_albedo_wy2004, april_albedo_wy2005,
                                   april_albedo_wy2006, april_albedo_wy2007, april_albedo_wy2008, april_albedo_wy2009, april_albedo_wy2010,
                                   april_albedo_wy2011, april_albedo_wy2012, april_albedo_wy2013, april_albedo_wy2014, april_albedo_wy2015,
                                   april_albedo_wy2016, april_albedo_wy2017, april_albedo_wy2018, april_albedo_wy2019, april_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/april_mean_albedo_stack.tif', april_mean_albedo_stack, ReferencingMatrix, Projection)

'stacked_tiffs/april_mean_albedo_stack.tif'

In [193]:
may_mean_albedo_array = np.array([may_albedo_wy2001, may_albedo_wy2002, may_albedo_wy2003, may_albedo_wy2004, may_albedo_wy2005, 
                                   may_albedo_wy2006, may_albedo_wy2007, may_albedo_wy2008, may_albedo_wy2009, may_albedo_wy2010, 
                                   may_albedo_wy2011, may_albedo_wy2012, may_albedo_wy2013, may_albedo_wy2014, may_albedo_wy2015, 
                                   may_albedo_wy2016, may_albedo_wy2017, may_albedo_wy2018, may_albedo_wy2019])

may_mean_albedo = np.nanmean(may_mean_albedo_array, axis = 0)

may_mean_albedo_stack = array([may_albedo_wy2001, may_albedo_wy2002, may_albedo_wy2003, may_albedo_wy2004, may_albedo_wy2005,
                                   may_albedo_wy2006, may_albedo_wy2007, may_albedo_wy2008, may_albedo_wy2009, may_albedo_wy2010,
                                   may_albedo_wy2011, may_albedo_wy2012, may_albedo_wy2013, may_albedo_wy2014, may_albedo_wy2015,
                                   may_albedo_wy2016, may_albedo_wy2017, may_albedo_wy2018, may_albedo_wy2019, may_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/may_mean_albedo_stack.tif', may_mean_albedo_stack, ReferencingMatrix, Projection)

'stacked_tiffs/may_mean_albedo_stack.tif'

In [194]:
june_mean_albedo_array = np.array([june_albedo_wy2001, june_albedo_wy2002, june_albedo_wy2003, june_albedo_wy2004, june_albedo_wy2005, 
                                   june_albedo_wy2006, june_albedo_wy2007, june_albedo_wy2008, june_albedo_wy2009, june_albedo_wy2010, 
                                   june_albedo_wy2011, june_albedo_wy2012, june_albedo_wy2013, june_albedo_wy2014, june_albedo_wy2015, 
                                   june_albedo_wy2016, june_albedo_wy2017, june_albedo_wy2018, june_albedo_wy2019])

june_mean_albedo = np.nanmean(june_mean_albedo_array, axis = 0)

june_mean_albedo_stack = array([june_albedo_wy2001, june_albedo_wy2002, june_albedo_wy2003, june_albedo_wy2004, june_albedo_wy2005,
                                   june_albedo_wy2006, june_albedo_wy2007, june_albedo_wy2008, june_albedo_wy2009, june_albedo_wy2010,
                                   june_albedo_wy2011, june_albedo_wy2012, june_albedo_wy2013, june_albedo_wy2014, june_albedo_wy2015,
                                   june_albedo_wy2016, june_albedo_wy2017, june_albedo_wy2018, june_albedo_wy2019, june_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/june_mean_albedo_stack.tif', june_mean_albedo_stack, ReferencingMatrix, Projection)

'stacked_tiffs/june_mean_albedo_stack.tif'

In [195]:
july_mean_albedo_array = np.array([july_albedo_wy2001, july_albedo_wy2002, july_albedo_wy2003, july_albedo_wy2004, july_albedo_wy2005, 
                                   july_albedo_wy2006, july_albedo_wy2007, july_albedo_wy2008, july_albedo_wy2009, july_albedo_wy2010, 
                                   july_albedo_wy2011, july_albedo_wy2012, july_albedo_wy2013, july_albedo_wy2014, july_albedo_wy2015, 
                                   july_albedo_wy2016, july_albedo_wy2017, july_albedo_wy2018, july_albedo_wy2019])

july_mean_albedo = np.nanmean(july_mean_albedo_array, axis = 0)

july_mean_albedo_stack = array([july_albedo_wy2001, july_albedo_wy2002, july_albedo_wy2003, july_albedo_wy2004, july_albedo_wy2005,
                                   july_albedo_wy2006, july_albedo_wy2007, july_albedo_wy2008, july_albedo_wy2009, july_albedo_wy2010,
                                   july_albedo_wy2011, july_albedo_wy2012, july_albedo_wy2013, july_albedo_wy2014, july_albedo_wy2015,
                                   july_albedo_wy2016, july_albedo_wy2017, july_albedo_wy2018, july_albedo_wy2019, july_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/july_mean_albedo_stack.tif', july_mean_albedo_stack, ReferencingMatrix, Projection)

'stacked_tiffs/july_mean_albedo_stack.tif'

In [196]:
august_mean_albedo_array = np.array([august_albedo_wy2001, august_albedo_wy2002, august_albedo_wy2003, august_albedo_wy2004, august_albedo_wy2005, 
                                   august_albedo_wy2006, august_albedo_wy2007, august_albedo_wy2008, august_albedo_wy2009, august_albedo_wy2010, 
                                   august_albedo_wy2011, august_albedo_wy2012, august_albedo_wy2013, august_albedo_wy2014, august_albedo_wy2015, 
                                   august_albedo_wy2016, august_albedo_wy2017, august_albedo_wy2018, august_albedo_wy2019])

august_mean_albedo = np.nanmean(august_mean_albedo_array, axis = 0)

august_mean_albedo_stack = array([august_albedo_wy2001, august_albedo_wy2002, august_albedo_wy2003, august_albedo_wy2004, august_albedo_wy2005,
                                   august_albedo_wy2006, august_albedo_wy2007, august_albedo_wy2008, august_albedo_wy2009, august_albedo_wy2010,
                                   august_albedo_wy2011, august_albedo_wy2012, august_albedo_wy2013, august_albedo_wy2014, august_albedo_wy2015,
                                   august_albedo_wy2016, august_albedo_wy2017, august_albedo_wy2018, august_albedo_wy2019, august_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/august_mean_albedo_stack.tif', august_mean_albedo_stack, ReferencingMatrix, Projection)

'stacked_tiffs/august_mean_albedo_stack.tif'

In [197]:
september_mean_albedo_array = np.array([september_albedo_wy2001, september_albedo_wy2002, september_albedo_wy2003, september_albedo_wy2004, september_albedo_wy2005, 
                                   september_albedo_wy2006, september_albedo_wy2007, september_albedo_wy2008, september_albedo_wy2009, september_albedo_wy2010, 
                                   september_albedo_wy2011, september_albedo_wy2012, september_albedo_wy2013, september_albedo_wy2014, september_albedo_wy2015, 
                                   september_albedo_wy2016, september_albedo_wy2017, september_albedo_wy2018, september_albedo_wy2019])

september_mean_albedo = np.nanmean(september_mean_albedo_array, axis = 0)

september_mean_albedo_stack = array([september_albedo_wy2001, september_albedo_wy2002, september_albedo_wy2003, september_albedo_wy2004, september_albedo_wy2005,
                                   september_albedo_wy2006, september_albedo_wy2007, september_albedo_wy2008, september_albedo_wy2009, september_albedo_wy2010,
                                   september_albedo_wy2011, september_albedo_wy2012, september_albedo_wy2013, september_albedo_wy2014, september_albedo_wy2015,
                                   september_albedo_wy2016, september_albedo_wy2017, september_albedo_wy2018, september_albedo_wy2019, september_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/september_mean_albedo_stack.tif', september_mean_albedo_stack, ReferencingMatrix, Projection)

'stacked_tiffs/september_mean_albedo_stack.tif'

In [198]:
october_mean_albedo_array = np.array([october_albedo_wy2001, october_albedo_wy2002, october_albedo_wy2003, october_albedo_wy2004, october_albedo_wy2005, 
                                   october_albedo_wy2006, october_albedo_wy2007, october_albedo_wy2008, october_albedo_wy2009, october_albedo_wy2010, 
                                   october_albedo_wy2011, october_albedo_wy2012, october_albedo_wy2013, october_albedo_wy2014, october_albedo_wy2015, 
                                   october_albedo_wy2016, october_albedo_wy2017, october_albedo_wy2018, october_albedo_wy2019])

october_mean_albedo = np.nanmean(october_mean_albedo_array, axis = 0)

october_mean_albedo_stack = array([october_albedo_wy2001, october_albedo_wy2002, october_albedo_wy2003, october_albedo_wy2004, october_albedo_wy2005,
                                   october_albedo_wy2006, october_albedo_wy2007, october_albedo_wy2008, october_albedo_wy2009, october_albedo_wy2010,
                                   october_albedo_wy2011, october_albedo_wy2012, october_albedo_wy2013, october_albedo_wy2014, october_albedo_wy2015,
                                   october_albedo_wy2016, october_albedo_wy2017, october_albedo_wy2018, october_albedo_wy2019, october_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/october_mean_albedo_stack.tif', october_mean_albedo_stack, ReferencingMatrix, Projection)

'stacked_tiffs/october_mean_albedo_stack.tif'

In [199]:
november_mean_albedo_array = np.array([november_albedo_wy2001, november_albedo_wy2002, november_albedo_wy2003, november_albedo_wy2004, november_albedo_wy2005, 
                                   november_albedo_wy2006, november_albedo_wy2007, november_albedo_wy2008, november_albedo_wy2009, november_albedo_wy2010, 
                                   november_albedo_wy2011, november_albedo_wy2012, november_albedo_wy2013, november_albedo_wy2014, november_albedo_wy2015, 
                                   november_albedo_wy2016, november_albedo_wy2017, november_albedo_wy2018, november_albedo_wy2019])

november_mean_albedo = np.nanmean(november_mean_albedo_array, axis = 0)

november_mean_albedo_stack = array([november_albedo_wy2001, november_albedo_wy2002, november_albedo_wy2003, november_albedo_wy2004, november_albedo_wy2005,
                                   november_albedo_wy2006, november_albedo_wy2007, november_albedo_wy2008, november_albedo_wy2009, november_albedo_wy2010,
                                   november_albedo_wy2011, november_albedo_wy2012, november_albedo_wy2013, november_albedo_wy2014, november_albedo_wy2015,
                                   november_albedo_wy2016, november_albedo_wy2017, november_albedo_wy2018, november_albedo_wy2019, november_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/november_mean_albedo_stack.tif', november_mean_albedo_stack, ReferencingMatrix, Projection)

'stacked_tiffs/november_mean_albedo_stack.tif'

In [200]:
december_mean_albedo_array = np.array([december_albedo_wy2001, december_albedo_wy2002, december_albedo_wy2003, december_albedo_wy2004, december_albedo_wy2005, 
                                   december_albedo_wy2006, december_albedo_wy2007, december_albedo_wy2008, december_albedo_wy2009, december_albedo_wy2010, 
                                   december_albedo_wy2011, december_albedo_wy2012, december_albedo_wy2013, december_albedo_wy2014, december_albedo_wy2015, 
                                   december_albedo_wy2016, december_albedo_wy2017, december_albedo_wy2018, december_albedo_wy2019])

december_mean_albedo = np.nanmean(december_mean_albedo_array, axis = 0)

december_mean_albedo_stack = array([december_albedo_wy2001, december_albedo_wy2002, december_albedo_wy2003, december_albedo_wy2004, december_albedo_wy2005,
                                   december_albedo_wy2006, december_albedo_wy2007, december_albedo_wy2008, december_albedo_wy2009, december_albedo_wy2010,
                                   december_albedo_wy2011, december_albedo_wy2012, december_albedo_wy2013, december_albedo_wy2014, december_albedo_wy2015,
                                   december_albedo_wy2016, december_albedo_wy2017, december_albedo_wy2018, december_albedo_wy2019, december_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/december_mean_albedo_stack.tif', december_mean_albedo_stack, ReferencingMatrix, Projection)

'stacked_tiffs/december_mean_albedo_stack.tif'

### Anomaly

In [260]:
# january snow cover percent anomaly
wy2001_january_albedo_anomaly = january_albedo_wy2001 - january_mean_albedo
wy2002_january_albedo_anomaly = january_albedo_wy2002 - january_mean_albedo
wy2003_january_albedo_anomaly = january_albedo_wy2003 - january_mean_albedo
wy2004_january_albedo_anomaly = january_albedo_wy2004 - january_mean_albedo
wy2005_january_albedo_anomaly = january_albedo_wy2005 - january_mean_albedo
wy2006_january_albedo_anomaly = january_albedo_wy2006 - january_mean_albedo
wy2007_january_albedo_anomaly = january_albedo_wy2007 - january_mean_albedo
wy2008_january_albedo_anomaly = january_albedo_wy2008 - january_mean_albedo
wy2009_january_albedo_anomaly = january_albedo_wy2009 - january_mean_albedo
wy2010_january_albedo_anomaly = january_albedo_wy2010 - january_mean_albedo
wy2011_january_albedo_anomaly = january_albedo_wy2011 - january_mean_albedo
wy2012_january_albedo_anomaly = january_albedo_wy2012 - january_mean_albedo
wy2013_january_albedo_anomaly = january_albedo_wy2013 - january_mean_albedo
wy2014_january_albedo_anomaly = january_albedo_wy2014 - january_mean_albedo
wy2015_january_albedo_anomaly = january_albedo_wy2015 - january_mean_albedo
wy2016_january_albedo_anomaly = january_albedo_wy2016 - january_mean_albedo
wy2017_january_albedo_anomaly = january_albedo_wy2017 - january_mean_albedo
wy2018_january_albedo_anomaly = january_albedo_wy2018 - january_mean_albedo
wy2019_january_albedo_anomaly = january_albedo_wy2019 - january_mean_albedo

january_albedo_anomaly_stack = array([wy2001_january_albedo_anomaly, wy2002_january_albedo_anomaly, wy2003_january_albedo_anomaly, wy2004_january_albedo_anomaly, wy2005_january_albedo_anomaly,
                                   wy2006_january_albedo_anomaly, wy2007_january_albedo_anomaly, wy2008_january_albedo_anomaly, wy2009_january_albedo_anomaly, wy2010_january_albedo_anomaly, 
                                   wy2011_january_albedo_anomaly, wy2012_january_albedo_anomaly, wy2013_january_albedo_anomaly, wy2014_january_albedo_anomaly, wy2015_january_albedo_anomaly,
                                   wy2016_january_albedo_anomaly, wy2017_january_albedo_anomaly, wy2018_january_albedo_anomaly, wy2019_january_albedo_anomaly, january_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/january_albedo_anomaly_stack.tif', january_albedo_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/january_albedo_anomaly_stack.tif'

In [261]:
np.nanmin(january_albedo_anomaly_stack)

-0.2415157489779881

In [262]:
np.nanmax(january_albedo_anomaly_stack)

0.85746

In [243]:
# february snow cover percent anomaly
wy2001_february_albedo_anomaly = february_albedo_wy2001 - february_mean_albedo
wy2002_february_albedo_anomaly = february_albedo_wy2002 - february_mean_albedo
wy2003_february_albedo_anomaly = february_albedo_wy2003 - february_mean_albedo
wy2004_february_albedo_anomaly = february_albedo_wy2004 - february_mean_albedo
wy2005_february_albedo_anomaly = february_albedo_wy2005 - february_mean_albedo
wy2006_february_albedo_anomaly = february_albedo_wy2006 - february_mean_albedo
wy2007_february_albedo_anomaly = february_albedo_wy2007 - february_mean_albedo
wy2008_february_albedo_anomaly = february_albedo_wy2008 - february_mean_albedo
wy2009_february_albedo_anomaly = february_albedo_wy2009 - february_mean_albedo
wy2010_february_albedo_anomaly = february_albedo_wy2010 - february_mean_albedo
wy2011_february_albedo_anomaly = february_albedo_wy2011 - february_mean_albedo
wy2012_february_albedo_anomaly = february_albedo_wy2012 - february_mean_albedo
wy2013_february_albedo_anomaly = february_albedo_wy2013 - february_mean_albedo
wy2014_february_albedo_anomaly = february_albedo_wy2014 - february_mean_albedo
wy2015_february_albedo_anomaly = february_albedo_wy2015 - february_mean_albedo
wy2016_february_albedo_anomaly = february_albedo_wy2016 - february_mean_albedo
wy2017_february_albedo_anomaly = february_albedo_wy2017 - february_mean_albedo
wy2018_february_albedo_anomaly = february_albedo_wy2018 - february_mean_albedo
wy2019_february_albedo_anomaly = february_albedo_wy2019 - february_mean_albedo

february_albedo_anomaly_stack = array([wy2001_february_albedo_anomaly, wy2002_february_albedo_anomaly, wy2003_february_albedo_anomaly, wy2004_february_albedo_anomaly, wy2005_february_albedo_anomaly,
                                   wy2006_february_albedo_anomaly, wy2007_february_albedo_anomaly, wy2008_february_albedo_anomaly, wy2009_february_albedo_anomaly, wy2010_february_albedo_anomaly, 
                                   wy2011_february_albedo_anomaly, wy2012_february_albedo_anomaly, wy2013_february_albedo_anomaly, wy2014_february_albedo_anomaly, wy2015_february_albedo_anomaly,
                                   wy2016_february_albedo_anomaly, wy2017_february_albedo_anomaly, wy2018_february_albedo_anomaly, wy2019_february_albedo_anomaly, february_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/february_albedo_anomaly_stack.tif', february_albedo_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/february_albedo_anomaly_stack.tif'

In [263]:
np.nanmin(february_albedo_anomaly_stack)

-0.28682640961513295

In [264]:
np.nanmax(february_albedo_anomaly_stack)

0.85262

In [244]:
# march snow cover percent anomaly
wy2001_march_albedo_anomaly = march_albedo_wy2001 - march_mean_albedo
wy2002_march_albedo_anomaly = march_albedo_wy2002 - march_mean_albedo
wy2003_march_albedo_anomaly = march_albedo_wy2003 - march_mean_albedo
wy2004_march_albedo_anomaly = march_albedo_wy2004 - march_mean_albedo
wy2005_march_albedo_anomaly = march_albedo_wy2005 - march_mean_albedo
wy2006_march_albedo_anomaly = march_albedo_wy2006 - march_mean_albedo
wy2007_march_albedo_anomaly = march_albedo_wy2007 - march_mean_albedo
wy2008_march_albedo_anomaly = march_albedo_wy2008 - march_mean_albedo
wy2009_march_albedo_anomaly = march_albedo_wy2009 - march_mean_albedo
wy2010_march_albedo_anomaly = march_albedo_wy2010 - march_mean_albedo
wy2011_march_albedo_anomaly = march_albedo_wy2011 - march_mean_albedo
wy2012_march_albedo_anomaly = march_albedo_wy2012 - march_mean_albedo
wy2013_march_albedo_anomaly = march_albedo_wy2013 - march_mean_albedo
wy2014_march_albedo_anomaly = march_albedo_wy2014 - march_mean_albedo
wy2015_march_albedo_anomaly = march_albedo_wy2015 - march_mean_albedo
wy2016_march_albedo_anomaly = march_albedo_wy2016 - march_mean_albedo
wy2017_march_albedo_anomaly = march_albedo_wy2017 - march_mean_albedo
wy2018_march_albedo_anomaly = march_albedo_wy2018 - march_mean_albedo
wy2019_march_albedo_anomaly = march_albedo_wy2019 - march_mean_albedo

march_albedo_anomaly_stack = array([wy2001_march_albedo_anomaly, wy2002_march_albedo_anomaly, wy2003_march_albedo_anomaly, wy2004_march_albedo_anomaly, wy2005_march_albedo_anomaly,
                                   wy2006_march_albedo_anomaly, wy2007_march_albedo_anomaly, wy2008_march_albedo_anomaly, wy2009_march_albedo_anomaly, wy2010_march_albedo_anomaly, 
                                   wy2011_march_albedo_anomaly, wy2012_march_albedo_anomaly, wy2013_march_albedo_anomaly, wy2014_march_albedo_anomaly, wy2015_march_albedo_anomaly,
                                   wy2016_march_albedo_anomaly, wy2017_march_albedo_anomaly, wy2018_march_albedo_anomaly, wy2019_march_albedo_anomaly, march_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/march_albedo_anomaly_stack.tif', march_albedo_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/march_albedo_anomaly_stack.tif'

In [245]:
# april snow cover percent anomaly
wy2001_april_albedo_anomaly = april_albedo_wy2001 - april_mean_albedo
wy2002_april_albedo_anomaly = april_albedo_wy2002 - april_mean_albedo
wy2003_april_albedo_anomaly = april_albedo_wy2003 - april_mean_albedo
wy2004_april_albedo_anomaly = april_albedo_wy2004 - april_mean_albedo
wy2005_april_albedo_anomaly = april_albedo_wy2005 - april_mean_albedo
wy2006_april_albedo_anomaly = april_albedo_wy2006 - april_mean_albedo
wy2007_april_albedo_anomaly = april_albedo_wy2007 - april_mean_albedo
wy2008_april_albedo_anomaly = april_albedo_wy2008 - april_mean_albedo
wy2009_april_albedo_anomaly = april_albedo_wy2009 - april_mean_albedo
wy2010_april_albedo_anomaly = april_albedo_wy2010 - april_mean_albedo
wy2011_april_albedo_anomaly = april_albedo_wy2011 - april_mean_albedo
wy2012_april_albedo_anomaly = april_albedo_wy2012 - april_mean_albedo
wy2013_april_albedo_anomaly = april_albedo_wy2013 - april_mean_albedo
wy2014_april_albedo_anomaly = april_albedo_wy2014 - april_mean_albedo
wy2015_april_albedo_anomaly = april_albedo_wy2015 - april_mean_albedo
wy2016_april_albedo_anomaly = april_albedo_wy2016 - april_mean_albedo
wy2017_april_albedo_anomaly = april_albedo_wy2017 - april_mean_albedo
wy2018_april_albedo_anomaly = april_albedo_wy2018 - april_mean_albedo
wy2019_april_albedo_anomaly = april_albedo_wy2019 - april_mean_albedo

april_albedo_anomaly_stack = array([wy2001_april_albedo_anomaly, wy2002_april_albedo_anomaly, wy2003_april_albedo_anomaly, wy2004_april_albedo_anomaly, wy2005_april_albedo_anomaly,
                                   wy2006_april_albedo_anomaly, wy2007_april_albedo_anomaly, wy2008_april_albedo_anomaly, wy2009_april_albedo_anomaly, wy2010_april_albedo_anomaly, 
                                   wy2011_april_albedo_anomaly, wy2012_april_albedo_anomaly, wy2013_april_albedo_anomaly, wy2014_april_albedo_anomaly, wy2015_april_albedo_anomaly,
                                   wy2016_april_albedo_anomaly, wy2017_april_albedo_anomaly, wy2018_april_albedo_anomaly, wy2019_april_albedo_anomaly, april_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/april_albedo_anomaly_stack.tif', april_albedo_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/april_albedo_anomaly_stack.tif'

In [246]:
# may snow cover percent anomaly
wy2001_may_albedo_anomaly = may_albedo_wy2001 - may_mean_albedo
wy2002_may_albedo_anomaly = may_albedo_wy2002 - may_mean_albedo
wy2003_may_albedo_anomaly = may_albedo_wy2003 - may_mean_albedo
wy2004_may_albedo_anomaly = may_albedo_wy2004 - may_mean_albedo
wy2005_may_albedo_anomaly = may_albedo_wy2005 - may_mean_albedo
wy2006_may_albedo_anomaly = may_albedo_wy2006 - may_mean_albedo
wy2007_may_albedo_anomaly = may_albedo_wy2007 - may_mean_albedo
wy2008_may_albedo_anomaly = may_albedo_wy2008 - may_mean_albedo
wy2009_may_albedo_anomaly = may_albedo_wy2009 - may_mean_albedo
wy2010_may_albedo_anomaly = may_albedo_wy2010 - may_mean_albedo
wy2011_may_albedo_anomaly = may_albedo_wy2011 - may_mean_albedo
wy2012_may_albedo_anomaly = may_albedo_wy2012 - may_mean_albedo
wy2013_may_albedo_anomaly = may_albedo_wy2013 - may_mean_albedo
wy2014_may_albedo_anomaly = may_albedo_wy2014 - may_mean_albedo
wy2015_may_albedo_anomaly = may_albedo_wy2015 - may_mean_albedo
wy2016_may_albedo_anomaly = may_albedo_wy2016 - may_mean_albedo
wy2017_may_albedo_anomaly = may_albedo_wy2017 - may_mean_albedo
wy2018_may_albedo_anomaly = may_albedo_wy2018 - may_mean_albedo
wy2019_may_albedo_anomaly = may_albedo_wy2019 - may_mean_albedo

may_albedo_anomaly_stack = array([wy2001_may_albedo_anomaly, wy2002_may_albedo_anomaly, wy2003_may_albedo_anomaly, wy2004_may_albedo_anomaly, wy2005_may_albedo_anomaly,
                                   wy2006_may_albedo_anomaly, wy2007_may_albedo_anomaly, wy2008_may_albedo_anomaly, wy2009_may_albedo_anomaly, wy2010_may_albedo_anomaly, 
                                   wy2011_may_albedo_anomaly, wy2012_may_albedo_anomaly, wy2013_may_albedo_anomaly, wy2014_may_albedo_anomaly, wy2015_may_albedo_anomaly,
                                   wy2016_may_albedo_anomaly, wy2017_may_albedo_anomaly, wy2018_may_albedo_anomaly, wy2019_may_albedo_anomaly, may_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/may_albedo_anomaly_stack.tif', may_albedo_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/may_albedo_anomaly_stack.tif'

In [247]:
# june snow cover percent anomaly
wy2001_june_albedo_anomaly = june_albedo_wy2001 - june_mean_albedo
wy2002_june_albedo_anomaly = june_albedo_wy2002 - june_mean_albedo
wy2003_june_albedo_anomaly = june_albedo_wy2003 - june_mean_albedo
wy2004_june_albedo_anomaly = june_albedo_wy2004 - june_mean_albedo
wy2005_june_albedo_anomaly = june_albedo_wy2005 - june_mean_albedo
wy2006_june_albedo_anomaly = june_albedo_wy2006 - june_mean_albedo
wy2007_june_albedo_anomaly = june_albedo_wy2007 - june_mean_albedo
wy2008_june_albedo_anomaly = june_albedo_wy2008 - june_mean_albedo
wy2009_june_albedo_anomaly = june_albedo_wy2009 - june_mean_albedo
wy2010_june_albedo_anomaly = june_albedo_wy2010 - june_mean_albedo
wy2011_june_albedo_anomaly = june_albedo_wy2011 - june_mean_albedo
wy2012_june_albedo_anomaly = june_albedo_wy2012 - june_mean_albedo
wy2013_june_albedo_anomaly = june_albedo_wy2013 - june_mean_albedo
wy2014_june_albedo_anomaly = june_albedo_wy2014 - june_mean_albedo
wy2015_june_albedo_anomaly = june_albedo_wy2015 - june_mean_albedo
wy2016_june_albedo_anomaly = june_albedo_wy2016 - june_mean_albedo
wy2017_june_albedo_anomaly = june_albedo_wy2017 - june_mean_albedo
wy2018_june_albedo_anomaly = june_albedo_wy2018 - june_mean_albedo
wy2019_june_albedo_anomaly = june_albedo_wy2019 - june_mean_albedo

june_albedo_anomaly_stack = array([wy2001_june_albedo_anomaly, wy2002_june_albedo_anomaly, wy2003_june_albedo_anomaly, wy2004_june_albedo_anomaly, wy2005_june_albedo_anomaly,
                                   wy2006_june_albedo_anomaly, wy2007_june_albedo_anomaly, wy2008_june_albedo_anomaly, wy2009_june_albedo_anomaly, wy2010_june_albedo_anomaly, 
                                   wy2011_june_albedo_anomaly, wy2012_june_albedo_anomaly, wy2013_june_albedo_anomaly, wy2014_june_albedo_anomaly, wy2015_june_albedo_anomaly,
                                   wy2016_june_albedo_anomaly, wy2017_june_albedo_anomaly, wy2018_june_albedo_anomaly, wy2019_june_albedo_anomaly, june_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/june_albedo_anomaly_stack.tif', june_albedo_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/june_albedo_anomaly_stack.tif'

In [248]:
# july snow cover percent anomaly
wy2001_july_albedo_anomaly = july_albedo_wy2001 - july_mean_albedo
wy2002_july_albedo_anomaly = july_albedo_wy2002 - july_mean_albedo
wy2003_july_albedo_anomaly = july_albedo_wy2003 - july_mean_albedo
wy2004_july_albedo_anomaly = july_albedo_wy2004 - july_mean_albedo
wy2005_july_albedo_anomaly = july_albedo_wy2005 - july_mean_albedo
wy2006_july_albedo_anomaly = july_albedo_wy2006 - july_mean_albedo
wy2007_july_albedo_anomaly = july_albedo_wy2007 - july_mean_albedo
wy2008_july_albedo_anomaly = july_albedo_wy2008 - july_mean_albedo
wy2009_july_albedo_anomaly = july_albedo_wy2009 - july_mean_albedo
wy2010_july_albedo_anomaly = july_albedo_wy2010 - july_mean_albedo
wy2011_july_albedo_anomaly = july_albedo_wy2011 - july_mean_albedo
wy2012_july_albedo_anomaly = july_albedo_wy2012 - july_mean_albedo
wy2013_july_albedo_anomaly = july_albedo_wy2013 - july_mean_albedo
wy2014_july_albedo_anomaly = july_albedo_wy2014 - july_mean_albedo
wy2015_july_albedo_anomaly = july_albedo_wy2015 - july_mean_albedo
wy2016_july_albedo_anomaly = july_albedo_wy2016 - july_mean_albedo
wy2017_july_albedo_anomaly = july_albedo_wy2017 - july_mean_albedo
wy2018_july_albedo_anomaly = july_albedo_wy2018 - july_mean_albedo
wy2019_july_albedo_anomaly = july_albedo_wy2019 - july_mean_albedo

july_albedo_anomaly_stack = array([wy2001_july_albedo_anomaly, wy2002_july_albedo_anomaly, wy2003_july_albedo_anomaly, wy2004_july_albedo_anomaly, wy2005_july_albedo_anomaly,
                                   wy2006_july_albedo_anomaly, wy2007_july_albedo_anomaly, wy2008_july_albedo_anomaly, wy2009_july_albedo_anomaly, wy2010_july_albedo_anomaly, 
                                   wy2011_july_albedo_anomaly, wy2012_july_albedo_anomaly, wy2013_july_albedo_anomaly, wy2014_july_albedo_anomaly, wy2015_july_albedo_anomaly,
                                   wy2016_july_albedo_anomaly, wy2017_july_albedo_anomaly, wy2018_july_albedo_anomaly, wy2019_july_albedo_anomaly, july_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/july_albedo_anomaly_stack.tif', july_albedo_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/july_albedo_anomaly_stack.tif'

In [249]:
# august snow cover percent anomaly
wy2001_august_albedo_anomaly = august_albedo_wy2001 - august_mean_albedo
wy2002_august_albedo_anomaly = august_albedo_wy2002 - august_mean_albedo
wy2003_august_albedo_anomaly = august_albedo_wy2003 - august_mean_albedo
wy2004_august_albedo_anomaly = august_albedo_wy2004 - august_mean_albedo
wy2005_august_albedo_anomaly = august_albedo_wy2005 - august_mean_albedo
wy2006_august_albedo_anomaly = august_albedo_wy2006 - august_mean_albedo
wy2007_august_albedo_anomaly = august_albedo_wy2007 - august_mean_albedo
wy2008_august_albedo_anomaly = august_albedo_wy2008 - august_mean_albedo
wy2009_august_albedo_anomaly = august_albedo_wy2009 - august_mean_albedo
wy2010_august_albedo_anomaly = august_albedo_wy2010 - august_mean_albedo
wy2011_august_albedo_anomaly = august_albedo_wy2011 - august_mean_albedo
wy2012_august_albedo_anomaly = august_albedo_wy2012 - august_mean_albedo
wy2013_august_albedo_anomaly = august_albedo_wy2013 - august_mean_albedo
wy2014_august_albedo_anomaly = august_albedo_wy2014 - august_mean_albedo
wy2015_august_albedo_anomaly = august_albedo_wy2015 - august_mean_albedo
wy2016_august_albedo_anomaly = august_albedo_wy2016 - august_mean_albedo
wy2017_august_albedo_anomaly = august_albedo_wy2017 - august_mean_albedo
wy2018_august_albedo_anomaly = august_albedo_wy2018 - august_mean_albedo
wy2019_august_albedo_anomaly = august_albedo_wy2019 - august_mean_albedo

august_albedo_anomaly_stack = array([wy2001_august_albedo_anomaly, wy2002_august_albedo_anomaly, wy2003_august_albedo_anomaly, wy2004_august_albedo_anomaly, wy2005_august_albedo_anomaly,
                                   wy2006_august_albedo_anomaly, wy2007_august_albedo_anomaly, wy2008_august_albedo_anomaly, wy2009_august_albedo_anomaly, wy2010_august_albedo_anomaly, 
                                   wy2011_august_albedo_anomaly, wy2012_august_albedo_anomaly, wy2013_august_albedo_anomaly, wy2014_august_albedo_anomaly, wy2015_august_albedo_anomaly,
                                   wy2016_august_albedo_anomaly, wy2017_august_albedo_anomaly, wy2018_august_albedo_anomaly, wy2019_august_albedo_anomaly, august_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/august_albedo_anomaly_stack.tif', august_albedo_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/august_albedo_anomaly_stack.tif'

In [250]:
# september snow cover percent anomaly
wy2001_september_albedo_anomaly = september_albedo_wy2001 - september_mean_albedo
wy2002_september_albedo_anomaly = september_albedo_wy2002 - september_mean_albedo
wy2003_september_albedo_anomaly = september_albedo_wy2003 - september_mean_albedo
wy2004_september_albedo_anomaly = september_albedo_wy2004 - september_mean_albedo
wy2005_september_albedo_anomaly = september_albedo_wy2005 - september_mean_albedo
wy2006_september_albedo_anomaly = september_albedo_wy2006 - september_mean_albedo
wy2007_september_albedo_anomaly = september_albedo_wy2007 - september_mean_albedo
wy2008_september_albedo_anomaly = september_albedo_wy2008 - september_mean_albedo
wy2009_september_albedo_anomaly = september_albedo_wy2009 - september_mean_albedo
wy2010_september_albedo_anomaly = september_albedo_wy2010 - september_mean_albedo
wy2011_september_albedo_anomaly = september_albedo_wy2011 - september_mean_albedo
wy2012_september_albedo_anomaly = september_albedo_wy2012 - september_mean_albedo
wy2013_september_albedo_anomaly = september_albedo_wy2013 - september_mean_albedo
wy2014_september_albedo_anomaly = september_albedo_wy2014 - september_mean_albedo
wy2015_september_albedo_anomaly = september_albedo_wy2015 - september_mean_albedo
wy2016_september_albedo_anomaly = september_albedo_wy2016 - september_mean_albedo
wy2017_september_albedo_anomaly = september_albedo_wy2017 - september_mean_albedo
wy2018_september_albedo_anomaly = september_albedo_wy2018 - september_mean_albedo
wy2019_september_albedo_anomaly = september_albedo_wy2019 - september_mean_albedo

september_albedo_anomaly_stack = array([wy2001_september_albedo_anomaly, wy2002_september_albedo_anomaly, wy2003_september_albedo_anomaly, wy2004_september_albedo_anomaly, wy2005_september_albedo_anomaly,
                                   wy2006_september_albedo_anomaly, wy2007_september_albedo_anomaly, wy2008_september_albedo_anomaly, wy2009_september_albedo_anomaly, wy2010_september_albedo_anomaly, 
                                   wy2011_september_albedo_anomaly, wy2012_september_albedo_anomaly, wy2013_september_albedo_anomaly, wy2014_september_albedo_anomaly, wy2015_september_albedo_anomaly,
                                   wy2016_september_albedo_anomaly, wy2017_september_albedo_anomaly, wy2018_september_albedo_anomaly, wy2019_september_albedo_anomaly, september_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/september_albedo_anomaly_stack.tif', september_albedo_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/september_albedo_anomaly_stack.tif'

In [251]:
# october snow cover percent anomaly
wy2001_october_albedo_anomaly = october_albedo_wy2001 - october_mean_albedo
wy2002_october_albedo_anomaly = october_albedo_wy2002 - october_mean_albedo
wy2003_october_albedo_anomaly = october_albedo_wy2003 - october_mean_albedo
wy2004_october_albedo_anomaly = october_albedo_wy2004 - october_mean_albedo
wy2005_october_albedo_anomaly = october_albedo_wy2005 - october_mean_albedo
wy2006_october_albedo_anomaly = october_albedo_wy2006 - october_mean_albedo
wy2007_october_albedo_anomaly = october_albedo_wy2007 - october_mean_albedo
wy2008_october_albedo_anomaly = october_albedo_wy2008 - october_mean_albedo
wy2009_october_albedo_anomaly = october_albedo_wy2009 - october_mean_albedo
wy2010_october_albedo_anomaly = october_albedo_wy2010 - october_mean_albedo
wy2011_october_albedo_anomaly = october_albedo_wy2011 - october_mean_albedo
wy2012_october_albedo_anomaly = october_albedo_wy2012 - october_mean_albedo
wy2013_october_albedo_anomaly = october_albedo_wy2013 - october_mean_albedo
wy2014_october_albedo_anomaly = october_albedo_wy2014 - october_mean_albedo
wy2015_october_albedo_anomaly = october_albedo_wy2015 - october_mean_albedo
wy2016_october_albedo_anomaly = october_albedo_wy2016 - october_mean_albedo
wy2017_october_albedo_anomaly = october_albedo_wy2017 - october_mean_albedo
wy2018_october_albedo_anomaly = october_albedo_wy2018 - october_mean_albedo
wy2019_october_albedo_anomaly = october_albedo_wy2019 - october_mean_albedo

october_albedo_anomaly_stack = array([wy2001_october_albedo_anomaly, wy2002_october_albedo_anomaly, wy2003_october_albedo_anomaly, wy2004_october_albedo_anomaly, wy2005_october_albedo_anomaly,
                                   wy2006_october_albedo_anomaly, wy2007_october_albedo_anomaly, wy2008_october_albedo_anomaly, wy2009_october_albedo_anomaly, wy2010_october_albedo_anomaly, 
                                   wy2011_october_albedo_anomaly, wy2012_october_albedo_anomaly, wy2013_october_albedo_anomaly, wy2014_october_albedo_anomaly, wy2015_october_albedo_anomaly,
                                   wy2016_october_albedo_anomaly, wy2017_october_albedo_anomaly, wy2018_october_albedo_anomaly, wy2019_october_albedo_anomaly, october_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/october_albedo_anomaly_stack.tif', october_albedo_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/october_albedo_anomaly_stack.tif'

In [252]:
# november snow cover percent anomaly
wy2001_november_albedo_anomaly = november_albedo_wy2001 - november_mean_albedo
wy2002_november_albedo_anomaly = november_albedo_wy2002 - november_mean_albedo
wy2003_november_albedo_anomaly = november_albedo_wy2003 - november_mean_albedo
wy2004_november_albedo_anomaly = november_albedo_wy2004 - november_mean_albedo
wy2005_november_albedo_anomaly = november_albedo_wy2005 - november_mean_albedo
wy2006_november_albedo_anomaly = november_albedo_wy2006 - november_mean_albedo
wy2007_november_albedo_anomaly = november_albedo_wy2007 - november_mean_albedo
wy2008_november_albedo_anomaly = november_albedo_wy2008 - november_mean_albedo
wy2009_november_albedo_anomaly = november_albedo_wy2009 - november_mean_albedo
wy2010_november_albedo_anomaly = november_albedo_wy2010 - november_mean_albedo
wy2011_november_albedo_anomaly = november_albedo_wy2011 - november_mean_albedo
wy2012_november_albedo_anomaly = november_albedo_wy2012 - november_mean_albedo
wy2013_november_albedo_anomaly = november_albedo_wy2013 - november_mean_albedo
wy2014_november_albedo_anomaly = november_albedo_wy2014 - november_mean_albedo
wy2015_november_albedo_anomaly = november_albedo_wy2015 - november_mean_albedo
wy2016_november_albedo_anomaly = november_albedo_wy2016 - november_mean_albedo
wy2017_november_albedo_anomaly = november_albedo_wy2017 - november_mean_albedo
wy2018_november_albedo_anomaly = november_albedo_wy2018 - november_mean_albedo
wy2019_november_albedo_anomaly = november_albedo_wy2019 - november_mean_albedo

november_albedo_anomaly_stack = array([wy2001_november_albedo_anomaly, wy2002_november_albedo_anomaly, wy2003_november_albedo_anomaly, wy2004_november_albedo_anomaly, wy2005_november_albedo_anomaly,
                                   wy2006_november_albedo_anomaly, wy2007_november_albedo_anomaly, wy2008_november_albedo_anomaly, wy2009_november_albedo_anomaly, wy2010_november_albedo_anomaly, 
                                   wy2011_november_albedo_anomaly, wy2012_november_albedo_anomaly, wy2013_november_albedo_anomaly, wy2014_november_albedo_anomaly, wy2015_november_albedo_anomaly,
                                   wy2016_november_albedo_anomaly, wy2017_november_albedo_anomaly, wy2018_november_albedo_anomaly, wy2019_november_albedo_anomaly, november_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/november_albedo_anomaly_stack.tif', november_albedo_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/november_albedo_anomaly_stack.tif'

In [253]:
# december snow cover percent anomaly
wy2001_december_albedo_anomaly = december_albedo_wy2001 - december_mean_albedo
wy2002_december_albedo_anomaly = december_albedo_wy2002 - december_mean_albedo
wy2003_december_albedo_anomaly = december_albedo_wy2003 - december_mean_albedo
wy2004_december_albedo_anomaly = december_albedo_wy2004 - december_mean_albedo
wy2005_december_albedo_anomaly = december_albedo_wy2005 - december_mean_albedo
wy2006_december_albedo_anomaly = december_albedo_wy2006 - december_mean_albedo
wy2007_december_albedo_anomaly = december_albedo_wy2007 - december_mean_albedo
wy2008_december_albedo_anomaly = december_albedo_wy2008 - december_mean_albedo
wy2009_december_albedo_anomaly = december_albedo_wy2009 - december_mean_albedo
wy2010_december_albedo_anomaly = december_albedo_wy2010 - december_mean_albedo
wy2011_december_albedo_anomaly = december_albedo_wy2011 - december_mean_albedo
wy2012_december_albedo_anomaly = december_albedo_wy2012 - december_mean_albedo
wy2013_december_albedo_anomaly = december_albedo_wy2013 - december_mean_albedo
wy2014_december_albedo_anomaly = december_albedo_wy2014 - december_mean_albedo
wy2015_december_albedo_anomaly = december_albedo_wy2015 - december_mean_albedo
wy2016_december_albedo_anomaly = december_albedo_wy2016 - december_mean_albedo
wy2017_december_albedo_anomaly = december_albedo_wy2017 - december_mean_albedo
wy2018_december_albedo_anomaly = december_albedo_wy2018 - december_mean_albedo
wy2019_december_albedo_anomaly = december_albedo_wy2019 - december_mean_albedo

december_albedo_anomaly_stack = array([wy2001_december_albedo_anomaly, wy2002_december_albedo_anomaly, wy2003_december_albedo_anomaly, wy2004_december_albedo_anomaly, wy2005_december_albedo_anomaly,
                                   wy2006_december_albedo_anomaly, wy2007_december_albedo_anomaly, wy2008_december_albedo_anomaly, wy2009_december_albedo_anomaly, wy2010_december_albedo_anomaly, 
                                   wy2011_december_albedo_anomaly, wy2012_december_albedo_anomaly, wy2013_december_albedo_anomaly, wy2014_december_albedo_anomaly, wy2015_december_albedo_anomaly,
                                   wy2016_december_albedo_anomaly, wy2017_december_albedo_anomaly, wy2018_december_albedo_anomaly, wy2019_december_albedo_anomaly, december_mean_albedo])

CreateGeoTiffStack('stacked_tiffs/december_albedo_anomaly_stack.tif', december_albedo_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/december_albedo_anomaly_stack.tif'

In [140]:
# wy2001_albedo_anomaly_stack = array([wy2001_january_albedo_anomaly, wy2001_february_albedo_anomaly, wy2001_march_albedo_anomaly, wy2001_april_albedo_anomaly,
#                                     wy2001_may_albedo_anomaly, wy2001_june_albedo_anomaly, wy2001_july_albedo_anomaly, wy2001_august_albedo_anomaly, 
#                                      wy2001_september_albedo_anomaly, wy2001_october_albedo_anomaly, wy2001_november_albedo_anomaly, wy2001_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2001_albedo_anomaly_stack.tif', wy2001_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [141]:
# wy2002_albedo_anomaly_stack = array([wy2002_january_albedo_anomaly, wy2002_february_albedo_anomaly, wy2002_march_albedo_anomaly, wy2002_april_albedo_anomaly,
#                                     wy2002_may_albedo_anomaly, wy2002_june_albedo_anomaly, wy2002_july_albedo_anomaly, wy2002_august_albedo_anomaly, 
#                                      wy2002_september_albedo_anomaly, wy2002_october_albedo_anomaly, wy2002_november_albedo_anomaly, wy2002_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2002_albedo_anomaly_stack.tif', wy2002_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [142]:
# wy2003_albedo_anomaly_stack = array([wy2003_january_albedo_anomaly, wy2003_february_albedo_anomaly, wy2003_march_albedo_anomaly, wy2003_april_albedo_anomaly,
#                                     wy2003_may_albedo_anomaly, wy2003_june_albedo_anomaly, wy2003_july_albedo_anomaly, wy2003_august_albedo_anomaly, 
#                                      wy2003_september_albedo_anomaly, wy2003_october_albedo_anomaly, wy2003_november_albedo_anomaly, wy2003_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2003_albedo_anomaly_stack.tif', wy2003_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [143]:
# wy2004_albedo_anomaly_stack = array([wy2004_january_albedo_anomaly, wy2004_february_albedo_anomaly, wy2004_march_albedo_anomaly, wy2004_april_albedo_anomaly,
#                                     wy2004_may_albedo_anomaly, wy2004_june_albedo_anomaly, wy2004_july_albedo_anomaly, wy2004_august_albedo_anomaly, 
#                                      wy2004_september_albedo_anomaly, wy2004_october_albedo_anomaly, wy2004_november_albedo_anomaly, wy2004_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2004_albedo_anomaly_stack.tif', wy2004_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [144]:
# wy2005_albedo_anomaly_stack = array([wy2005_january_albedo_anomaly, wy2005_february_albedo_anomaly, wy2005_march_albedo_anomaly, wy2005_april_albedo_anomaly,
#                                     wy2005_may_albedo_anomaly, wy2005_june_albedo_anomaly, wy2005_july_albedo_anomaly, wy2005_august_albedo_anomaly, 
#                                      wy2005_september_albedo_anomaly, wy2005_october_albedo_anomaly, wy2005_november_albedo_anomaly, wy2005_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2005_albedo_anomaly_stack.tif', wy2005_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [145]:
# wy2006_albedo_anomaly_stack = array([wy2006_january_albedo_anomaly, wy2006_february_albedo_anomaly, wy2006_march_albedo_anomaly, wy2006_april_albedo_anomaly,
#                                     wy2006_may_albedo_anomaly, wy2006_june_albedo_anomaly, wy2006_july_albedo_anomaly, wy2006_august_albedo_anomaly, 
#                                      wy2006_september_albedo_anomaly, wy2006_october_albedo_anomaly, wy2006_november_albedo_anomaly, wy2006_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2006_albedo_anomaly_stack.tif', wy2006_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [146]:
# wy2007_albedo_anomaly_stack = array([wy2007_january_albedo_anomaly, wy2007_february_albedo_anomaly, wy2007_march_albedo_anomaly, wy2007_april_albedo_anomaly,
#                                     wy2007_may_albedo_anomaly, wy2007_june_albedo_anomaly, wy2007_july_albedo_anomaly, wy2007_august_albedo_anomaly, 
#                                      wy2007_september_albedo_anomaly, wy2007_october_albedo_anomaly, wy2007_november_albedo_anomaly, wy2007_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2007_albedo_anomaly_stack.tif', wy2007_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [147]:
# wy2008_albedo_anomaly_stack = array([wy2008_january_albedo_anomaly, wy2008_february_albedo_anomaly, wy2008_march_albedo_anomaly, wy2008_april_albedo_anomaly,
#                                     wy2008_may_albedo_anomaly, wy2008_june_albedo_anomaly, wy2008_july_albedo_anomaly, wy2008_august_albedo_anomaly, 
#                                      wy2008_september_albedo_anomaly, wy2008_october_albedo_anomaly, wy2008_november_albedo_anomaly, wy2008_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2008_albedo_anomaly_stack.tif', wy2008_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [148]:
# wy2009_albedo_anomaly_stack = array([wy2009_january_albedo_anomaly, wy2009_february_albedo_anomaly, wy2009_march_albedo_anomaly, wy2009_april_albedo_anomaly,
#                                     wy2009_may_albedo_anomaly, wy2009_june_albedo_anomaly, wy2009_july_albedo_anomaly, wy2009_august_albedo_anomaly, 
#                                      wy2009_september_albedo_anomaly, wy2009_october_albedo_anomaly, wy2009_november_albedo_anomaly, wy2009_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2009_albedo_anomaly_stack.tif', wy2009_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [149]:
# wy2010_albedo_anomaly_stack = array([wy2010_january_albedo_anomaly, wy2010_february_albedo_anomaly, wy2010_march_albedo_anomaly, wy2010_april_albedo_anomaly,
#                                     wy2010_may_albedo_anomaly, wy2010_june_albedo_anomaly, wy2010_july_albedo_anomaly, wy2010_august_albedo_anomaly, 
#                                      wy2010_september_albedo_anomaly, wy2010_october_albedo_anomaly, wy2010_november_albedo_anomaly, wy2010_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2010_albedo_anomaly_stack.tif', wy2010_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [150]:
# wy2011_albedo_anomaly_stack = array([wy2011_january_albedo_anomaly, wy2011_february_albedo_anomaly, wy2011_march_albedo_anomaly, wy2011_april_albedo_anomaly,
#                                     wy2011_may_albedo_anomaly, wy2011_june_albedo_anomaly, wy2011_july_albedo_anomaly, wy2011_august_albedo_anomaly, 
#                                      wy2011_september_albedo_anomaly, wy2011_october_albedo_anomaly, wy2011_november_albedo_anomaly, wy2011_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2011_albedo_anomaly_stack.tif', wy2011_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [151]:
# wy2012_albedo_anomaly_stack = array([wy2012_january_albedo_anomaly, wy2012_february_albedo_anomaly, wy2012_march_albedo_anomaly, wy2012_april_albedo_anomaly,
#                                     wy2012_may_albedo_anomaly, wy2012_june_albedo_anomaly, wy2012_july_albedo_anomaly, wy2012_august_albedo_anomaly, 
#                                      wy2012_september_albedo_anomaly, wy2012_october_albedo_anomaly, wy2012_november_albedo_anomaly, wy2012_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2012_albedo_anomaly_stack.tif', wy2012_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [152]:
# wy2013_albedo_anomaly_stack = array([wy2013_january_albedo_anomaly, wy2013_february_albedo_anomaly, wy2013_march_albedo_anomaly, wy2013_april_albedo_anomaly,
#                                     wy2013_may_albedo_anomaly, wy2013_june_albedo_anomaly, wy2013_july_albedo_anomaly, wy2013_august_albedo_anomaly, 
#                                      wy2013_september_albedo_anomaly, wy2013_october_albedo_anomaly, wy2013_november_albedo_anomaly, wy2013_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2013_albedo_anomaly_stack.tif', wy2013_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [153]:
# wy2014_albedo_anomaly_stack = array([wy2014_january_albedo_anomaly, wy2014_february_albedo_anomaly, wy2014_march_albedo_anomaly, wy2014_april_albedo_anomaly,
#                                     wy2014_may_albedo_anomaly, wy2014_june_albedo_anomaly, wy2014_july_albedo_anomaly, wy2014_august_albedo_anomaly, 
#                                      wy2014_september_albedo_anomaly, wy2014_october_albedo_anomaly, wy2014_november_albedo_anomaly, wy2014_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2014_albedo_anomaly_stack.tif', wy2014_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [154]:
# wy2015_albedo_anomaly_stack = array([wy2015_january_albedo_anomaly, wy2015_february_albedo_anomaly, wy2015_march_albedo_anomaly, wy2015_april_albedo_anomaly,
#                                     wy2015_may_albedo_anomaly, wy2015_june_albedo_anomaly, wy2015_july_albedo_anomaly, wy2015_august_albedo_anomaly, 
#                                      wy2015_september_albedo_anomaly, wy2015_october_albedo_anomaly, wy2015_november_albedo_anomaly, wy2015_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2015_albedo_anomaly_stack.tif', wy2015_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [155]:
# wy2016_albedo_anomaly_stack = array([wy2016_january_albedo_anomaly, wy2016_february_albedo_anomaly, wy2016_march_albedo_anomaly, wy2016_april_albedo_anomaly,
#                                     wy2016_may_albedo_anomaly, wy2016_june_albedo_anomaly, wy2016_july_albedo_anomaly, wy2016_august_albedo_anomaly, 
#                                      wy2016_september_albedo_anomaly, wy2016_october_albedo_anomaly, wy2016_november_albedo_anomaly, wy2016_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2016_albedo_anomaly_stack.tif', wy2016_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [156]:
# wy2017_albedo_anomaly_stack = array([wy2017_january_albedo_anomaly, wy2017_february_albedo_anomaly, wy2017_march_albedo_anomaly, wy2017_april_albedo_anomaly,
#                                     wy2017_may_albedo_anomaly, wy2017_june_albedo_anomaly, wy2017_july_albedo_anomaly, wy2017_august_albedo_anomaly, 
#                                      wy2017_september_albedo_anomaly, wy2017_october_albedo_anomaly, wy2017_november_albedo_anomaly, wy2017_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2017_albedo_anomaly_stack.tif', wy2017_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [157]:
# wy2018_albedo_anomaly_stack = array([wy2018_january_albedo_anomaly, wy2018_february_albedo_anomaly, wy2018_march_albedo_anomaly, wy2018_april_albedo_anomaly,
#                                     wy2018_may_albedo_anomaly, wy2018_june_albedo_anomaly, wy2018_july_albedo_anomaly, wy2018_august_albedo_anomaly, 
#                                      wy2018_september_albedo_anomaly, wy2018_october_albedo_anomaly, wy2018_november_albedo_anomaly, wy2018_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2018_albedo_anomaly_stack.tif', wy2018_albedo_anomaly_stack, ReferencingMatrix, Projection)

In [158]:
# wy2019_albedo_anomaly_stack = array([wy2019_january_albedo_anomaly, wy2019_february_albedo_anomaly, wy2019_march_albedo_anomaly, wy2019_april_albedo_anomaly,
#                                     wy2019_may_albedo_anomaly, wy2019_june_albedo_anomaly, wy2019_july_albedo_anomaly, wy2019_august_albedo_anomaly, 
#                                      wy2019_september_albedo_anomaly, wy2019_october_albedo_anomaly, wy2019_november_albedo_anomaly, wy2019_december_albedo_anomaly])

# CreateGeoTiffStack('stacked_tiffs/wy2019_albedo_anomaly_stack.tif', wy2019_albedo_anomaly_stack, ReferencingMatrix, Projection)

'stacked_tiffs/wy2019_albedo_anomaly_stack.tif'

# Project Area Mask

In [159]:
warnings.filterwarnings('ignore')

project_area_mask = rioxarray.open_rasterio('../data/snow_fraction_data/Sierra2001.h5')
project_area_mask_xarray = project_area_mask['Grid_MODIS_GRID_500m_snow_fraction']
project_area_mask_xarray = project_area_mask_xarray.where(project_area_mask_xarray != 255)
project_area_mask_xarray = project_area_mask_xarray.isel(band=slice(0, 364)).mean(dim="band") 
project_area_mask_numpy = project_area_mask_xarray.to_numpy()
project_area_mask_numpy = np.transpose(project_area_mask_numpy)
project_area_mask_numpy[project_area_mask_numpy >= 0] = 999

In [160]:
project_area_mask = array([project_area_mask_numpy, project_area_mask_numpy])

CreateGeoTiffStack('stacked_tiffs/project_area_mask.tif', project_area_mask, ReferencingMatrix, Projection)

'stacked_tiffs/project_area_mask.tif'